In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "49b0d935-5339-443c-9a3a-aaeafc79136a",
   "metadata": {},
   "source": [
    "<h1>Kaggle Predict the Introverts from the Extroverts</h1>\n",
    "<h3>Playground Series - Season 5, Episode 7</h3>"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b41106aa-73ad-4537-ad7e-2c7425c12058",
   "metadata": {},
   "source": [
    "<h2>Import libraries</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1203,
   "id": "5d9c838e-cb3b-4c51-886a-692410174728",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from sklearn.impute import SimpleImputer\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.preprocessing import OneHotEncoder\n",
    "from sklearn.linear_model import SGDClassifier\n",
    "from sklearn.model_selection import cross_val_score\n",
    "from sklearn.dummy import DummyClassifier\n",
    "from sklearn.model_selection import cross_val_predict\n",
    "from sklearn.metrics import confusion_matrix\n",
    "from sklearn.metrics import precision_score, recall_score, f1_score\n",
    "from sklearn.metrics import precision_recall_curve\n",
    "from sklearn.metrics import roc_curve\n",
    "from sklearn.ensemble import RandomForestClassifier"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1204,
   "id": "eb335b71-5dde-4180-85b5-763db925b367",
   "metadata": {},
   "outputs": [],
   "source": [
    "people = pd.read_csv('dataset/train.csv')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "add92846-8d2f-423a-8d6d-338ef3ecce7a",
   "metadata": {},
   "source": [
    "<h2>Some statistics</h2>\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1205,
   "id": "6a1ea704-d299-47c2-8aac-0af6711bb835",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Stage_fear</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Drained_after_socializing</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "      <th>Personality</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>No</td>\n",
       "      <td>6.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>No</td>\n",
       "      <td>15.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>Extrovert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1</td>\n",
       "      <td>1.0</td>\n",
       "      <td>No</td>\n",
       "      <td>7.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>10.0</td>\n",
       "      <td>8.0</td>\n",
       "      <td>Extrovert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>2</td>\n",
       "      <td>6.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>NaN</td>\n",
       "      <td>3.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>Introvert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>3</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>7.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>11.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>Extrovert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>4</td>\n",
       "      <td>1.0</td>\n",
       "      <td>No</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>No</td>\n",
       "      <td>13.0</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Extrovert</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   id  Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \\\n",
       "0   0               0.0         No                      6.0            4.0   \n",
       "1   1               1.0         No                      7.0            3.0   \n",
       "2   2               6.0        Yes                      1.0            0.0   \n",
       "3   3               3.0         No                      7.0            3.0   \n",
       "4   4               1.0         No                      4.0            4.0   \n",
       "\n",
       "  Drained_after_socializing  Friends_circle_size  Post_frequency Personality  \n",
       "0                        No                 15.0             5.0   Extrovert  \n",
       "1                        No                 10.0             8.0   Extrovert  \n",
       "2                       NaN                  3.0             0.0   Introvert  \n",
       "3                        No                 11.0             5.0   Extrovert  \n",
       "4                        No                 13.0             NaN   Extrovert  "
      ]
     },
     "execution_count": 1205,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1206,
   "id": "b6318669-e91f-4d14-ae31-dcc308e3e42d",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "<class 'pandas.core.frame.DataFrame'>\n",
      "RangeIndex: 18524 entries, 0 to 18523\n",
      "Data columns (total 9 columns):\n",
      " #   Column                     Non-Null Count  Dtype  \n",
      "---  ------                     --------------  -----  \n",
      " 0   id                         18524 non-null  int64  \n",
      " 1   Time_spent_Alone           17334 non-null  float64\n",
      " 2   Stage_fear                 16631 non-null  object \n",
      " 3   Social_event_attendance    17344 non-null  float64\n",
      " 4   Going_outside              17058 non-null  float64\n",
      " 5   Drained_after_socializing  17375 non-null  object \n",
      " 6   Friends_circle_size        17470 non-null  float64\n",
      " 7   Post_frequency             17260 non-null  float64\n",
      " 8   Personality                18524 non-null  object \n",
      "dtypes: float64(5), int64(1), object(3)\n",
      "memory usage: 1.3+ MB\n"
     ]
    }
   ],
   "source": [
    "people.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1207,
   "id": "14a44e12-7397-4db4-8e69-ef189134d728",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Stage_fear\n",
       "No     12609\n",
       "Yes     4022\n",
       "Name: count, dtype: int64"
      ]
     },
     "execution_count": 1207,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people['Stage_fear'].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1208,
   "id": "fbc5ec24-80a6-4893-a0b1-82edd23004ed",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Drained_after_socializing\n",
       "No     13313\n",
       "Yes     4062\n",
       "Name: count, dtype: int64"
      ]
     },
     "execution_count": 1208,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people['Drained_after_socializing'].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1209,
   "id": "9818cb41-1b46-4060-aa14-78f463589545",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Time_spent_Alone\n",
       "0.0     3139\n",
       "3.0     3081\n",
       "2.0     3039\n",
       "1.0     2973\n",
       "4.0     1079\n",
       "5.0      633\n",
       "10.0     587\n",
       "8.0      582\n",
       "7.0      581\n",
       "6.0      574\n",
       "9.0      574\n",
       "11.0     492\n",
       "Name: count, dtype: int64"
      ]
     },
     "execution_count": 1209,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people['Time_spent_Alone'].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1210,
   "id": "d72e604a-8492-476c-88ed-c0f40353c004",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Personality\n",
       "Extrovert    13699\n",
       "Introvert     4825\n",
       "Name: count, dtype: int64"
      ]
     },
     "execution_count": 1210,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people['Personality'].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1211,
   "id": "760aad02-7d13-4e0b-b5fa-21f2c067e26c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>count</th>\n",
       "      <td>18524.000000</td>\n",
       "      <td>17334.000000</td>\n",
       "      <td>17344.000000</td>\n",
       "      <td>17058.000000</td>\n",
       "      <td>17470.000000</td>\n",
       "      <td>17260.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>mean</th>\n",
       "      <td>9261.500000</td>\n",
       "      <td>3.137764</td>\n",
       "      <td>5.265106</td>\n",
       "      <td>4.044319</td>\n",
       "      <td>7.996737</td>\n",
       "      <td>4.982097</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>std</th>\n",
       "      <td>5347.562529</td>\n",
       "      <td>3.003786</td>\n",
       "      <td>2.753359</td>\n",
       "      <td>2.062580</td>\n",
       "      <td>4.223484</td>\n",
       "      <td>2.879139</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>min</th>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>25%</th>\n",
       "      <td>4630.750000</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>3.000000</td>\n",
       "      <td>3.000000</td>\n",
       "      <td>5.000000</td>\n",
       "      <td>3.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>50%</th>\n",
       "      <td>9261.500000</td>\n",
       "      <td>2.000000</td>\n",
       "      <td>5.000000</td>\n",
       "      <td>4.000000</td>\n",
       "      <td>8.000000</td>\n",
       "      <td>5.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>75%</th>\n",
       "      <td>13892.250000</td>\n",
       "      <td>4.000000</td>\n",
       "      <td>8.000000</td>\n",
       "      <td>6.000000</td>\n",
       "      <td>12.000000</td>\n",
       "      <td>7.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>max</th>\n",
       "      <td>18523.000000</td>\n",
       "      <td>11.000000</td>\n",
       "      <td>10.000000</td>\n",
       "      <td>7.000000</td>\n",
       "      <td>15.000000</td>\n",
       "      <td>10.000000</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                 id  Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "count  18524.000000      17334.000000             17344.000000   17058.000000   \n",
       "mean    9261.500000          3.137764                 5.265106       4.044319   \n",
       "std     5347.562529          3.003786                 2.753359       2.062580   \n",
       "min        0.000000          0.000000                 0.000000       0.000000   \n",
       "25%     4630.750000          1.000000                 3.000000       3.000000   \n",
       "50%     9261.500000          2.000000                 5.000000       4.000000   \n",
       "75%    13892.250000          4.000000                 8.000000       6.000000   \n",
       "max    18523.000000         11.000000                10.000000       7.000000   \n",
       "\n",
       "       Friends_circle_size  Post_frequency  \n",
       "count         17470.000000    17260.000000  \n",
       "mean              7.996737        4.982097  \n",
       "std               4.223484        2.879139  \n",
       "min               0.000000        0.000000  \n",
       "25%               5.000000        3.000000  \n",
       "50%               8.000000        5.000000  \n",
       "75%              12.000000        7.000000  \n",
       "max              15.000000       10.000000  "
      ]
     },
     "execution_count": 1211,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people.describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1212,
   "id": "1e4e0441-63e2-4920-bb88-04486a4afe9b",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 1400x600 with 6 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(nrows=2, ncols=3,figsize=(14,6))\n",
    "ax1.bar(np.arange(people['Time_spent_Alone'].max() + 1),people['Time_spent_Alone'].value_counts())\n",
    "ax2.bar(np.arange(people['Social_event_attendance'].max() + 1),people['Social_event_attendance'].value_counts())\n",
    "ax3.bar(np.arange(people['Going_outside'].max() + 1),people['Going_outside'].value_counts())\n",
    "ax4.bar(np.arange(people['Friends_circle_size'].max() + 1),people['Friends_circle_size'].value_counts())\n",
    "ax5.bar(np.arange(people['Post_frequency'].max() + 1),people['Post_frequency'].value_counts())\n",
    "ax6.set_visible(False)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1213,
   "id": "7ed08b71-7172-4b82-879a-6015717c5454",
   "metadata": {},
   "outputs": [],
   "source": [
    "corr_matrix = people.loc[:, people.columns != 'id'].corr(numeric_only=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1214,
   "id": "41ec88c0-3845-4e5a-ab2c-fb1b9d77f0ef",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <td>1.000000</td>\n",
       "      <td>-0.628806</td>\n",
       "      <td>-0.640884</td>\n",
       "      <td>-0.598014</td>\n",
       "      <td>-0.611544</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <td>-0.628806</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.585224</td>\n",
       "      <td>0.566675</td>\n",
       "      <td>0.566679</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>Going_outside</th>\n",
       "      <td>-0.640884</td>\n",
       "      <td>0.585224</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.549864</td>\n",
       "      <td>0.579305</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <td>-0.598014</td>\n",
       "      <td>0.566675</td>\n",
       "      <td>0.549864</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.522272</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>Post_frequency</th>\n",
       "      <td>-0.611544</td>\n",
       "      <td>0.566679</td>\n",
       "      <td>0.579305</td>\n",
       "      <td>0.522272</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                         Time_spent_Alone  Social_event_attendance  \\\n",
       "Time_spent_Alone                 1.000000                -0.628806   \n",
       "Social_event_attendance         -0.628806                 1.000000   \n",
       "Going_outside                   -0.640884                 0.585224   \n",
       "Friends_circle_size             -0.598014                 0.566675   \n",
       "Post_frequency                  -0.611544                 0.566679   \n",
       "\n",
       "                         Going_outside  Friends_circle_size  Post_frequency  \n",
       "Time_spent_Alone             -0.640884            -0.598014       -0.611544  \n",
       "Social_event_attendance       0.585224             0.566675        0.566679  \n",
       "Going_outside                 1.000000             0.549864        0.579305  \n",
       "Friends_circle_size           0.549864             1.000000        0.522272  \n",
       "Post_frequency                0.579305             0.522272        1.000000  "
      ]
     },
     "execution_count": 1214,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "corr_matrix"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1215,
   "id": "bae10819-a3cd-4537-b4fc-a42ece79c4a2",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGNCAYAAABXBKWxAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAK+RJREFUeJzt3X9wVfWd//HXJZCECvdayCYQCRDXFWIx2iYVE8X6i4vxR9W2IwydYBVYs4CKqY6NTFdk3Ea7u0xskQD+YlQqbFfxx2wazXdafm2IQkiKoxRry5qoCRFWbwKWhNx7vn9gsl5yEnLvuZ+c5Ob5mPmMcw/nc8/7xCFv3p/P53yOx7IsSwAADLARbgcAABieSEAAAFeQgAAAriABAQBcQQICALiCBAQAcAUJCADgChIQAMAVJCAAgCtIQAAAV5CAACAO7dixQzfddJPS09Pl8Xj06quvnrHP9u3blZOTo+TkZJ177rlat26d0RhJQAAQh44fP66LLrpIa9as6df5hw4d0vXXX69Zs2aprq5ODz30kO655x69/PLLxmL0sBkpAMQ3j8ejrVu36pZbbun1nAcffFCvv/66Dhw40H2sqKhIf/zjH7V7924jcY008q0AAEnSiRMn1NHR4fh7LMuSx+MJO5aUlKSkpCTH3y1Ju3fvlt/vDzs2Z84cPfPMMzp58qRGjRoVk+t8HQkIAAw5ceKEMqeMUXNL0PF3jRkzRseOHQs79vDDD2vlypWOv1uSmpublZaWFnYsLS1NnZ2dOnLkiCZOnBiT63wdCQgADOno6FBzS1CHaqfIOzb6KffWtpAycz5SY2OjvF5v9/FYVT9dTq+wumZoTj8eKyQgADDMO3aEowTU/T1eb1gCiqUJEyaoubk57FhLS4tGjhyp8ePHG7kmCQgADAtaIQUdLPcKWqHYBdOLvLw8vfHGG2HH3nrrLeXm5hqZ/5FYhg0AxoVkOW6ROnbsmOrr61VfXy/p1DLr+vp6NTQ0SJJKSkq0YMGC7vOLior00Ucfqbi4WAcOHNCzzz6rZ555Rvfff39MfgZ2qIAAIA7t3btXV111Vffn4uJiSdLtt9+ujRs3qqmpqTsZSVJmZqYqKip033336cknn1R6erp+9atf6Yc//KGxGHkOCAAMaW1tlc/n06cHJzlehJA+7WMFAgFjc0BuoAICAMOClqWgg3/rO+k7mDEHBABwBRUQABgW7UKCr/ePRyQgADAsJEtBElAPDMEBAFxBBQQAhjEEZ48EBACGsQrOHkNwAABXDIsEtHbtWmVmZio5OVk5OTnauXOn2yEZEc0reIeq0tJSffe739XYsWOVmpqqW265RQcPHnQ7LGPKy8uVnZ3dvRllXl6efve737kd1oAoLS2Vx+PR8uXL3Q4laqEYtHgU9wloy5YtWr58uVasWKG6ujrNmjVLBQUFYVtQxItIX8E7lG3fvl1Lly5VTU2Nqqqq1NnZKb/fr+PHj7sdmhGTJk3SY489pr1792rv3r26+uqrdfPNN+u9995zOzSj9uzZow0bNig7O9vtUBwJfrUKzkmLR3G/Fc/MmTP1ne98R+Xl5d3HsrKydMstt6i0tNTFyMzqzyt448lnn32m1NRUbd++XVdccYXb4QyIcePG6V//9V+1cOFCt0Mx4tixY/rOd76jtWvX6tFHH9XFF1+ssrIyt8OKSNdWPPvfT9VYB1vxtLWFlH1BS9xtxRPXFVBHR4dqa2t7vGbW7/erurrapahgQiAQkHTql3K8CwaD2rx5s44fP668vDy3wzFm6dKluuGGG3Tttde6HQoMietVcEeOHFEwGLR9zezpL17C0GVZloqLi3X55ZdrxowZbodjzLvvvqu8vDydOHFCY8aM0datW3XBBRe4HZYRmzdv1r59+7Rnzx63Q4kJp/M48ToHFNcJqIvda2ZNvWIWA2/ZsmXav3+/du3a5XYoRk2bNk319fX64osv9PLLL+v222/X9u3b4y4JNTY26t5779Vbb72l5ORkt8OJiZA8Cir63zkhB30Hs7hOQCkpKUpISLB9zezpVRGGprvvvluvv/66duzYoUmTJrkdjlGJiYk677zzJEm5ubnas2ePnnjiCa1fv97lyGKrtrZWLS0tysnJ6T4WDAa1Y8cOrVmzRu3t7UpISHAxQsRKXM8BJSYmKicnR1VVVWHHq6qqlJ+f71JUiAXLsrRs2TK98sor+v3vf6/MzEy3QxpwlmWpvb3d7TBi7pprrtG7777b/TbP+vp65ebm6sc//rHq6+uHZPIJWc5bPIrrCkg69RbAwsJC5ebmKi8vTxs2bFBDQ4OKiorcDi3mjh07pg8//LD7c9creMeNG6fJkye7GFnsLV26VL/5zW/02muvaezYsd1Vrs/n0+jRo12OLvYeeughFRQUKCMjQ21tbdq8ebO2bdumyspKt0OLubFjx/aYyzvrrLM0fvz4ITvHF3Q4BOek72AW9wlo7ty5Onr0qFatWqWmpibNmDFDFRUVmjJlituhxdyZXsEbT7qW1V955ZVhx5977jn95Cc/GfiADDt8+LAKCwvV1NQkn8+n7OxsVVZWavbs2W6HBkQt7p8DAgC3dD0HVP3eRI1x8BzQsbaQ8r/VFHfPAcV9BQQAbgtZHoUsB6vgHPQdzOJ6EQIAYPCiAgIAw1iEYI8EBACGBTVCQQcDTsEYxjKYMAQHAHAFFRAAGGY5XIRgsQhh6Gpvb9fKlSvj8qlxO8PpfofTvUrD637j6V675oCctHg0LJ4D6lqLH29r6HsznO53ON2rNLzuNx7utesefrc/U2c5eA7oeFtIBdmHhvTPws6wqIAAAIMPc0AAYFhIHoUc/Hs/FKev5B7wBBQKhfTpp59q7NixA/ZOntbW1rD/xrvhdL/D6V6l4XW/bt2rZVlqa2tTenq6RoyIzSARzwHZG/AE9OmnnyojI2OgLytJrl3XLcPpfofTvUrD637dutfGxsa4f8eU2wY8AY0dO1aS9NG+qfKOGR5TULdOy3Y7hIET/2tawoxITnI7hAFjBYfH/9tO66R2dr7a/bsqFoLWCAUtBw+ixunfqwFPQF3Dbt4xI+R1sCpkKBnpGeV2CAMoPv+i9GaEJ9HtEAaM5Qm5HcKAiuUUwak5IF7JfbrhkQEAAIMOCQgADAt9tRdctC3aFXRr165VZmamkpOTlZOTo507d/Z5/qZNm3TRRRfpG9/4hiZOnKg77rhDR48ejera/UECAgDDuuaAnLRIbdmyRcuXL9eKFStUV1enWbNmqaCgQA0NDbbn79q1SwsWLNDChQv13nvv6be//a327NmjRYsWOb39XpGAACAOrV69WgsXLtSiRYuUlZWlsrIyZWRkdL/O/nQ1NTWaOnWq7rnnHmVmZuryyy/XXXfdpb179xqLkQQEAIaFvhpGc9KkU89Efb31tk9eR0eHamtr5ff7w477/X5VV1fb9snPz9fHH3+siooKWZalw4cP6z//8z91ww03xPaH8TUkIAAwLGh5HDfp1DNRPp+vu5WWltpe78iRIwoGg0pLSws7npaWpubmZts++fn52rRpk+bOnavExERNmDBBZ599tn7961/H9ofxNSQgABgiGhsbFQgEultJSUmf55++lNyyrF6Xl7///vu655579M///M+qra1VZWWlDh06pKKiopjFfzr2ggMAw5y/EfXU83Ver7dfu2GnpKQoISGhR7XT0tLSoyrqUlpaqssuu0wPPPCAJCk7O1tnnXWWZs2apUcffVQTJ06MOv7eUAEBgGEha4TjFonExETl5OSoqqoq7HhVVZXy8/Nt+3z55Zc99r5LSEiQdKpyMoEKCAAMi1UFFIni4mIVFhYqNzdXeXl52rBhgxoaGrqH1EpKSvTJJ5/o+eeflyTddNNNWrx4scrLyzVnzhw1NTVp+fLluuSSS5Senh517H0hAQFAHJo7d66OHj2qVatWqampSTNmzFBFRYWmTJkiSWpqagp7JugnP/mJ2tratGbNGv30pz/V2WefrauvvlqPP/64sRgH/I2oXW8I/PyDc4fNXnBzzvm22yEMnDjdNLE3I5KT3Q5hwFjB4bEXXKd1Un84+duYvH206/fd+n05Gj0m+n/v/+1Yp+76Tm3cvRGVCggADAs52E6nq388is+7AgAMelRAAGCY8/cBxWetQAICAMN4H5C9+EyrAIBBjwoIAAxjCM4eCQgADHP+IGp8JqD4vCsAwKBHBQQAhoUsj0KWg0UIDvoOZiQgADAs5HAILl4fRCUBAYBh0exofXr/eBTVXa1du1aZmZlKTk5WTk6Odu7cGeu4AABxLuIEtGXLFi1fvlwrVqxQXV2dZs2apYKCgrBdVQEA/ycoj+MWjyJOQKtXr9bChQu1aNEiZWVlqaysTBkZGSovLzcRHwAMeQP9QrqhIqK76ujoUG1trfx+f9hxv9+v6upq2z7t7e1qbW0NawAARJSAjhw5omAw2OOd4mlpaT3ePd6ltLRUPp+vu2VkZEQfLQAMQUE5HYaLT1HVdR5P+HikZVk9jnUpKSlRIBDobo2NjdFcEgCGLIbg7EW0DDslJUUJCQk9qp2WlpYeVVGXpKQkJSUlRR8hACAuRZRWExMTlZOTo6qqqrDjVVVVys/Pj2lgABAvujYjddLiUcQPohYXF6uwsFC5ubnKy8vThg0b1NDQoKKiIhPxAcCQZzl8H5AVp8uwI05Ac+fO1dGjR7Vq1So1NTVpxowZqqio0JQpU0zEBwCIU1FtxbNkyRItWbIk1rEAQFzifUD22AsOAAxjN2x78ZlWAQCDHhUQABjGG1HtkYAAwDCG4OyRgADAsJBGOHqpXLy+kC4+7woAMOhRAQGAYUHLo6CDYTQnfQczEhAAGMYckD2G4AAArqACAgDDLIevVLDYCQEAEI2uF8s56R+P4jOtAgAGPSogADAsZDlbSBCyYhjMIEIFBACGufVK7rVr1yozM1PJycnKycnRzp07+zy/vb1dK1as0JQpU5SUlKS///u/17PPPhvVtfuDCggA4tCWLVu0fPlyrV27VpdddpnWr1+vgoICvf/++5o8ebJtn9tuu02HDx/WM888o/POO08tLS3q7Ow0FiMJCAAMCzl8I2o0fVevXq2FCxdq0aJFkqSysjK9+eabKi8vV2lpaY/zKysrtX37dv31r3/VuHHjJElTp06NOub+YAgOAAzr2gnBSZOk1tbWsNbe3m57vY6ODtXW1srv94cd9/v9qq6utu3z+uuvKzc3V7/85S91zjnn6Pzzz9f999+vv/3tb7H9YXwNFRAADBEZGRlhnx9++GGtXLmyx3lHjhxRMBhUWlpa2PG0tDQ1Nzfbfvdf//pX7dq1S8nJydq6dauOHDmiJUuW6H//93+NzQORgADAMCcLCbr6S1JjY6O8Xm/38aSkpD77eTzhQ3eWZfU41n2NUEgej0ebNm2Sz+eTdGoY70c/+pGefPJJjR49Our4e0MCAgDDQnK4F9xXc0BerzcsAfUmJSVFCQkJPaqdlpaWHlVRl4kTJ+qcc87pTj6SlJWVJcuy9PHHH+sf/uEfoo6/N8wBAYBh1leLEKJtVoSLEBITE5WTk6Oqqqqw41VVVcrPz7ftc9lll+nTTz/VsWPHuo998MEHGjFihCZNmhT5TfcDCQgA4lBxcbGefvppPfvsszpw4IDuu+8+NTQ0qKioSJJUUlKiBQsWdJ8/f/58jR8/XnfccYfef/997dixQw888IDuvPNOI8NvkotDcLdOy9ZIzyi3Lj+g3vykzu0QBsyCj65wO4QB1RFiFDvenDzeIc2O7Xe68TqGuXPn6ujRo1q1apWampo0Y8YMVVRUaMqUKZKkpqYmNTQ0dJ8/ZswYVVVV6e6771Zubq7Gjx+v2267TY8++mjUcZ8Jf3sAwLBYLUKI1JIlS7RkyRLbP9u4cWOPY9OnT+8xbGcSQ3AAAFdQAQGAYbwR1R4JCAAMc2MrnqGAITgAgCuogADAMIbg7JGAAMAwEpA9huAAAK6gAgIAw6iA7JGAAMAwEpA9huAAAK6gAgIAwyw5e5bHil0ogwoJCAAMYwjOHgkIAAwjAdljDggA4AoqIAAwjArIHgkIAAwjAdljCA4A4AoqIAAwzLI8shxUMU76DmYkIAAwjPcB2WMIDgDgCiogADCMRQj2SEAAYBhzQPYYggMAuIIKCAAMYwjOXsQV0I4dO3TTTTcpPT1dHo9Hr776qoGwACB+dA3BOWnxKOIEdPz4cV100UVas2aNiXgAAMNExENwBQUFKigoMBELAMQly+EQXLxWQMbngNrb29Xe3t79ubW11fQlAWBQsSRZDt4qF68vpDO+Cq60tFQ+n6+7ZWRkmL4kAAwqXTshOGnxyHgCKikpUSAQ6G6NjY2mLwkAGAKMD8ElJSUpKSnJ9GUAYNDiQVR7PAcEAIaFLI88PAfUQ8QJ6NixY/rwww+7Px86dEj19fUaN26cJk+eHNPgAADxK+IEtHfvXl111VXdn4uLiyVJt99+uzZu3BizwAAgXliWw1VwcboMLuJFCFdeeaUsy+rRSD4AYM+tnRDWrl2rzMxMJScnKycnRzt37uxXv//+7//WyJEjdfHFF0d13f5iM1IAiENbtmzR8uXLtWLFCtXV1WnWrFkqKChQQ0NDn/0CgYAWLFiga665xniMJCAAMMyNCmj16tVauHChFi1apKysLJWVlSkjI0Pl5eV99rvrrrs0f/585eXlRXu7/UYCAgDDunbDdtKkUzvJfL19fZeZr+vo6FBtba38fn/Ycb/fr+rq6l7jfO655/SXv/xFDz/8cOxuvg8kIAAYIjIyMsJ2liktLbU978iRIwoGg0pLSws7npaWpubmZts+f/7zn/Wzn/1MmzZt0siRA/OEDs8BAYBhsVoF19jYKK/X2338TA/5ezzhQ3eWZfU4JknBYFDz58/XI488ovPPPz/6QCNEAgIAw04lICc7IZz6r9frDUtAvUlJSVFCQkKPaqelpaVHVSRJbW1t2rt3r+rq6rRs2TJJUigUkmVZGjlypN566y1dffXVUcffG4bgACDOJCYmKicnR1VVVWHHq6qqlJ+f3+N8r9erd999V/X19d2tqKhI06ZNU319vWbOnGkkTiogADDMjb3giouLVVhYqNzcXOXl5WnDhg1qaGhQUVGRpFMbRX/yySd6/vnnNWLECM2YMSOsf2pqqpKTk3scjyUSEAAYZsnZO32i6Tt37lwdPXpUq1atUlNTk2bMmKGKigpNmTJFktTU1HTGZ4JM81jWwG7y0NraKp/Ppys9t2ikZ9RAXto1b35S53YIA2bBR1e4HcKA6gjxb7h4c/J4h16dvVGBQKBf8y196fp9d+7zDynhG8lRf0/wyxP664JfxCSmwYQ5IACAK/jnGwCY5sYY3BBAAgIA0xwuQlCcvg+IITgAgCuogADAMN4HZI8EBACGufEc0FDAEBwAwBVUQABgmuVxtpAgTisgEhAAGMYckD2G4AAArqACAgDTeBDVFgkIAAxjFZw9EhAADIQ4rWKccC8BWU5r0qFjOO0Q/fyUHW6HMKB+8OFst0MYMN8YedLtEAaE5Qm5HcKwQQUEAIYxBGePBAQAprEIwRbLsAEArqACAgDjPF81J/3jDwkIAExjCM4WQ3AAAFdQAQGAaVRAtkhAAGAau2HbYggOAOAKKiAAMIzXMdgjAQGAacwB2WIIDgDgCiogADCNRQi2SEAAYJjHOtWc9I9HJCAAMI05IFvMAQEAXEEFBACmMQdkiwQEAKYxBGeLITgAgCuogADANCogW1RAAGCaFYMWhbVr1yozM1PJycnKycnRzp07ez33lVde0ezZs/V3f/d38nq9ysvL05tvvhndhfuJBAQAcWjLli1avny5VqxYobq6Os2aNUsFBQVqaGiwPX/Hjh2aPXu2KioqVFtbq6uuuko33XST6urqjMXIEBwAmObCKrjVq1dr4cKFWrRokSSprKxMb775psrLy1VaWtrj/LKysrDPv/jFL/Taa6/pjTfe0Le//e2owj4TKiAAMKxrJwQnTZJaW1vDWnt7u+31Ojo6VFtbK7/fH3bc7/erurq6XzGHQiG1tbVp3Lhxju69LyQgABgiMjIy5PP5uptdJSNJR44cUTAYVFpaWtjxtLQ0NTc39+ta//7v/67jx4/rtttucxx3byIagistLdUrr7yiP/3pTxo9erTy8/P1+OOPa9q0aabiA4ChL0ar4BobG+X1ersPJyUl9dnN4wkfurMsq8cxOy+99JJWrlyp1157TampqZHH208RVUDbt2/X0qVLVVNTo6qqKnV2dsrv9+v48eOm4gMAfMXr9Ya13hJQSkqKEhISelQ7LS0tPaqi023ZskULFy7Uf/zHf+jaa6+NWex2IqqAKisrwz4/99xzSk1NVW1tra644oqYBgYAiE5iYqJycnJUVVWlW2+9tft4VVWVbr755l77vfTSS7rzzjv10ksv6YYbbjAep6NVcIFAQJL6nKRqb28PmyhrbW11ckkAGHI8cvg6hij6FBcXq7CwULm5ucrLy9OGDRvU0NCgoqIiSVJJSYk++eQTPf/885JOJZ8FCxboiSee0KWXXtpdPY0ePVo+ny/64PsQ9SIEy7JUXFysyy+/XDNmzOj1vNLS0rBJs4yMjGgvCQBDU9cybCctQnPnzlVZWZlWrVqliy++WDt27FBFRYWmTJkiSWpqagp7Jmj9+vXq7OzU0qVLNXHixO527733xuzHcLqoK6Bly5Zp//792rVrV5/nlZSUqLi4uPtza2srSQjA8OLSVjxLlizRkiVLbP9s48aNYZ+3bdsW3UUciCoB3X333Xr99de1Y8cOTZo0qc9zk5KSzrhSAwAw/ESUgCzL0t13362tW7dq27ZtyszMNBUXAMQPNiO1FVECWrp0qX7zm9/otdde09ixY7snqXw+n0aPHm0kQAAY6r6+m0G0/eNRRIsQysvLFQgEdOWVV4ZNUm3ZssVUfACAOBXxEBwAIEIMwdliN2wAMI0EZIvNSAEArqACAgDDWIRgjwQEAKa58EK6oYAhOACAK6iAAMA0FiHYIgEBgGHMAdljCA4A4AoqIAAwjSE4WyQgADDN4RAcCQgAEB0qIFvMAQEAXEEFBACmUQHZIgEBgGEsw7bHEBwAwBUkIACAKxiCAwDTmAOyRQUEAHAFFRAAGMYiBHskIAAYCHGaRJxgCA4A4AoqIAAwjUUItkhAAGAYc0D2XEtAI5KTNMKT6NblB1RHaPjk+R98ONvtEAbUK+dVuR3CgLn+4PVuhzAgOtsT3A5h2Bg+vxkBwC0MwdkiAQGAYQzB2SMBAYBpVEC2WIYNAHAFFRAAmEYFZIsEBACGMQdkjyE4AIhTa9euVWZmppKTk5WTk6OdO3f2ef727duVk5Oj5ORknXvuuVq3bp3R+EhAAGCaFYMWoS1btmj58uVasWKF6urqNGvWLBUUFKihocH2/EOHDun666/XrFmzVFdXp4ceekj33HOPXn755cgv3k8kIAAwzYUEtHr1ai1cuFCLFi1SVlaWysrKlJGRofLyctvz161bp8mTJ6usrExZWVlatGiR7rzzTv3bv/1b5BfvJxIQAAwRra2tYa29vd32vI6ODtXW1srv94cd9/v9qq6utu2ze/fuHufPmTNHe/fu1cmTJ2NzA6chAQGAYV2LEJw0ScrIyJDP5+tupaWlttc7cuSIgsGg0tLSwo6npaWpubnZtk9zc7Pt+Z2dnTpy5IjzH4INVsEBgGkxWobd2Ngor9fbfTgpKanPbh6PJ/xrLKvHsTOdb3c8VkhAADBEeL3esATUm5SUFCUkJPSodlpaWnpUOV0mTJhge/7IkSM1fvz46IPuA0NwAGBYrIbg+isxMVE5OTmqqgrfrb2qqkr5+fm2ffLy8nqc/9Zbbyk3N1ejRo2KLIB+IgEBgGkurIIrLi7W008/rWeffVYHDhzQfffdp4aGBhUVFUmSSkpKtGDBgu7zi4qK9NFHH6m4uFgHDhzQs88+q2eeeUb3339/tHd9RgzBAUAcmjt3ro4ePapVq1apqalJM2bMUEVFhaZMmSJJampqCnsmKDMzUxUVFbrvvvv05JNPKj09Xb/61a/0wx/+0FiMJCAAMM2lveCWLFmiJUuW2P7Zxo0bexz73ve+p3379kV3sSiQgADAMM9XzUn/eEQCAgDT2A3bFosQAACuoAICAMN4HYM9EhAAmMYQnC2G4AAArqACAoCBEKdVjBMkIAAwjDkgexENwZWXlys7O7t7Q7y8vDz97ne/MxUbACCORZSAJk2apMcee0x79+7V3r17dfXVV+vmm2/We++9Zyo+ABj6XNgLbiiIaAjupptuCvv8L//yLyovL1dNTY2+9a1vxTQwAIgXDMHZi3oOKBgM6re//a2OHz+uvLy8Xs9rb28Pe21sa2trtJcEAMSRiJdhv/vuuxozZoySkpJUVFSkrVu36oILLuj1/NLS0rBXyGZkZDgKGACGHIbgbEWcgKZNm6b6+nrV1NTon/7pn3T77bfr/fff7/X8kpISBQKB7tbY2OgoYAAYagb6hXRDRcRDcImJiTrvvPMkSbm5udqzZ4+eeOIJrV+/3vb8pKSkM763HADiGjsh2HK8E4JlWWFzPAAA9EdEFdBDDz2kgoICZWRkqK2tTZs3b9a2bdtUWVlpKj4AGPqogGxFlIAOHz6swsJCNTU1yefzKTs7W5WVlZo9e7ap+ABgyGMZtr2IEtAzzzxjKg4AwDDDXnAAYBpDcLZIQABgmMey5LGizyJO+g5mvA8IAOAKKiAAMI0hOFskIAAwjFVw9hiCAwC4ggoIAExjCM4WCQgADGMIzh5DcAAAV1ABAYBpDMHZIgEBgGEMwdkjAQGAaVRAtpgDAgC4ggoIAAZAvA6jOUECAgDTLOtUc9I/DjEEBwDD3Oeff67CwkL5fD75fD4VFhbqiy++6PX8kydP6sEHH9SFF16os846S+np6VqwYIE+/fTTiK5LAgIAw7pWwTlpJs2fP1/19fWqrKxUZWWl6uvrVVhY2Ov5X375pfbt26ef//zn2rdvn1555RV98MEH+v73vx/RdRmCAwDTBvEquAMHDqiyslI1NTWaOXOmJOmpp55SXl6eDh48qGnTpvXo4/P5VFVVFXbs17/+tS655BI1NDRo8uTJ/bo2FRAADBGtra1hrb293fF37t69Wz6frzv5SNKll14qn8+n6urqfn9PIBCQx+PR2Wef3e8+JCAAMMwTct4kKSMjo3uexufzqbS01HFszc3NSk1N7XE8NTVVzc3N/fqOEydO6Gc/+5nmz58vr9fb72szBAcApsVoCK6xsTHsF3xSUlKvXVauXKlHHnmkz6/ds2ePJMnj8fS8pGXZHj/dyZMnNW/ePIVCIa1du/aM538dCQgAhgiv19vvCmPZsmWaN29en+dMnTpV+/fv1+HDh3v82Weffaa0tLQ++588eVK33XabDh06pN///vcRVT+SiwnIClqyuupKxI1vjDzpdggD6vqD17sdwoCpmFbhdggDorUtpG/G+Dvd2AsuJSVFKSkpZzwvLy9PgUBA77zzji655BJJ0ttvv61AIKD8/Pxe+3Ulnz//+c/6wx/+oPHjx0ccI3NAAGBa14OoTpohWVlZuu6667R48WLV1NSopqZGixcv1o033hi2Am769OnaunWrJKmzs1M/+tGPtHfvXm3atEnBYFDNzc1qbm5WR0dHv69NAgKAYW7Tpk268MIL5ff75ff7lZ2drRdeeCHsnIMHDyoQCEiSPv74Y73++uv6+OOPdfHFF2vixIndLZKVc8wBAYBhg/11DOPGjdOLL77Y5znW16qwqVOnhn2OFgkIAEwbxA+iuokEBACGDfYKyC3MAQEAXEEFBACm8ToGWyQgADCMITh7DMEBAFxBBQQAprEKzhYJCAAMYwjOHkNwAABXUAEBgGkh61Rz0j8OkYAAwDTmgGwxBAcAcAUVEAAY5pHDRQgxi2RwIQEBgGnshGCLITgAgCuogADAMJ4DskcCAgDTWAVniwQEAIZ5LEseB/M4TvoOZswBAQBcQQUEAKaFvmpO+schEhAAGMYQnD2G4AAArnCUgEpLS+XxeLR8+fIYhQMAcciKQYtDUQ/B7dmzRxs2bFB2dnYs4wGA+MNOCLaiqoCOHTumH//4x3rqqaf0zW9+M9YxAQCGgagS0NKlS3XDDTfo2muvPeO57e3tam1tDWsAMJx07YTgpMWjiIfgNm/erH379mnPnj39Or+0tFSPPPJIxIEBQNxgCM5WRBVQY2Oj7r33Xr344otKTk7uV5+SkhIFAoHu1tjYGFWgAID4ElEFVFtbq5aWFuXk5HQfCwaD2rFjh9asWaP29nYlJCSE9UlKSlJSUlJsogWAIcgTOtWc9I9HESWga665Ru+++27YsTvuuEPTp0/Xgw8+2CP5AADEEFwvIkpAY8eO1YwZM8KOnXXWWRo/fnyP4wAA9IWteADANF7HYMtxAtq2bVsMwgCA+MVecPaogADANOaAbLEZKQAMc59//rkKCwvl8/nk8/lUWFioL774ot/977rrLnk8HpWVlUV0XRIQAJhm6f/eCRRNM1wAzZ8/X/X19aqsrFRlZaXq6+tVWFjYr76vvvqq3n77baWnp0d8XYbgAMCwwTwHdODAAVVWVqqmpkYzZ86UJD311FPKy8vTwYMHNW3atF77fvLJJ1q2bJnefPNN3XDDDRFfmwoIAIaI0/fVbG9vd/ydu3fvls/n604+knTppZfK5/Opurq6136hUEiFhYV64IEH9K1vfSuqa5OAAMA0S/+3ECGqduprMjIyuudpfD6fSktLHYfW3Nys1NTUHsdTU1PV3Nzca7/HH39cI0eO1D333BP1tRmCAwDTYrQKrrGxUV6vt/twX9ucrVy58owbQXdtKu3xeGwuadkel05ty/bEE09o3759vZ7THyQgABgivF5vWALqy7JlyzRv3rw+z5k6dar279+vw4cP9/izzz77TGlpabb9du7cqZaWFk2ePLn7WDAY1E9/+lOVlZXpf/7nf/oVIwkIAEwLSYq+UDjVP0IpKSlKSUk543l5eXkKBAJ65513dMkll0iS3n77bQUCAeXn59v2KSws7PE+uDlz5qiwsFB33HFHv2MkAQGAYYN5FVxWVpauu+46LV68WOvXr5ck/eM//qNuvPHGsBVw06dPV2lpqW699VaNHz9e48ePD/ueUaNGacKECX2umjsdixAAYJjbtGmTLrzwQvn9fvn9fmVnZ+uFF14IO+fgwYMKBAIxvS4VEACYNsi34hk3bpxefPHFM4TQdwz9nff5OhIQAJg2yBOQWxiCAwC4ggoIAEyjArJFAgIA01xYhj0UkIAAwLDBvAzbTcwBAQBcQQUEAKYxB2SLBAQApoUsyeMgiYTiMwExBAcAcAUVEACYxhCcrQFPQF3bOXRaJwf60q45ebzD7RAGjOWJ0/WivehsT3A7hAHT2jY8/t+2Hjt1n2faeiYyDhOQSEAx0dbWJkna2fnqQF/aPbPdDgBw7ptuBzDA2tra5PP53A4jrg14AkpPT1djY6PGjh3r6E16kWhtbVVGRkaPtwnGq+F0v8PpXqXhdb9u3atlWWpra1N6enosv5QhOBsDnoBGjBihSZMmDfRlJUX2NsF4MJzudzjdqzS87teNe4155ROy5GgYjVVwAADEDqvgAMA0K3SqOekfh4ZFAkpKStLDDz+spKQkt0MZEMPpfofTvUrD637j6l6ZA7LlsWK71hAA8JXW1lb5fD5de06RRo6IPpF2htr1/z5Zp0AgEFdzf8wBAQBcMSyG4ADAVQzB2SIBAYBplhwmoJhFMqgwBAcAcAUVEACYxhCcLRIQAJgWCkly8CxPKD6fA2IIDgDgCiogADCNIThbJCAAMI0EZIshOACAK6iAAMA0XsdgiwQEAIZZVkiWgx2tnfQdzBiCAwC4ggoIAEyzLGfDaHG6CIEEBACmWQ7ngEhAAICohEKShzeino45IACAK0hAAGBa14OoTppBn3/+uQoLC+Xz+eTz+VRYWKgvvvjijP0OHDig73//+/L5fBo7dqwuvfRSNTQ09Pu6JCAAMMwKhRw3k+bPn6/6+npVVlaqsrJS9fX1Kiws7LPPX/7yF11++eWaPn26tm3bpj/+8Y/6+c9/ruTk5H5f12NZcTq7BQAua21tlc/n09XfmKeRnsSov6fT6tDvv9ysQCAgr9cbwwhPVTEXXHCBampqNHPmTElSTU2N8vLy9Kc//UnTpk2z7Tdv3jyNGjVKL7zwQtTXpgICANNiNATX2toa1trb2x2Htnv3bvl8vu7kI0mXXnqpfD6fqqurbfuEQiH913/9l84//3zNmTNHqampmjlzpl599dWIrk0CAgDTQpbzJikjI6N7nsbn86m0tNRxaM3NzUpNTe1xPDU1Vc3NzbZ9WlpadOzYMT322GO67rrr9NZbb+nWW2/VD37wA23fvr3f12YZNgAMEY2NjWFDcElJSb2eu3LlSj3yyCN9ft+ePXskSR6Pp8efWZZle1w6VQFJ0s0336z77rtPknTxxRerurpa69at0/e+972+b+QrJCAAMM2y5OiNqF8NwXm93n7PAS1btkzz5s3r85ypU6dq//79Onz4cI8/++yzz5SWlmbbLyUlRSNHjtQFF1wQdjwrK0u7du3qV3wSCQgAjLNClixP9Ou9olkrlpKSopSUlDOel5eXp0AgoHfeeUeXXHKJJOntt99WIBBQfn6+bZ/ExER997vf1cGDB8OOf/DBB5oyZUq/Y2QOCACGsaysLF133XVavHixampqVFNTo8WLF+vGG28MWwE3ffp0bd26tfvzAw88oC1btuipp57Shx9+qDVr1uiNN97QkiVL+n1tEhAAmGaFnDeDNm3apAsvvFB+v19+v1/Z2dk9llcfPHhQgUCg+/Ott96qdevW6Ze//KUuvPBCPf3003r55Zd1+eWX9/u6PAcEAIZ0PQd0pedWjfSMivp7Oq2T2mZtNfIckJuogAAArmARAgAY1mm1OxpG69TJGEYzeJCAAMCQxMRETZgwQbuaKxx/14QJE5SYGP12PoMRc0AAYNCJEyfU0dHh+HsSExMj2uhzKCABAQBcwSIEAIArSEAAAFeQgAAAriABAQBcQQICALiCBAQAcAUJCADgiv8PPjauseELl9AAAAAASUVORK5CYII=",
      "text/plain": [
       "<Figure size 480x480 with 2 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.matshow(corr_matrix)\n",
    "cb = plt.colorbar()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1216,
   "id": "fe335a9e-5f65-41ed-ae85-00b5ff00ca53",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_labels = people['Personality'].copy()\n",
    "people = people.drop('Personality',axis=1)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1217,
   "id": "05df1ed7-7d78-4d89-afa7-9f4064aca68e",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0        Extrovert\n",
       "1        Extrovert\n",
       "2        Introvert\n",
       "3        Extrovert\n",
       "4        Extrovert\n",
       "           ...    \n",
       "18519    Extrovert\n",
       "18520    Extrovert\n",
       "18521    Introvert\n",
       "18522    Introvert\n",
       "18523    Extrovert\n",
       "Name: Personality, Length: 18524, dtype: object"
      ]
     },
     "execution_count": 1217,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_labels"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "e2622b3c-4403-45c7-bdc9-e76705d0ff6c",
   "metadata": {},
   "source": [
    "<h2>Fill missing values</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1218,
   "id": "8759d0cb-1375-459b-9696-d858fa18cae4",
   "metadata": {},
   "outputs": [],
   "source": [
    "imputer = SimpleImputer(strategy=\"median\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1219,
   "id": "0dc871ed-a471-4296-936e-a9561c7ccb5d",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_num = people.select_dtypes(include=[np.number])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1220,
   "id": "118350c0-996f-41a3-b8a3-91fcb4397d39",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>15.0</td>\n",
       "      <td>5.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1</td>\n",
       "      <td>1.0</td>\n",
       "      <td>7.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>10.0</td>\n",
       "      <td>8.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>2</td>\n",
       "      <td>6.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>3</td>\n",
       "      <td>3.0</td>\n",
       "      <td>7.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>11.0</td>\n",
       "      <td>5.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>4</td>\n",
       "      <td>1.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>13.0</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   id  Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "0   0               0.0                      6.0            4.0   \n",
       "1   1               1.0                      7.0            3.0   \n",
       "2   2               6.0                      1.0            0.0   \n",
       "3   3               3.0                      7.0            3.0   \n",
       "4   4               1.0                      4.0            4.0   \n",
       "\n",
       "   Friends_circle_size  Post_frequency  \n",
       "0                 15.0             5.0  \n",
       "1                 10.0             8.0  \n",
       "2                  3.0             0.0  \n",
       "3                 11.0             5.0  \n",
       "4                 13.0             NaN  "
      ]
     },
     "execution_count": 1220,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_num.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1221,
   "id": "a6d5dc53-a775-4806-aaa7-c50157fc1f4b",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "id                            0\n",
       "Time_spent_Alone           1190\n",
       "Social_event_attendance    1180\n",
       "Going_outside              1466\n",
       "Friends_circle_size        1054\n",
       "Post_frequency             1264\n",
       "dtype: int64"
      ]
     },
     "execution_count": 1221,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_num.isna().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1222,
   "id": "8079e65e-2513-41a5-bdfa-41243901a485",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<style>#sk-container-id-42 {\n",
       "  /* Definition of color scheme common for light and dark mode */\n",
       "  --sklearn-color-text: #000;\n",
       "  --sklearn-color-text-muted: #666;\n",
       "  --sklearn-color-line: gray;\n",
       "  /* Definition of color scheme for unfitted estimators */\n",
       "  --sklearn-color-unfitted-level-0: #fff5e6;\n",
       "  --sklearn-color-unfitted-level-1: #f6e4d2;\n",
       "  --sklearn-color-unfitted-level-2: #ffe0b3;\n",
       "  --sklearn-color-unfitted-level-3: chocolate;\n",
       "  /* Definition of color scheme for fitted estimators */\n",
       "  --sklearn-color-fitted-level-0: #f0f8ff;\n",
       "  --sklearn-color-fitted-level-1: #d4ebff;\n",
       "  --sklearn-color-fitted-level-2: #b3dbfd;\n",
       "  --sklearn-color-fitted-level-3: cornflowerblue;\n",
       "\n",
       "  /* Specific color for light theme */\n",
       "  --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, white)));\n",
       "  --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-icon: #696969;\n",
       "\n",
       "  @media (prefers-color-scheme: dark) {\n",
       "    /* Redefinition of color scheme for dark theme */\n",
       "    --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, #111)));\n",
       "    --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-icon: #878787;\n",
       "  }\n",
       "}\n",
       "\n",
       "#sk-container-id-42 {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 pre {\n",
       "  padding: 0;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 input.sk-hidden--visually {\n",
       "  border: 0;\n",
       "  clip: rect(1px 1px 1px 1px);\n",
       "  clip: rect(1px, 1px, 1px, 1px);\n",
       "  height: 1px;\n",
       "  margin: -1px;\n",
       "  overflow: hidden;\n",
       "  padding: 0;\n",
       "  position: absolute;\n",
       "  width: 1px;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-dashed-wrapped {\n",
       "  border: 1px dashed var(--sklearn-color-line);\n",
       "  margin: 0 0.4em 0.5em 0.4em;\n",
       "  box-sizing: border-box;\n",
       "  padding-bottom: 0.4em;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-container {\n",
       "  /* jupyter's `normalize.less` sets `[hidden] { display: none; }`\n",
       "     but bootstrap.min.css set `[hidden] { display: none !important; }`\n",
       "     so we also need the `!important` here to be able to override the\n",
       "     default hidden behavior on the sphinx rendered scikit-learn.org.\n",
       "     See: https://github.com/scikit-learn/scikit-learn/issues/21755 */\n",
       "  display: inline-block !important;\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-text-repr-fallback {\n",
       "  display: none;\n",
       "}\n",
       "\n",
       "div.sk-parallel-item,\n",
       "div.sk-serial,\n",
       "div.sk-item {\n",
       "  /* draw centered vertical line to link estimators */\n",
       "  background-image: linear-gradient(var(--sklearn-color-text-on-default-background), var(--sklearn-color-text-on-default-background));\n",
       "  background-size: 2px 100%;\n",
       "  background-repeat: no-repeat;\n",
       "  background-position: center center;\n",
       "}\n",
       "\n",
       "/* Parallel-specific style estimator block */\n",
       "\n",
       "#sk-container-id-42 div.sk-parallel-item::after {\n",
       "  content: \"\";\n",
       "  width: 100%;\n",
       "  border-bottom: 2px solid var(--sklearn-color-text-on-default-background);\n",
       "  flex-grow: 1;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-parallel {\n",
       "  display: flex;\n",
       "  align-items: stretch;\n",
       "  justify-content: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-parallel-item {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-parallel-item:first-child::after {\n",
       "  align-self: flex-end;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-parallel-item:last-child::after {\n",
       "  align-self: flex-start;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-parallel-item:only-child::after {\n",
       "  width: 0;\n",
       "}\n",
       "\n",
       "/* Serial-specific style estimator block */\n",
       "\n",
       "#sk-container-id-42 div.sk-serial {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "  align-items: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  padding-right: 1em;\n",
       "  padding-left: 1em;\n",
       "}\n",
       "\n",
       "\n",
       "/* Toggleable style: style used for estimator/Pipeline/ColumnTransformer box that is\n",
       "clickable and can be expanded/collapsed.\n",
       "- Pipeline and ColumnTransformer use this feature and define the default style\n",
       "- Estimators will overwrite some part of the style using the `sk-estimator` class\n",
       "*/\n",
       "\n",
       "/* Pipeline and ColumnTransformer style (default) */\n",
       "\n",
       "#sk-container-id-42 div.sk-toggleable {\n",
       "  /* Default theme specific background. It is overwritten whether we have a\n",
       "  specific estimator or a Pipeline/ColumnTransformer */\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "/* Toggleable label */\n",
       "#sk-container-id-42 label.sk-toggleable__label {\n",
       "  cursor: pointer;\n",
       "  display: flex;\n",
       "  width: 100%;\n",
       "  margin-bottom: 0;\n",
       "  padding: 0.5em;\n",
       "  box-sizing: border-box;\n",
       "  text-align: center;\n",
       "  align-items: start;\n",
       "  justify-content: space-between;\n",
       "  gap: 0.5em;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 label.sk-toggleable__label .caption {\n",
       "  font-size: 0.6rem;\n",
       "  font-weight: lighter;\n",
       "  color: var(--sklearn-color-text-muted);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 label.sk-toggleable__label-arrow:before {\n",
       "  /* Arrow on the left of the label */\n",
       "  content: \"▸\";\n",
       "  float: left;\n",
       "  margin-right: 0.25em;\n",
       "  color: var(--sklearn-color-icon);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 label.sk-toggleable__label-arrow:hover:before {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "/* Toggleable content - dropdown */\n",
       "\n",
       "#sk-container-id-42 div.sk-toggleable__content {\n",
       "  max-height: 0;\n",
       "  max-width: 0;\n",
       "  overflow: hidden;\n",
       "  text-align: left;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-toggleable__content.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-toggleable__content pre {\n",
       "  margin: 0.2em;\n",
       "  border-radius: 0.25em;\n",
       "  color: var(--sklearn-color-text);\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-toggleable__content.fitted pre {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 input.sk-toggleable__control:checked~div.sk-toggleable__content {\n",
       "  /* Expand drop-down */\n",
       "  max-height: 200px;\n",
       "  max-width: 100%;\n",
       "  overflow: auto;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 input.sk-toggleable__control:checked~label.sk-toggleable__label-arrow:before {\n",
       "  content: \"▾\";\n",
       "}\n",
       "\n",
       "/* Pipeline/ColumnTransformer-specific style */\n",
       "\n",
       "#sk-container-id-42 div.sk-label input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-label.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator-specific style */\n",
       "\n",
       "/* Colorize estimator box */\n",
       "#sk-container-id-42 div.sk-estimator input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-estimator.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-label label.sk-toggleable__label,\n",
       "#sk-container-id-42 div.sk-label label {\n",
       "  /* The background is the default theme color */\n",
       "  color: var(--sklearn-color-text-on-default-background);\n",
       "}\n",
       "\n",
       "/* On hover, darken the color of the background */\n",
       "#sk-container-id-42 div.sk-label:hover label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "/* Label box, darken color on hover, fitted */\n",
       "#sk-container-id-42 div.sk-label.fitted:hover label.sk-toggleable__label.fitted {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator label */\n",
       "\n",
       "#sk-container-id-42 div.sk-label label {\n",
       "  font-family: monospace;\n",
       "  font-weight: bold;\n",
       "  display: inline-block;\n",
       "  line-height: 1.2em;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-label-container {\n",
       "  text-align: center;\n",
       "}\n",
       "\n",
       "/* Estimator-specific */\n",
       "#sk-container-id-42 div.sk-estimator {\n",
       "  font-family: monospace;\n",
       "  border: 1px dotted var(--sklearn-color-border-box);\n",
       "  border-radius: 0.25em;\n",
       "  box-sizing: border-box;\n",
       "  margin-bottom: 0.5em;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-estimator.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "/* on hover */\n",
       "#sk-container-id-42 div.sk-estimator:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-42 div.sk-estimator.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Specification for estimator info (e.g. \"i\" and \"?\") */\n",
       "\n",
       "/* Common style for \"i\" and \"?\" */\n",
       "\n",
       ".sk-estimator-doc-link,\n",
       "a:link.sk-estimator-doc-link,\n",
       "a:visited.sk-estimator-doc-link {\n",
       "  float: right;\n",
       "  font-size: smaller;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1em;\n",
       "  height: 1em;\n",
       "  width: 1em;\n",
       "  text-decoration: none !important;\n",
       "  margin-left: 0.5em;\n",
       "  text-align: center;\n",
       "  /* unfitted */\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted,\n",
       "a:link.sk-estimator-doc-link.fitted,\n",
       "a:visited.sk-estimator-doc-link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "div.sk-estimator:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "div.sk-estimator.fitted:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "/* Span, style for the box shown on hovering the info icon */\n",
       ".sk-estimator-doc-link span {\n",
       "  display: none;\n",
       "  z-index: 9999;\n",
       "  position: relative;\n",
       "  font-weight: normal;\n",
       "  right: .2ex;\n",
       "  padding: .5ex;\n",
       "  margin: .5ex;\n",
       "  width: min-content;\n",
       "  min-width: 20ex;\n",
       "  max-width: 50ex;\n",
       "  color: var(--sklearn-color-text);\n",
       "  box-shadow: 2pt 2pt 4pt #999;\n",
       "  /* unfitted */\n",
       "  background: var(--sklearn-color-unfitted-level-0);\n",
       "  border: .5pt solid var(--sklearn-color-unfitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted span {\n",
       "  /* fitted */\n",
       "  background: var(--sklearn-color-fitted-level-0);\n",
       "  border: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link:hover span {\n",
       "  display: block;\n",
       "}\n",
       "\n",
       "/* \"?\"-specific style due to the `<a>` HTML tag */\n",
       "\n",
       "#sk-container-id-42 a.estimator_doc_link {\n",
       "  float: right;\n",
       "  font-size: 1rem;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1rem;\n",
       "  height: 1rem;\n",
       "  width: 1rem;\n",
       "  text-decoration: none;\n",
       "  /* unfitted */\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 a.estimator_doc_link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "#sk-container-id-42 a.estimator_doc_link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "#sk-container-id-42 a.estimator_doc_link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "</style><div id=\"sk-container-id-42\" class=\"sk-top-container\"><div class=\"sk-text-repr-fallback\"><pre>SimpleImputer(strategy=&#x27;median&#x27;)</pre><b>In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. <br />On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.</b></div><div class=\"sk-container\" hidden><div class=\"sk-item\"><div class=\"sk-estimator fitted sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-42\" type=\"checkbox\" checked><label for=\"sk-estimator-id-42\" class=\"sk-toggleable__label fitted sk-toggleable__label-arrow\"><div><div>SimpleImputer</div></div><div><a class=\"sk-estimator-doc-link fitted\" rel=\"noreferrer\" target=\"_blank\" href=\"https://scikit-learn.org/1.6/modules/generated/sklearn.impute.SimpleImputer.html\">?<span>Documentation for SimpleImputer</span></a><span class=\"sk-estimator-doc-link fitted\">i<span>Fitted</span></span></div></label><div class=\"sk-toggleable__content fitted\"><pre>SimpleImputer(strategy=&#x27;median&#x27;)</pre></div> </div></div></div></div>"
      ],
      "text/plain": [
       "SimpleImputer(strategy='median')"
      ]
     },
     "execution_count": 1222,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "imputer.fit(people_num)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1223,
   "id": "47437f2e-6b3d-48b0-a723-a3ea8e658de0",
   "metadata": {},
   "outputs": [],
   "source": [
    "X = imputer.transform(people_num)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1224,
   "id": "d7002213-c28e-40ec-87d5-004dcfeb44fa",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_num = pd.DataFrame(X, columns=people_num.columns)\n",
    "people_num = people_num.drop('id',axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1225,
   "id": "1f5dda6e-d196-4acf-8f71-cb360fc74f4a",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>15.0</td>\n",
       "      <td>5.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1.0</td>\n",
       "      <td>7.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>10.0</td>\n",
       "      <td>8.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>6.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>3.0</td>\n",
       "      <td>7.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>11.0</td>\n",
       "      <td>5.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>1.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>13.0</td>\n",
       "      <td>5.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "0               0.0                      6.0            4.0   \n",
       "1               1.0                      7.0            3.0   \n",
       "2               6.0                      1.0            0.0   \n",
       "3               3.0                      7.0            3.0   \n",
       "4               1.0                      4.0            4.0   \n",
       "\n",
       "   Friends_circle_size  Post_frequency  \n",
       "0                 15.0             5.0  \n",
       "1                 10.0             8.0  \n",
       "2                  3.0             0.0  \n",
       "3                 11.0             5.0  \n",
       "4                 13.0             5.0  "
      ]
     },
     "execution_count": 1225,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_num.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1226,
   "id": "9c3dc8c1-7f96-4555-adb0-287ee44b5901",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Time_spent_Alone           0\n",
       "Social_event_attendance    0\n",
       "Going_outside              0\n",
       "Friends_circle_size        0\n",
       "Post_frequency             0\n",
       "dtype: int64"
      ]
     },
     "execution_count": 1226,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_num.isna().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1227,
   "id": "530df8e2-6160-4769-8270-b6dd0a646db3",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_cat = people.select_dtypes(include=[object])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1228,
   "id": "81b6f35a-541a-4dfb-9a41-e544a4c3ce91",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Stage_fear</th>\n",
       "      <th>Drained_after_socializing</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>Yes</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "  Stage_fear Drained_after_socializing\n",
       "0         No                        No\n",
       "1         No                        No\n",
       "2        Yes                       NaN\n",
       "3         No                        No\n",
       "4         No                        No"
      ]
     },
     "execution_count": 1228,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_cat.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1229,
   "id": "c0c1849c-24da-4532-83d2-f3d60965ab62",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Stage_fear                   1893\n",
       "Drained_after_socializing    1149\n",
       "dtype: int64"
      ]
     },
     "execution_count": 1229,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_cat.isna().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1230,
   "id": "8770dc31-6cad-42ea-9b06-5f44172fb168",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_cat = people_cat.fillna(people_cat.mode().iloc[0])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1231,
   "id": "dfca7a31-9b50-4a1c-9604-bd5b4a92817b",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Stage_fear                   0\n",
       "Drained_after_socializing    0\n",
       "dtype: int64"
      ]
     },
     "execution_count": 1231,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_cat.isna().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1232,
   "id": "fb09db38-da4d-4f4d-a217-eb94b824997d",
   "metadata": {},
   "outputs": [],
   "source": [
    "std_scaler = StandardScaler()\n",
    "people_num_std_scaled = std_scaler.fit_transform(people_num)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1233,
   "id": "593365f1-4417-40db-ae5a-68c217fcce5a",
   "metadata": {},
   "outputs": [],
   "source": [
    "cat_encoder = OneHotEncoder()\n",
    "people_cat_1hot = cat_encoder.fit_transform(people_cat)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1234,
   "id": "cbbe6ff0-994c-4132-a968-2a6bc81871d3",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[1., 0., 1., 0.],\n",
       "       [1., 0., 1., 0.],\n",
       "       [0., 1., 1., 0.],\n",
       "       ...,\n",
       "       [0., 1., 0., 1.],\n",
       "       [0., 1., 0., 1.],\n",
       "       [1., 0., 1., 0.]])"
      ]
     },
     "execution_count": 1234,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_cat_1hot.toarray()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1235,
   "id": "042cecc9-424d-4d8f-83f9-5a9bfc3edaee",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[-1.04991287,  0.28210179, -0.0206198 ,  1.70746346,  0.00600233],\n",
       "       [-0.70732724,  0.65734618, -0.52585938,  0.48838259,  1.08548984],\n",
       "       [ 1.00560089, -1.5941202 , -2.04157811, -1.21833063, -1.79314352],\n",
       "       ...,\n",
       "       [ 1.34818652, -1.5941202 , -1.53633853, -1.70596297,  0.00600233],\n",
       "       [-0.36474162, -1.5941202 , -2.04157811, -0.73069828, -1.07348518],\n",
       "       [-0.70732724,  1.03259058,  0.98985936, -0.97451445,  0.72566067]])"
      ]
     },
     "execution_count": 1235,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_num_std_scaled"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "03350c9b-92e4-455f-97e0-f0ba5c7ac961",
   "metadata": {},
   "source": [
    "<h2>Build dataframe</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1236,
   "id": "cf0fc508-3d57-4da7-a3c5-25c54a1635ae",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_people_num = pd.DataFrame(people_num_std_scaled,columns=std_scaler.get_feature_names_out(),index=people.id)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1237,
   "id": "843b3dce-30e2-4324-9a90-66f438117a7b",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>id</th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>-1.049913</td>\n",
       "      <td>0.282102</td>\n",
       "      <td>-0.020620</td>\n",
       "      <td>1.707463</td>\n",
       "      <td>0.006002</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>-0.707327</td>\n",
       "      <td>0.657346</td>\n",
       "      <td>-0.525859</td>\n",
       "      <td>0.488383</td>\n",
       "      <td>1.085490</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>1.005601</td>\n",
       "      <td>-1.594120</td>\n",
       "      <td>-2.041578</td>\n",
       "      <td>-1.218331</td>\n",
       "      <td>-1.793144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>-0.022156</td>\n",
       "      <td>0.657346</td>\n",
       "      <td>-0.525859</td>\n",
       "      <td>0.732199</td>\n",
       "      <td>0.006002</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>-0.707327</td>\n",
       "      <td>-0.468387</td>\n",
       "      <td>-0.020620</td>\n",
       "      <td>1.219831</td>\n",
       "      <td>0.006002</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18519</th>\n",
       "      <td>-0.022156</td>\n",
       "      <td>0.657346</td>\n",
       "      <td>-0.525859</td>\n",
       "      <td>0.244566</td>\n",
       "      <td>0.725661</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18520</th>\n",
       "      <td>-0.707327</td>\n",
       "      <td>0.282102</td>\n",
       "      <td>1.495099</td>\n",
       "      <td>-0.486882</td>\n",
       "      <td>0.006002</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18521</th>\n",
       "      <td>1.348187</td>\n",
       "      <td>-1.594120</td>\n",
       "      <td>-1.536339</td>\n",
       "      <td>-1.705963</td>\n",
       "      <td>0.006002</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18522</th>\n",
       "      <td>-0.364742</td>\n",
       "      <td>-1.594120</td>\n",
       "      <td>-2.041578</td>\n",
       "      <td>-0.730698</td>\n",
       "      <td>-1.073485</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18523</th>\n",
       "      <td>-0.707327</td>\n",
       "      <td>1.032591</td>\n",
       "      <td>0.989859</td>\n",
       "      <td>-0.974514</td>\n",
       "      <td>0.725661</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>18524 rows × 5 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "       Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "id                                                                \n",
       "0             -1.049913                 0.282102      -0.020620   \n",
       "1             -0.707327                 0.657346      -0.525859   \n",
       "2              1.005601                -1.594120      -2.041578   \n",
       "3             -0.022156                 0.657346      -0.525859   \n",
       "4             -0.707327                -0.468387      -0.020620   \n",
       "...                 ...                      ...            ...   \n",
       "18519         -0.022156                 0.657346      -0.525859   \n",
       "18520         -0.707327                 0.282102       1.495099   \n",
       "18521          1.348187                -1.594120      -1.536339   \n",
       "18522         -0.364742                -1.594120      -2.041578   \n",
       "18523         -0.707327                 1.032591       0.989859   \n",
       "\n",
       "       Friends_circle_size  Post_frequency  \n",
       "id                                          \n",
       "0                 1.707463        0.006002  \n",
       "1                 0.488383        1.085490  \n",
       "2                -1.218331       -1.793144  \n",
       "3                 0.732199        0.006002  \n",
       "4                 1.219831        0.006002  \n",
       "...                    ...             ...  \n",
       "18519             0.244566        0.725661  \n",
       "18520            -0.486882        0.006002  \n",
       "18521            -1.705963        0.006002  \n",
       "18522            -0.730698       -1.073485  \n",
       "18523            -0.974514        0.725661  \n",
       "\n",
       "[18524 rows x 5 columns]"
      ]
     },
     "execution_count": 1237,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_people_num"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1238,
   "id": "a8854bcd-73fb-4d01-a16d-87f375456892",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_people_cat = pd.DataFrame(people_cat_1hot.toarray(),index=people.id)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1239,
   "id": "b4d06ac1-61ed-4d32-b0fb-436c3bfa4c64",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_data = pd.DataFrame(pd.merge(df_people_num,df_people_cat, on='id'))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1240,
   "id": "e9825987-7202-4c96-8c00-822ad539ee40",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_data.columns = people_data.columns.astype(str)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5b769dfc-b17d-4783-b161-72819cf27040",
   "metadata": {},
   "source": [
    "<h2>Train SGD Classifier</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1241,
   "id": "6fb3323d-8eb3-47db-a876-148da3aedf6c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<style>#sk-container-id-43 {\n",
       "  /* Definition of color scheme common for light and dark mode */\n",
       "  --sklearn-color-text: #000;\n",
       "  --sklearn-color-text-muted: #666;\n",
       "  --sklearn-color-line: gray;\n",
       "  /* Definition of color scheme for unfitted estimators */\n",
       "  --sklearn-color-unfitted-level-0: #fff5e6;\n",
       "  --sklearn-color-unfitted-level-1: #f6e4d2;\n",
       "  --sklearn-color-unfitted-level-2: #ffe0b3;\n",
       "  --sklearn-color-unfitted-level-3: chocolate;\n",
       "  /* Definition of color scheme for fitted estimators */\n",
       "  --sklearn-color-fitted-level-0: #f0f8ff;\n",
       "  --sklearn-color-fitted-level-1: #d4ebff;\n",
       "  --sklearn-color-fitted-level-2: #b3dbfd;\n",
       "  --sklearn-color-fitted-level-3: cornflowerblue;\n",
       "\n",
       "  /* Specific color for light theme */\n",
       "  --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, white)));\n",
       "  --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-icon: #696969;\n",
       "\n",
       "  @media (prefers-color-scheme: dark) {\n",
       "    /* Redefinition of color scheme for dark theme */\n",
       "    --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, #111)));\n",
       "    --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-icon: #878787;\n",
       "  }\n",
       "}\n",
       "\n",
       "#sk-container-id-43 {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 pre {\n",
       "  padding: 0;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 input.sk-hidden--visually {\n",
       "  border: 0;\n",
       "  clip: rect(1px 1px 1px 1px);\n",
       "  clip: rect(1px, 1px, 1px, 1px);\n",
       "  height: 1px;\n",
       "  margin: -1px;\n",
       "  overflow: hidden;\n",
       "  padding: 0;\n",
       "  position: absolute;\n",
       "  width: 1px;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-dashed-wrapped {\n",
       "  border: 1px dashed var(--sklearn-color-line);\n",
       "  margin: 0 0.4em 0.5em 0.4em;\n",
       "  box-sizing: border-box;\n",
       "  padding-bottom: 0.4em;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-container {\n",
       "  /* jupyter's `normalize.less` sets `[hidden] { display: none; }`\n",
       "     but bootstrap.min.css set `[hidden] { display: none !important; }`\n",
       "     so we also need the `!important` here to be able to override the\n",
       "     default hidden behavior on the sphinx rendered scikit-learn.org.\n",
       "     See: https://github.com/scikit-learn/scikit-learn/issues/21755 */\n",
       "  display: inline-block !important;\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-text-repr-fallback {\n",
       "  display: none;\n",
       "}\n",
       "\n",
       "div.sk-parallel-item,\n",
       "div.sk-serial,\n",
       "div.sk-item {\n",
       "  /* draw centered vertical line to link estimators */\n",
       "  background-image: linear-gradient(var(--sklearn-color-text-on-default-background), var(--sklearn-color-text-on-default-background));\n",
       "  background-size: 2px 100%;\n",
       "  background-repeat: no-repeat;\n",
       "  background-position: center center;\n",
       "}\n",
       "\n",
       "/* Parallel-specific style estimator block */\n",
       "\n",
       "#sk-container-id-43 div.sk-parallel-item::after {\n",
       "  content: \"\";\n",
       "  width: 100%;\n",
       "  border-bottom: 2px solid var(--sklearn-color-text-on-default-background);\n",
       "  flex-grow: 1;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-parallel {\n",
       "  display: flex;\n",
       "  align-items: stretch;\n",
       "  justify-content: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-parallel-item {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-parallel-item:first-child::after {\n",
       "  align-self: flex-end;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-parallel-item:last-child::after {\n",
       "  align-self: flex-start;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-parallel-item:only-child::after {\n",
       "  width: 0;\n",
       "}\n",
       "\n",
       "/* Serial-specific style estimator block */\n",
       "\n",
       "#sk-container-id-43 div.sk-serial {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "  align-items: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  padding-right: 1em;\n",
       "  padding-left: 1em;\n",
       "}\n",
       "\n",
       "\n",
       "/* Toggleable style: style used for estimator/Pipeline/ColumnTransformer box that is\n",
       "clickable and can be expanded/collapsed.\n",
       "- Pipeline and ColumnTransformer use this feature and define the default style\n",
       "- Estimators will overwrite some part of the style using the `sk-estimator` class\n",
       "*/\n",
       "\n",
       "/* Pipeline and ColumnTransformer style (default) */\n",
       "\n",
       "#sk-container-id-43 div.sk-toggleable {\n",
       "  /* Default theme specific background. It is overwritten whether we have a\n",
       "  specific estimator or a Pipeline/ColumnTransformer */\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "/* Toggleable label */\n",
       "#sk-container-id-43 label.sk-toggleable__label {\n",
       "  cursor: pointer;\n",
       "  display: flex;\n",
       "  width: 100%;\n",
       "  margin-bottom: 0;\n",
       "  padding: 0.5em;\n",
       "  box-sizing: border-box;\n",
       "  text-align: center;\n",
       "  align-items: start;\n",
       "  justify-content: space-between;\n",
       "  gap: 0.5em;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 label.sk-toggleable__label .caption {\n",
       "  font-size: 0.6rem;\n",
       "  font-weight: lighter;\n",
       "  color: var(--sklearn-color-text-muted);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 label.sk-toggleable__label-arrow:before {\n",
       "  /* Arrow on the left of the label */\n",
       "  content: \"▸\";\n",
       "  float: left;\n",
       "  margin-right: 0.25em;\n",
       "  color: var(--sklearn-color-icon);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 label.sk-toggleable__label-arrow:hover:before {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "/* Toggleable content - dropdown */\n",
       "\n",
       "#sk-container-id-43 div.sk-toggleable__content {\n",
       "  max-height: 0;\n",
       "  max-width: 0;\n",
       "  overflow: hidden;\n",
       "  text-align: left;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-toggleable__content.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-toggleable__content pre {\n",
       "  margin: 0.2em;\n",
       "  border-radius: 0.25em;\n",
       "  color: var(--sklearn-color-text);\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-toggleable__content.fitted pre {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 input.sk-toggleable__control:checked~div.sk-toggleable__content {\n",
       "  /* Expand drop-down */\n",
       "  max-height: 200px;\n",
       "  max-width: 100%;\n",
       "  overflow: auto;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 input.sk-toggleable__control:checked~label.sk-toggleable__label-arrow:before {\n",
       "  content: \"▾\";\n",
       "}\n",
       "\n",
       "/* Pipeline/ColumnTransformer-specific style */\n",
       "\n",
       "#sk-container-id-43 div.sk-label input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-label.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator-specific style */\n",
       "\n",
       "/* Colorize estimator box */\n",
       "#sk-container-id-43 div.sk-estimator input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-estimator.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-label label.sk-toggleable__label,\n",
       "#sk-container-id-43 div.sk-label label {\n",
       "  /* The background is the default theme color */\n",
       "  color: var(--sklearn-color-text-on-default-background);\n",
       "}\n",
       "\n",
       "/* On hover, darken the color of the background */\n",
       "#sk-container-id-43 div.sk-label:hover label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "/* Label box, darken color on hover, fitted */\n",
       "#sk-container-id-43 div.sk-label.fitted:hover label.sk-toggleable__label.fitted {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator label */\n",
       "\n",
       "#sk-container-id-43 div.sk-label label {\n",
       "  font-family: monospace;\n",
       "  font-weight: bold;\n",
       "  display: inline-block;\n",
       "  line-height: 1.2em;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-label-container {\n",
       "  text-align: center;\n",
       "}\n",
       "\n",
       "/* Estimator-specific */\n",
       "#sk-container-id-43 div.sk-estimator {\n",
       "  font-family: monospace;\n",
       "  border: 1px dotted var(--sklearn-color-border-box);\n",
       "  border-radius: 0.25em;\n",
       "  box-sizing: border-box;\n",
       "  margin-bottom: 0.5em;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-estimator.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "/* on hover */\n",
       "#sk-container-id-43 div.sk-estimator:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-43 div.sk-estimator.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Specification for estimator info (e.g. \"i\" and \"?\") */\n",
       "\n",
       "/* Common style for \"i\" and \"?\" */\n",
       "\n",
       ".sk-estimator-doc-link,\n",
       "a:link.sk-estimator-doc-link,\n",
       "a:visited.sk-estimator-doc-link {\n",
       "  float: right;\n",
       "  font-size: smaller;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1em;\n",
       "  height: 1em;\n",
       "  width: 1em;\n",
       "  text-decoration: none !important;\n",
       "  margin-left: 0.5em;\n",
       "  text-align: center;\n",
       "  /* unfitted */\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted,\n",
       "a:link.sk-estimator-doc-link.fitted,\n",
       "a:visited.sk-estimator-doc-link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "div.sk-estimator:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "div.sk-estimator.fitted:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "/* Span, style for the box shown on hovering the info icon */\n",
       ".sk-estimator-doc-link span {\n",
       "  display: none;\n",
       "  z-index: 9999;\n",
       "  position: relative;\n",
       "  font-weight: normal;\n",
       "  right: .2ex;\n",
       "  padding: .5ex;\n",
       "  margin: .5ex;\n",
       "  width: min-content;\n",
       "  min-width: 20ex;\n",
       "  max-width: 50ex;\n",
       "  color: var(--sklearn-color-text);\n",
       "  box-shadow: 2pt 2pt 4pt #999;\n",
       "  /* unfitted */\n",
       "  background: var(--sklearn-color-unfitted-level-0);\n",
       "  border: .5pt solid var(--sklearn-color-unfitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted span {\n",
       "  /* fitted */\n",
       "  background: var(--sklearn-color-fitted-level-0);\n",
       "  border: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link:hover span {\n",
       "  display: block;\n",
       "}\n",
       "\n",
       "/* \"?\"-specific style due to the `<a>` HTML tag */\n",
       "\n",
       "#sk-container-id-43 a.estimator_doc_link {\n",
       "  float: right;\n",
       "  font-size: 1rem;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1rem;\n",
       "  height: 1rem;\n",
       "  width: 1rem;\n",
       "  text-decoration: none;\n",
       "  /* unfitted */\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 a.estimator_doc_link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "#sk-container-id-43 a.estimator_doc_link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "#sk-container-id-43 a.estimator_doc_link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "</style><div id=\"sk-container-id-43\" class=\"sk-top-container\"><div class=\"sk-text-repr-fallback\"><pre>SGDClassifier(random_state=42)</pre><b>In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. <br />On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.</b></div><div class=\"sk-container\" hidden><div class=\"sk-item\"><div class=\"sk-estimator fitted sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-43\" type=\"checkbox\" checked><label for=\"sk-estimator-id-43\" class=\"sk-toggleable__label fitted sk-toggleable__label-arrow\"><div><div>SGDClassifier</div></div><div><a class=\"sk-estimator-doc-link fitted\" rel=\"noreferrer\" target=\"_blank\" href=\"https://scikit-learn.org/1.6/modules/generated/sklearn.linear_model.SGDClassifier.html\">?<span>Documentation for SGDClassifier</span></a><span class=\"sk-estimator-doc-link fitted\">i<span>Fitted</span></span></div></label><div class=\"sk-toggleable__content fitted\"><pre>SGDClassifier(random_state=42)</pre></div> </div></div></div></div>"
      ],
      "text/plain": [
       "SGDClassifier(random_state=42)"
      ]
     },
     "execution_count": 1241,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "sgd_clf = SGDClassifier(random_state=42)\n",
    "sgd_clf.fit(people_data,people_labels)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "12566a15-9b55-477d-9595-2e167035d256",
   "metadata": {},
   "source": [
    "<h2>Validation scores</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1242,
   "id": "156adedb-3d38-4b17-bbb7-7b52a158c430",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([0.96842105, 0.96696356, 0.97149336])"
      ]
     },
     "execution_count": 1242,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cross_val_score(sgd_clf,people_data,people_labels,cv=3,scoring=\"accuracy\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1243,
   "id": "683161a4-5683-4e42-8a46-40af78d9c29f",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<style>#sk-container-id-44 {\n",
       "  /* Definition of color scheme common for light and dark mode */\n",
       "  --sklearn-color-text: #000;\n",
       "  --sklearn-color-text-muted: #666;\n",
       "  --sklearn-color-line: gray;\n",
       "  /* Definition of color scheme for unfitted estimators */\n",
       "  --sklearn-color-unfitted-level-0: #fff5e6;\n",
       "  --sklearn-color-unfitted-level-1: #f6e4d2;\n",
       "  --sklearn-color-unfitted-level-2: #ffe0b3;\n",
       "  --sklearn-color-unfitted-level-3: chocolate;\n",
       "  /* Definition of color scheme for fitted estimators */\n",
       "  --sklearn-color-fitted-level-0: #f0f8ff;\n",
       "  --sklearn-color-fitted-level-1: #d4ebff;\n",
       "  --sklearn-color-fitted-level-2: #b3dbfd;\n",
       "  --sklearn-color-fitted-level-3: cornflowerblue;\n",
       "\n",
       "  /* Specific color for light theme */\n",
       "  --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, white)));\n",
       "  --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-icon: #696969;\n",
       "\n",
       "  @media (prefers-color-scheme: dark) {\n",
       "    /* Redefinition of color scheme for dark theme */\n",
       "    --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, #111)));\n",
       "    --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-icon: #878787;\n",
       "  }\n",
       "}\n",
       "\n",
       "#sk-container-id-44 {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 pre {\n",
       "  padding: 0;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 input.sk-hidden--visually {\n",
       "  border: 0;\n",
       "  clip: rect(1px 1px 1px 1px);\n",
       "  clip: rect(1px, 1px, 1px, 1px);\n",
       "  height: 1px;\n",
       "  margin: -1px;\n",
       "  overflow: hidden;\n",
       "  padding: 0;\n",
       "  position: absolute;\n",
       "  width: 1px;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-dashed-wrapped {\n",
       "  border: 1px dashed var(--sklearn-color-line);\n",
       "  margin: 0 0.4em 0.5em 0.4em;\n",
       "  box-sizing: border-box;\n",
       "  padding-bottom: 0.4em;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-container {\n",
       "  /* jupyter's `normalize.less` sets `[hidden] { display: none; }`\n",
       "     but bootstrap.min.css set `[hidden] { display: none !important; }`\n",
       "     so we also need the `!important` here to be able to override the\n",
       "     default hidden behavior on the sphinx rendered scikit-learn.org.\n",
       "     See: https://github.com/scikit-learn/scikit-learn/issues/21755 */\n",
       "  display: inline-block !important;\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-text-repr-fallback {\n",
       "  display: none;\n",
       "}\n",
       "\n",
       "div.sk-parallel-item,\n",
       "div.sk-serial,\n",
       "div.sk-item {\n",
       "  /* draw centered vertical line to link estimators */\n",
       "  background-image: linear-gradient(var(--sklearn-color-text-on-default-background), var(--sklearn-color-text-on-default-background));\n",
       "  background-size: 2px 100%;\n",
       "  background-repeat: no-repeat;\n",
       "  background-position: center center;\n",
       "}\n",
       "\n",
       "/* Parallel-specific style estimator block */\n",
       "\n",
       "#sk-container-id-44 div.sk-parallel-item::after {\n",
       "  content: \"\";\n",
       "  width: 100%;\n",
       "  border-bottom: 2px solid var(--sklearn-color-text-on-default-background);\n",
       "  flex-grow: 1;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-parallel {\n",
       "  display: flex;\n",
       "  align-items: stretch;\n",
       "  justify-content: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-parallel-item {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-parallel-item:first-child::after {\n",
       "  align-self: flex-end;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-parallel-item:last-child::after {\n",
       "  align-self: flex-start;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-parallel-item:only-child::after {\n",
       "  width: 0;\n",
       "}\n",
       "\n",
       "/* Serial-specific style estimator block */\n",
       "\n",
       "#sk-container-id-44 div.sk-serial {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "  align-items: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  padding-right: 1em;\n",
       "  padding-left: 1em;\n",
       "}\n",
       "\n",
       "\n",
       "/* Toggleable style: style used for estimator/Pipeline/ColumnTransformer box that is\n",
       "clickable and can be expanded/collapsed.\n",
       "- Pipeline and ColumnTransformer use this feature and define the default style\n",
       "- Estimators will overwrite some part of the style using the `sk-estimator` class\n",
       "*/\n",
       "\n",
       "/* Pipeline and ColumnTransformer style (default) */\n",
       "\n",
       "#sk-container-id-44 div.sk-toggleable {\n",
       "  /* Default theme specific background. It is overwritten whether we have a\n",
       "  specific estimator or a Pipeline/ColumnTransformer */\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "/* Toggleable label */\n",
       "#sk-container-id-44 label.sk-toggleable__label {\n",
       "  cursor: pointer;\n",
       "  display: flex;\n",
       "  width: 100%;\n",
       "  margin-bottom: 0;\n",
       "  padding: 0.5em;\n",
       "  box-sizing: border-box;\n",
       "  text-align: center;\n",
       "  align-items: start;\n",
       "  justify-content: space-between;\n",
       "  gap: 0.5em;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 label.sk-toggleable__label .caption {\n",
       "  font-size: 0.6rem;\n",
       "  font-weight: lighter;\n",
       "  color: var(--sklearn-color-text-muted);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 label.sk-toggleable__label-arrow:before {\n",
       "  /* Arrow on the left of the label */\n",
       "  content: \"▸\";\n",
       "  float: left;\n",
       "  margin-right: 0.25em;\n",
       "  color: var(--sklearn-color-icon);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 label.sk-toggleable__label-arrow:hover:before {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "/* Toggleable content - dropdown */\n",
       "\n",
       "#sk-container-id-44 div.sk-toggleable__content {\n",
       "  max-height: 0;\n",
       "  max-width: 0;\n",
       "  overflow: hidden;\n",
       "  text-align: left;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-toggleable__content.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-toggleable__content pre {\n",
       "  margin: 0.2em;\n",
       "  border-radius: 0.25em;\n",
       "  color: var(--sklearn-color-text);\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-toggleable__content.fitted pre {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 input.sk-toggleable__control:checked~div.sk-toggleable__content {\n",
       "  /* Expand drop-down */\n",
       "  max-height: 200px;\n",
       "  max-width: 100%;\n",
       "  overflow: auto;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 input.sk-toggleable__control:checked~label.sk-toggleable__label-arrow:before {\n",
       "  content: \"▾\";\n",
       "}\n",
       "\n",
       "/* Pipeline/ColumnTransformer-specific style */\n",
       "\n",
       "#sk-container-id-44 div.sk-label input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-label.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator-specific style */\n",
       "\n",
       "/* Colorize estimator box */\n",
       "#sk-container-id-44 div.sk-estimator input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-estimator.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-label label.sk-toggleable__label,\n",
       "#sk-container-id-44 div.sk-label label {\n",
       "  /* The background is the default theme color */\n",
       "  color: var(--sklearn-color-text-on-default-background);\n",
       "}\n",
       "\n",
       "/* On hover, darken the color of the background */\n",
       "#sk-container-id-44 div.sk-label:hover label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "/* Label box, darken color on hover, fitted */\n",
       "#sk-container-id-44 div.sk-label.fitted:hover label.sk-toggleable__label.fitted {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator label */\n",
       "\n",
       "#sk-container-id-44 div.sk-label label {\n",
       "  font-family: monospace;\n",
       "  font-weight: bold;\n",
       "  display: inline-block;\n",
       "  line-height: 1.2em;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-label-container {\n",
       "  text-align: center;\n",
       "}\n",
       "\n",
       "/* Estimator-specific */\n",
       "#sk-container-id-44 div.sk-estimator {\n",
       "  font-family: monospace;\n",
       "  border: 1px dotted var(--sklearn-color-border-box);\n",
       "  border-radius: 0.25em;\n",
       "  box-sizing: border-box;\n",
       "  margin-bottom: 0.5em;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-estimator.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "/* on hover */\n",
       "#sk-container-id-44 div.sk-estimator:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-44 div.sk-estimator.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Specification for estimator info (e.g. \"i\" and \"?\") */\n",
       "\n",
       "/* Common style for \"i\" and \"?\" */\n",
       "\n",
       ".sk-estimator-doc-link,\n",
       "a:link.sk-estimator-doc-link,\n",
       "a:visited.sk-estimator-doc-link {\n",
       "  float: right;\n",
       "  font-size: smaller;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1em;\n",
       "  height: 1em;\n",
       "  width: 1em;\n",
       "  text-decoration: none !important;\n",
       "  margin-left: 0.5em;\n",
       "  text-align: center;\n",
       "  /* unfitted */\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted,\n",
       "a:link.sk-estimator-doc-link.fitted,\n",
       "a:visited.sk-estimator-doc-link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "div.sk-estimator:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "div.sk-estimator.fitted:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "/* Span, style for the box shown on hovering the info icon */\n",
       ".sk-estimator-doc-link span {\n",
       "  display: none;\n",
       "  z-index: 9999;\n",
       "  position: relative;\n",
       "  font-weight: normal;\n",
       "  right: .2ex;\n",
       "  padding: .5ex;\n",
       "  margin: .5ex;\n",
       "  width: min-content;\n",
       "  min-width: 20ex;\n",
       "  max-width: 50ex;\n",
       "  color: var(--sklearn-color-text);\n",
       "  box-shadow: 2pt 2pt 4pt #999;\n",
       "  /* unfitted */\n",
       "  background: var(--sklearn-color-unfitted-level-0);\n",
       "  border: .5pt solid var(--sklearn-color-unfitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted span {\n",
       "  /* fitted */\n",
       "  background: var(--sklearn-color-fitted-level-0);\n",
       "  border: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link:hover span {\n",
       "  display: block;\n",
       "}\n",
       "\n",
       "/* \"?\"-specific style due to the `<a>` HTML tag */\n",
       "\n",
       "#sk-container-id-44 a.estimator_doc_link {\n",
       "  float: right;\n",
       "  font-size: 1rem;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1rem;\n",
       "  height: 1rem;\n",
       "  width: 1rem;\n",
       "  text-decoration: none;\n",
       "  /* unfitted */\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 a.estimator_doc_link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "#sk-container-id-44 a.estimator_doc_link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "#sk-container-id-44 a.estimator_doc_link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "</style><div id=\"sk-container-id-44\" class=\"sk-top-container\"><div class=\"sk-text-repr-fallback\"><pre>DummyClassifier()</pre><b>In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. <br />On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.</b></div><div class=\"sk-container\" hidden><div class=\"sk-item\"><div class=\"sk-estimator fitted sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-44\" type=\"checkbox\" checked><label for=\"sk-estimator-id-44\" class=\"sk-toggleable__label fitted sk-toggleable__label-arrow\"><div><div>DummyClassifier</div></div><div><a class=\"sk-estimator-doc-link fitted\" rel=\"noreferrer\" target=\"_blank\" href=\"https://scikit-learn.org/1.6/modules/generated/sklearn.dummy.DummyClassifier.html\">?<span>Documentation for DummyClassifier</span></a><span class=\"sk-estimator-doc-link fitted\">i<span>Fitted</span></span></div></label><div class=\"sk-toggleable__content fitted\"><pre>DummyClassifier()</pre></div> </div></div></div></div>"
      ],
      "text/plain": [
       "DummyClassifier()"
      ]
     },
     "execution_count": 1243,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "dummy_clf = DummyClassifier()\n",
    "dummy_clf.fit(people_data,people_labels)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1244,
   "id": "809cb95c-83e9-4aa3-84b5-0977a2c6de66",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([0.73959514, 0.7394332 , 0.73955296])"
      ]
     },
     "execution_count": 1244,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cross_val_score(dummy_clf, people_data, people_labels, cv=3, scoring=\"accuracy\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1245,
   "id": "8e39b735-9c2a-4b3b-a812-d9548e358175",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_labels_pred = cross_val_predict(sgd_clf, people_data, people_labels, cv=3)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1246,
   "id": "7bf8595a-64e9-4323-8cdd-0add0fb3d91d",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "['Extrovert' 'Extrovert' 'Introvert' ... 'Introvert' 'Introvert'\n",
      " 'Extrovert']\n"
     ]
    }
   ],
   "source": [
    "print(people_labels_pred)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1247,
   "id": "3987d207-f185-4ead-95d8-f15cd9995ad2",
   "metadata": {},
   "outputs": [],
   "source": [
    "cm = confusion_matrix(people_labels,people_labels_pred)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1248,
   "id": "70db1a0a-245b-41c5-afd3-74b85a02e94c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[13432,   267],\n",
       "       [  308,  4517]])"
      ]
     },
     "execution_count": 1248,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cm"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1249,
   "id": "2c9772dd-1ddf-41dd-9b32-2f1ece411fbe",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9775836972343522"
      ]
     },
     "execution_count": 1249,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "precision_score(people_labels,people_labels_pred,pos_label='Extrovert')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1250,
   "id": "6a969239-4993-4aec-8e01-7e108e2394ca",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9441889632107023"
      ]
     },
     "execution_count": 1250,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "precision_score(people_labels,people_labels_pred,pos_label='Introvert')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1251,
   "id": "99abe62f-1d44-40b9-86e6-4b642edc92db",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9805095262427914"
      ]
     },
     "execution_count": 1251,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "recall_score(people_labels,people_labels_pred,pos_label=\"Extrovert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1252,
   "id": "c9eb291e-7ddd-4480-9b52-29cd574b7a75",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9790444258172674"
      ]
     },
     "execution_count": 1252,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f1_score(people_labels,people_labels_pred,pos_label=\"Extrovert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1253,
   "id": "e6251531-36ab-4fb1-ba28-5043e7d3fcf7",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9401602664169009"
      ]
     },
     "execution_count": 1253,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f1_score(people_labels,people_labels_pred,pos_label=\"Introvert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1254,
   "id": "9487fec2-1339-443a-b926-cd78c5d957a1",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_scores = cross_val_predict(sgd_clf, people_data, people_labels, cv=3, method=\"decision_function\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1255,
   "id": "0af1f5e1-2db4-43d1-8fed-155666c08fcb",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[-2.31545754 -2.30101506  1.78522674 ...  2.52759721  2.14855122\n",
      " -1.83931529]\n"
     ]
    }
   ],
   "source": [
    "print(people_scores)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1256,
   "id": "453fba25-d542-48c2-8ed6-21fc2845692c",
   "metadata": {},
   "outputs": [],
   "source": [
    "precisionsI, recallsI, thresholdsI = precision_recall_curve(people_labels,people_scores,pos_label=\"Introvert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1257,
   "id": "b8ece3a6-2d03-4821-97d7-6a150fdca45b",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[        inf  3.8113973   3.7664525  ... -3.09598003 -3.44661855\n",
      " -3.5232931 ]\n"
     ]
    }
   ],
   "source": [
    "print(thresholds)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1309,
   "id": "68404c40-d564-45d3-80c0-3d4a3660b18e",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.plot(thresholdsI, precisionsI[:-1],\"b--\", label=\"Precision\", linewidth=2)\n",
    "plt.plot(thresholdsI, recallsI[:-1],\"g-\", label=\"Recall\", linewidth=2)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1259,
   "id": "2271c56d-0cb8-4602-a1cc-7fbe7679637d",
   "metadata": {},
   "outputs": [],
   "source": [
    "precisionsE, recallsE, thresholdsE = precision_recall_curve(people_labels,people_scores,pos_label=\"Extrovert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1260,
   "id": "67d1c787-bb19-4914-af29-35068c2ef361",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.plot(thresholdsE, precisionsE[:-1],\"b--\", label=\"Precision\", linewidth=2)\n",
    "plt.plot(thresholdsE, recallsE[:-1],\"g-\", label=\"Recall\", linewidth=2)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1261,
   "id": "b2c8a88c-b389-4bda-8d42-d5bc42b9c754",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[<matplotlib.lines.Line2D at 0x7f761d7b4190>]"
      ]
     },
     "execution_count": 1261,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAiMAAAGdCAYAAADAAnMpAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAANxpJREFUeJzt3XlclWX+//E3O4JwEFAWQURFRU1NSEOlmlLKnPYmp+abVjaT36YxdWpGx36V1neomaZpmlGbFtvGyrKaaUamomUUl8kkXBJccgOURTA5LLId7t8f4EkClYPAzTm8no/HecS5uO+bz7lD77f3dd3X5WYYhiEAAACTuJtdAAAA6NkIIwAAwFSEEQAAYCrCCAAAMBVhBAAAmIowAgAATEUYAQAApiKMAAAAU3maXUBbNDQ06OjRowoICJCbm5vZ5QAAgDYwDEPl5eWKjIyUu/uZ7384RRg5evSooqOjzS4DAAC0Q15enqKios74facIIwEBAZIaP0xgYKDJ1QAAgLawWq2Kjo62X8fPxCnCyKmumcDAQMIIAABO5lxDLBjACgAATEUYAQAApiKMAAAAUxFGAACAqQgjAADAVIQRAABgKsIIAAAwFWEEAACYijACAABM5XAYWb9+va655hpFRkbKzc1Nf//738+5z7p165SQkCBfX18NGjRIzz33XHtqBQAALsjhMFJZWakxY8boL3/5S5u2P3jwoK6++molJycrKytLv/nNbzR37ly9++67DhcLAABcj8Nr00ybNk3Tpk1r8/bPPfecBgwYoGeeeUaSFB8fr61bt+qpp57STTfd5OiPBwAALqbTF8rbvHmzUlJSmrVdeeWVeumll1RXVycvL68W+9TU1Kimpsb+3mq1dkptAxeulSR5uLtp/2+v7pSfAQAAzq7TB7AWFhYqLCysWVtYWJjq6+tVUlLS6j6pqamyWCz2V3R0dKfWaGswOvX4AADgzLrkaZrvLx1sGEar7acsWrRIZWVl9ldeXl6n1wgAAMzR6d004eHhKiwsbNZWXFwsT09PhYSEtLqPj4+PfHx8Ors0AADQDXT6nZGkpCSlp6c3a/v444+VmJjY6ngRAADQszgcRioqKrRt2zZt27ZNUuOju9u2bVNubq6kxi6WmTNn2refM2eODh8+rAULFignJ0crV67USy+9pAceeKBjPgEAAHBqDnfTbN26VT/4wQ/s7xcsWCBJmjVrll555RUVFBTYg4kkxcbGKi0tTfPnz9eyZcsUGRmpZ599lsd6AQCAJMnNODWatBuzWq2yWCwqKytTYGBghx331KO9knToiekddlwAAND26zdr0wAAAFMRRgAAgKkIIwAAwFSEEQAAYCrCCAAAMBVhBAAAmIowAgAATEUYAQAApiKMAAAAUxFGAACAqQgjAADAVIQRAABgKsIIAAAwFWEEAACYijACAABMRRgBAACmIowAAABTEUYAAICpCCMAAMBUhBEAAGAqwggAADAVYQQAAJiKMAIAAExFGAEAAKYijAAAAFMRRgAAgKkIIwAAwFSEEQAAYCrCCAAAMBVhBAAAmIowAgAATEUYAQAApiKMAAAAUxFGAACAqQgjAADAVIQRAABgKsIIAAAwFWEEAACYijACAABMRRgBAACmIowAAABTEUYAAICpCCMAAMBUhBEAAGAqwggAADAVYQQAAJiKMAIAAExFGAEAAKYijAAAAFMRRgAAgKkIIwAAwFSEEQAAYCrCCAAAMBVhBAAAmIowAgAATEUYAQAApiKMAAAAU7UrjCxfvlyxsbHy9fVVQkKCMjIyzrr9qlWrNGbMGPn5+SkiIkJ33nmnSktL21UwAABwLQ6HkdWrV2vevHlavHixsrKylJycrGnTpik3N7fV7Tds2KCZM2dq9uzZ2rVrl9555x19+eWXuvvuu8+7eAAA4PwcDiNPP/20Zs+erbvvvlvx8fF65plnFB0drRUrVrS6/X//+18NHDhQc+fOVWxsrCZPnqx77rlHW7duPe/iAQCA83MojNTW1iozM1MpKSnN2lNSUrRp06ZW95k4caLy8/OVlpYmwzBUVFSkNWvWaPr06Wf8OTU1NbJarc1eAADANTkURkpKSmSz2RQWFtasPSwsTIWFha3uM3HiRK1atUozZsyQt7e3wsPDFRQUpD//+c9n/DmpqamyWCz2V3R0tCNlAgAAJ9KuAaxubm7N3huG0aLtlOzsbM2dO1cPP/ywMjMz9eGHH+rgwYOaM2fOGY+/aNEilZWV2V95eXntKRMAADgBT0c2Dg0NlYeHR4u7IMXFxS3ulpySmpqqSZMm6cEHH5QkjR49Wv7+/kpOTtbjjz+uiIiIFvv4+PjIx8fHkdIAAICTcujOiLe3txISEpSent6sPT09XRMnTmx1n6qqKrm7N/8xHh4ekhrvqAAAgJ7N4W6aBQsW6MUXX9TKlSuVk5Oj+fPnKzc3197tsmjRIs2cOdO+/TXXXKP33ntPK1as0IEDB7Rx40bNnTtX48ePV2RkZMd9EgAA4JQc6qaRpBkzZqi0tFRLly5VQUGBRo0apbS0NMXExEiSCgoKms05cscdd6i8vFx/+ctf9Mtf/lJBQUG6/PLL9eSTT3bcpwAAAE7LzXCCvhKr1SqLxaKysjIFBgZ22HEHLlxr//rQE2d+1BgAADiurddv1qYBAACmIowAAABTEUYAAICpCCMAAMBUhBEAAGAqwggAADAVYQQAAJiKMAIAAExFGAEAAKYijAAAAFMRRgAAgKkcXigPAAA4v4qaeuUdr1Lu8SrZGgxNHREmLw9z7lEQRgAAcEG19Q06cuKkjp44qfxvG0NH7vGTyj1epbzjVTpeWdts+/+9bLB+fdVwU2oljAAA4KSs1XXKLW0MGodLq5R7vFKHSxu/Lig7qQaj7cfaV1TReYWeA2EEAIBuzFpdp0MllTrY9DpUUqmDpVU6cKxC5dX17TpmhMVX0X381DfQR9tyT+jGcf01Pja4gytvO8IIAAAmq6yp16HSSh0qqdLBkgodLKlqel+p0u91p7RFoK+nYkL8FdWnl/oH9dKAED9FB/tpQLCf+gf1kq+XRyd8ivYjjAAA0AVO1trsAeNA0x2OQ6WV2nmkTNV1De065rgBQRrSr7diQvw1oClsxIT4KcjPu4Or71yEEQAAOlBJRY32FpVrX1GF9haVa/+xiqYxHNUOHyss0EcDQ/wVG+qvgaH+9q9jQvy63d2N80EYAQCgHb6trNXeonLtLa7Q3sLyxgBSXNHiKZW2SIzpo4GhTaHDHj785OfdMy7TPeNTAgDQTmVVddpbXN7sbsfeogqVVNS0+Rh9/LzsYSM2xP+74BHqr94+XIo5AwAAqPGplX1NQeP04FFc3vbQ0TfAR0PDeiuuX4CGhgXYv7b4eXVi5c6PMAIA6FEqauq17/S7HMUV2ldU7tCYjhB/b8WF9dawsADFhTUGj7h+vdXH37kGjnYXhBEAgEsyDEP5355UdoFV2UetyimwKrvAqvxvT7b5GH38vJrCRu+mwNH4dUhvn06svOchjAAAnF51nU17i8obA8dRq3IKGr8ur2nbpGCWXl6NXSphARraryl4hAUotLe33NzcOrl6EEYAAE7lWHmNsguspwUPqw6UVMrWhrnP/bw9NCw84LTulcbg0S/Ah9BhIsIIAKBbqrc16GBJZWM3y2l3PNr6FEukxVfxEYEaERnY+N+IQA0I9pO7O6GjuyGMAABMZ62u0+6CcmUfLWvsYim0ak9huWrqzz0zqZeHm+L6BZwWPAI0IiLQ6WYh7ckIIwCALnP6oFJ7N0uhVXnH2zaotI+fV2PgCA+0h4/BfXvL29O9kytHZyKMAAA6TZG1WtvzTmhHfpm25zf+t+xk3Tn3c3OTYkP8v3e3w6KwQMZ2uCLCCACgQ5RV1WnHkabgkXdC2/NPqMh67vEdft4eGh4e0Gx8x/DwgB4zFToIIwCAdqius2nX0TJtz/vujsfBkspz7hfa20ejoywacVrwiGFQaY9HGAEAnFW9rUF7iyqaQscJbc8r056i8nM+Shvg46kLoiwaHRWkMVEWjYkOUoTFl24WtEAYAQDYGYahQ6VV2pF/QtuaxnrsOlqm6rqzP9Xi7emuERGBGhsdpNFNAWRQqD93PNAmhBEA6MGKrNVNoeO7sR7W6rPPWuruJg0NC7CHjrHRQRoaFsATLWg3wggA9BCnDzA9FUDaMsB0QLCfxkQ3drWMjgrSqP6BDC5Fh+K3CQBcUJ2tQTuPlCkr97u7Hm0ZYNo3wMceOsZEB2l0fwsr0aLTEUYAwAVU19mUlXtCWw4e15ZDpfrq8AmdrLOddZ8AX097V8upAabhgQwwRdcjjACAE6qoqVfm4W/1xYFSbTl4XNvzT6jOduanW7w93TUyMlBjooI0JroxgMSGMMAU3QNhBACcwLeVtfry0PGmOx/H9fWRMp3tydpIi68mDApRQkwfjY0O0rDwAHl5MMAU3RNhBAC6oWJrtb442BQ+Dh7XnqLys24/KNRf42OD7a+oPn5dVClw/ggjAGCyU4vHnQoeWw4dP+dg0+HhAd+Fj4HB6hfo20XVAh2PMAIAXcwwDO0/VtkUPhrHfBwtqz7j9h7ubhoVGdgUPkJ00cA+CvLjCRe4DsIIAHSB3NIqrd93TBu/KdGWg8dVWll7xm29Pdw1Jtqi8bHBmhAbonExfdTbh7+u4br47QaATmCtrtPm/aXK2HdMGftKdLi06ozb9vLyUEJMH3u3y9joIPl6eXRhtYC5CCMA0AFsDYZ25J9Qxr4Srd97TFl5J864kFyAr6fGD/xusOmo/haedEGPRhgBgHbKO16lDd+UKGPfMW3YV3LGNV28PNyUENNHyXF9lRwXqpGRFnkwvwdgRxgBgDaqqKm3d71s2FeiA2d54mVwX38lx/XVJUNDNSE2RP6M+QDOiD8dAHAGtgZDO4+UKWNv47iPr3K/Vf0Zul6C/Lw0aUioLokL1eS4vuof1KuLqwWcF2EEAE5z9MRJZew7pvX7SrTxmxKdqKprdTtPdzeNi+mjS+JClRzXV6P60/UCtBdhBECPZhiGvj5iVXpOkT7JLlJ2gfWM2w4K9VdyU/i4eHAIj9sCHYQ/SQB6nJp6mzbvL9UnOUX6JLtYhdbWJxwL9PXU5KbwMXlIqKKDmWId6AyEEQA9wreVtfp8T7HSs4u0fu8xVdbaWt1uTJRFlw8P0yVDQzU6KoiuF6ALEEYAuKxDJZVKzy5Sek6Rth463uoqt96e7po0OERTR4Trivh+CmONF6DLEUYAuAxbg6Fted8qPbtYn+QU6Zviila36+PnpcuHh2nqiDAlx4Xy2C1gMv4EAnBqJ2ttyth3TJ/kFOmz3cUqqWh9zZfYUH9NHdEYQMYN6EP3C9CNEEYAOJ2q2np9vvuY1u48qs92F6u6rqHFNm5uUmJMH02JD9OUEWEa3Le3CZUCaIt2hZHly5fr97//vQoKCjRy5Eg988wzSk5OPuP2NTU1Wrp0qf72t7+psLBQUVFRWrx4se666652Fw6gZ6mus+k/e4r1rx0F+jSnWCfrWg5A7eXloUuGhmpKfJguH95PIb19TKgUgKMcDiOrV6/WvHnztHz5ck2aNEl//etfNW3aNGVnZ2vAgAGt7nPLLbeoqKhIL730koYMGaLi4mLV17e+hgMAnFJdZ9O6vce0dkeBPs0pavUJmBB/b6WMbOx+mTg4lNVuASfkZhhG63Mbn8GECRM0btw4rVixwt4WHx+v66+/XqmpqS22//DDD/XjH/9YBw4cUHBwcLuKtFqtslgsKisrU2BgYLuO0ZqBC9favz70xPQOOy6A9quptyljb4nW7ixQenaRKmpa/sOlj5+XrhoVrukXROriQcHyZMVboFtq6/XboTsjtbW1yszM1MKFC5u1p6SkaNOmTa3u88EHHygxMVG/+93v9Prrr8vf31/XXnutHnvsMfXq1fraDTU1NaqpqWn2YQC4rtr6Bm38pkT/3HFU6buKVN5KALH08tJVI8M1fXSEkgaHyIsAArgMh8JISUmJbDabwsLCmrWHhYWpsLCw1X0OHDigDRs2yNfXV++//75KSkp077336vjx41q5cmWr+6SmpmrJkiWOlAbAydTZGgPI2h0F+mhXoazVLQNIgK+nrmwKIJMGh8rbkwACuKJ2DWB1c2v+SJxhGC3aTmloaJCbm5tWrVoli8UiSXr66ad18803a9myZa3eHVm0aJEWLFhgf2+1WhUdHd2eUgF0I/W2Bm0+UKq1Owr04a7CVhehC/Dx1NSRYfrh6AhNGhIqH0/GgACuzqEwEhoaKg8PjxZ3QYqLi1vcLTklIiJC/fv3twcRqXGMiWEYys/PV1xcXIt9fHx85OPDKHjAVeQUWLUmM1//2Hak1XlA/L09NHVEmKaPjlRyHINQgZ7GoTDi7e2thIQEpaen64YbbrC3p6en67rrrmt1n0mTJumdd95RRUWFevdufM5/7969cnd3V1RU1HmUDqA7O15Zq39sO6I1mfnadbTluC8/bw9dEd94B+TSoX0JIEAP5nA3zYIFC3T77bcrMTFRSUlJev7555Wbm6s5c+ZIauxiOXLkiF577TVJ0m233abHHntMd955p5YsWaKSkhI9+OCDuuuuu844gBWAc6qzNejz3cVak5mvz/cUq87W/GE9bw93XRHfT9eOidRlw/qplzcBBEA7wsiMGTNUWlqqpUuXqqCgQKNGjVJaWppiYmIkSQUFBcrNzbVv37t3b6Wnp+sXv/iFEhMTFRISoltuuUWPP/54x30KAKbKPvpdN0xpZctumDFRFt2cEKVrxkQqyM/bhAoBdGcOzzNiBuYZAbqfkooa/WPbUa3JzFdOQctumH4BPrphXH/dPC5KcWEBJlQIwGydMs8IgJ6ttr5Bn+9p6obZXaz6hpbdMFNHhunmhCglDwllMjIAbUIYAXBWhmFoV1M3zAfbj+p4K90wY6ODGrthRkfK4udlQpUAnBlhBECrKmrq9fesI1r1RW6r3TBhgT664cIo3ZzQX0P60Q0DoP0IIwCa2VdUrr/997De/epIi3VhvD3ddeXIcN2cEKXJQ0Ll4d76ZIcA4AjCCADV2RqUnl2k1zYf0n8PHG/x/THRQbolMUo/HB0pSy+6YQB0LMII0IMVWav15pZcvbklV0XWmmbf8/Vy1/Vj++t/Lo7RqP6WMxwBAM4fYQToYQzD0H8PHNff/ntYH+0qbPFEzKBQf/3PxTG6KSGKuyAAugRhBOghyqvr9H7WEb2++bD2FVc0+567mzR1RJhuv3igJg4OkTtjQQB0IcII4OL2FJbr9f8e0vtfHVFlra3Z90J7++jW8dG6dfwARQaxPAMAcxBGABdUW9+gj3YV6vXNh7XlUMsBqeMHBut/kmJ01chweXsyMRkAcxFGABdSUVOvN7/I1YsbDrQYkOrn7aEbLmwckBof0XHLKgDA+SKMAC7gWHmNXtl0UK9vPixrdfO5QYb0663bL47RjeP6K8CXAakAuh/CCODEckur9HzGfr2zNV819Q32djc3aWp8mO6YNFBJg0Lk5saAVADdF2EEcEJfHynTc+v2K21ngU5/MtfLw003XNhfP7tksIb0621egQDgAMII4CQMw9Dm/aVasW6/MvaVNPuev7eHbpswQLMnD1K4xdekCgGgfQgjQDdnGIY+zi7S8s+/0fb8smbfC+3trTsnxep/JsSwWi4Ap0UYAbqpUyHkT5/sU/b3Vs0dEOynn14ySD9KiJKvl4dJFQJAxyCMAN1MQ0NTCPl0n3K+F0JGRATqfy8brGmjwuXpwfwgAFwDYQToJs4WQkZHWTRvSpx+MKwfT8YAcDmEEcBkjSGkUM98sk+7C8ubfW9MlEXzpgzVZcP6EkIAuCzCCGASQggANCKMAF2socHQR7sK9adPWwkh0UGaNyVOlw0lhADoOQgjQBcxDEPr95XoyX/vbvF0DCEEQE9GGAG6wI78E3ri37u1aX9ps/axTSHkUkIIgB6MMAJ0okMllXrq4z36146CZu2j+gfqlynDuBMCACKMAJ2ipKJGz366T298kav60xaPGRDspwevHKbpF0TI3Z0QAgASYQToUBU19Xox44BeWH9AlbU2e3uIv7fmXhGnW8cPkLcnk5UBwOkII0AHqK1v0Ftf5urZT/eppKLW3u7n7aGfJg/STy8ZpN4+/HEDgNbwtyNwHhoaDK3dWaCnPt6jw6VV9nZPdzfdOn6A5l4Rp74BPiZWCADdH2EEaKdN35ToiQ93a8f3VtKdPjpCD6QMU2yov0mVAYBzIYwADsotrdKSf+7Sp7uLm7UnDQrRwmnDNSY6yJzCAMBJEUaANqqus2nFf/Zrxbr9qq1vsLfHRwRq4bThuiQulMd0AaAdCCNAG3ySXaQl/9qlvOMn7W1hgT769VXDdf3Y/jymCwDngTACnEVrXTKe7m6aPTlWv7gijidkAKAD8Dcp0IrqOpueW7dfy//TvEsmaVCIll43UnFhASZWBwCuhTACfM+nOUV69J8tu2Qemj5CPxwdwbgQAOhghBGgSbG1Wo/+c5fSdhba2+iSAYDOx9+u6PEaGgyt3pqn36blqLy63t5OlwwAdA3CCHq0/ccqtOi9ndpy8Li9LdjfWw//cISuGxtJlwwAdAHCCHqk2voGPb9+v5797JtmA1RvHNdfD00foWB/bxOrA4CehTCCHicr91stfHen9hSV29uig3vptzdcoOS4viZWBgA9E2EEPUZFTb2e+miPXt18SIbR2ObuJv00eZDmTRmqXt4e5hYIAD0UYQQ9wme7i/TQ+1/raFm1vW1kZKCevGm0RvW3mFgZAIAwApdmra7Tox/s0ntfHbG3+Xq5a8HUobprUqw8PdxNrA4AIBFG4MI27y/VA+9s15ET301eNnlIqH57wwUaEOJnYmUAgNMRRuByqutseuqjPXpxw0F7W4CPpx6+ZoRuTojicV0A6GYII3ApXx8p04K3t2lvUYW97eJBwXrqR2MU1Ye7IQDQHRFG4BJsDYaeW7dfz3yyV3W2xkdlvD3d9asrh+muSbFyd+duCAB0V4QROL3DpZVa8PZ2ZR7+1t4WHxGoZ2aM1bBwpnIHgO6OMAKnZRiGVn+Zp6X/ylZVrU1S47whcy4drHlThsrbkydlAMAZEEbglKzVdVr07k6t3VlgbxsQ7KenbxmjxIHBJlYGAHAUYQROZ0f+Cd33RpZyj1fZ224dH63F00eotw+/0gDgbPibG07DMAy9sumQfpuWYx+kGujrqd/dPEZXjQo3uToAQHsRRuAUyqrq9Kt3t+ujXUX2tjHRQfrLrRcqOphHdgHAmRFG0O1l5X6r+97IajaT6k+TY/XglcMZpAoALoAwgm7LMAy9mHFQT364W/UNjd0yQX5eeurmMZoyIszk6gAAHaVd/6xcvny5YmNj5evrq4SEBGVkZLRpv40bN8rT01Njx45tz49FD3KiqlZ3v7pV/5eWYw8iiTF9lDY3mSACAC7G4TCyevVqzZs3T4sXL1ZWVpaSk5M1bdo05ebmnnW/srIyzZw5U1dccUW7i0XPsLvQqmv/slGf7i62t9172WC9+bOLFRnUy8TKAACdweEw8vTTT2v27Nm6++67FR8fr2eeeUbR0dFasWLFWfe75557dNtttykpKandxcL1/WvHUd2wbJP9sd1gf2+9cudF+tVVw+XlwfgQAHBFDv3tXltbq8zMTKWkpDRrT0lJ0aZNm86438svv6z9+/frkUceadPPqampkdVqbfaCa7M1GHri37t13xtZOlnXOJvq6CiL/vWLybpsWD+TqwMAdCaHBrCWlJTIZrMpLKx5n31YWJgKCwtb3Wffvn1auHChMjIy5OnZth+XmpqqJUuWOFIanNiJqlrNfWub1u89Zm+7aVyU/u+GUfL18jCxMgBAV2jXfW83t+YroBqG0aJNkmw2m2677TYtWbJEQ4cObfPxFy1apLKyMvsrLy+vPWXCCZwaH3IqiHi4u+mRa0boqR+NJogAQA/h0J2R0NBQeXh4tLgLUlxc3OJuiSSVl5dr69atysrK0n333SdJamhokGEY8vT01Mcff6zLL7+8xX4+Pj7y8fFxpDQ4obSdBXrgne32Re6C/b217LZxShocYnJlAICu5FAY8fb2VkJCgtLT03XDDTfY29PT03Xddde12D4wMFA7d+5s1rZ8+XJ99tlnWrNmjWJjY9tZNpyZYRj64yf79Oyn++xto/oH6q+3J6o/T8sAQI/j8KRnCxYs0O23367ExEQlJSXp+eefV25urubMmSOpsYvlyJEjeu211+Tu7q5Ro0Y1279fv37y9fVt0Y6eobrOpl+t2aEPth+1t91wYX+l3ngB3TIA0EM5HEZmzJih0tJSLV26VAUFBRo1apTS0tIUExMjSSooKDjnnCPomUoranTP65naevhbSZKbm7T46njNnhzb6pgjAEDP4GYYhmF2EeditVplsVhUVlamwMDADjvuwIVr7V8femJ6hx0XLe0/VqE7X/7SPn9ILy8PPXvrhZrKbKoA4LLaev1mbRp0us37SzXnb5kqO1knSeoX4KOXZl2kC6IsJlcGAOgOCCPoVGsy87XovR2qszXegBseHqCVd1zEtO4AADvCCDqFYRj6Y/pePfvZN/a2y4b11V9uG6fePvzaAQC+w1UBHa7e1qDfvL9Tb2/Nt7fNTIrRwz8cIU/WlwEAfA9hBB3qZK1Nv3jzK32S07jirpub9ND0Ebpr0kCemAEAtIowgg5TVlWn2a9+aX9019vDXX+cMVbTR0eYXBkAoDsjjKBDFJZVa9bKLdpTVC5J6u3jqedvT9DEIaEmVwYA6O4IIzhv3xRXaNbKLTpy4qQkKbS3t165c7xG9efRXQDAuRFGcF625Z3QnS9v0bdVjXOIDAj202t3jdfAUH+TKwMAOAvCCNpt3d5jmvN6pk7WNa66Gx8RqFfvukj9AnxNrgwA4EwII2iXtJ0FmvtmluobGiczu3hQsJ6fmahAXy+TKwMAOBvCCBz2j21HtODt7bI1BZFpo8L1xxljWXUXANAuhBE45N3MfD24ZruacohuSYxS6o2j5eHOHCIAgPYhjKDN3v4yT79+b4dOrfP8kwkD9Nh1o+ROEAEAnAfCCNpk1ReHtfj9r+3vZyXF6NFrRzKrKgDgvBFGcE6vbDyoR/+ZbX8/e3KsHpoeTxABAHQIwgjO6sWMA3p8bY79/ZxLB+vXVw0jiAAAOgxhBGf06qZDzYLILy4fogVThxJEAAAdijCCVr21JVePfLDL/n7elDjNmzLUxIoAAK7K3ewC0P28n5WvRe/vtL+/7wdDCCIAgE5DGEEza3cU6Jdvb7c/vnv35Fj9MoUgAgDoPIQR2KVnF+n+t7LsE5rdfnGMFvPUDACgkxFGIKlx0bufr/rKvtbMLYlRWsI8IgCALkAYgTbvL9XPXtuqWluDJOm6sZFKvXE0M6sCALoEYaSH25F/QrNf/VI19Y1B5KqR4frDj8aw1gwAoMsQRnqwgyWVuvPlL1VVa5MkXT68n5699UJ5evBrAQDoOlx1eqji8mrNXPmFSitrJUnjBwZr+U/GyduTXwkAQNfiytMDlVfX6Y6VXyrv+ElJ0rCwAL0wK1G+Xh4mVwYA6IkIIz1MTb1N97yeqewCqySpf1AvvXrXeFl6eZlcGQCgpyKM9CANDYYWvL1dm/aXSpKC/Lz06l3jFW7xNbkyAEBPRhjpIQzD0GNrs7V2R4EkydfLXSvvuEhD+vU2uTIAQE9HGOkhVm48pJc3HpIkebi7aflPxmncgD7mFgUAgAgjPcIn2UV6fG22/X3qjRfo8uFhJlYEAMB3CCMu7usjZZr7VpZ94bu5lw/RLYnR5hYFAMBpCCMurLCsWne/utU+qdk1YyI1fyor8AIAuhfCiIuqrKnX7Fe/VKG1WpI0bkCQfn/zaBa+AwB0O4QRF2RrMHT/W9u062jjXCLRwb30wkwmNQMAdE+EEReUmpajT3KKJEkBvp5aOesihfT2MbkqAABaRxhxMW9/macXNxyU1PgI74qfJCguLMDkqgAAODPCiAv5KvdbPfT3r+3vH7tulCbHhZpYEQAA50YYcRHF1mrNeT1TtbYGSdKspBjdNmGAyVUBAHBuhBEXUFNv05y/Zaq4vEaSND42WA/9cITJVQEA0DaEERfw6AfZ+ir3hCQp0uKr5T8ZJy8P/tcCAJwDVywnt+qLw3pzS64kycfTXX+9PVGhPDkDAHAihBEntvXQcT36wS77+9QbL9AFURYTKwIAwHGEESdVZK3W/676SnW2xkVn7poUqxvHRZlcFQAAjiOMOKF6W4N+8WaWjjUNWJ04OES/uXq4yVUBANA+hBEn9MdP9mrLweOSpPBAX/351gvlyYBVAICT4grmZD7fU6xln++XJHm6u2nZTy5kqncAgFMjjDiRoydOav7qbfb3v7pqmBJigs0rCACADkAYcRJ1tgbd98ZXOlFVJ0maEt9PP00eZHJVAACcP8KIk/jdh7vtE5tF9emlP/xorNzc3MwtCgCADkAYcQIf7yrUCxmNK/F6ebhp2W3jZPHzMrkqAAA6BmGkmzty4qQeeGe7/f3iq+M1JjrIvIIAAOhghJFuzNZgaP7qbbJW10uSrr4gXLMmDjS3KAAAOhhhpBt7bt1++3wi/YN6KfXG0YwTAQC4nHaFkeXLlys2Nla+vr5KSEhQRkbGGbd97733NHXqVPXt21eBgYFKSkrSRx991O6Ce4rteSf0x/S9kiR3N+mPM8bK0otxIgAA1+NwGFm9erXmzZunxYsXKysrS8nJyZo2bZpyc3Nb3X79+vWaOnWq0tLSlJmZqR/84Ae65pprlJWVdd7Fu6rKmnrNW71N9Q2N687ce9kQjY9lPhEAgGtyMwzDcGSHCRMmaNy4cVqxYoW9LT4+Xtdff71SU1PbdIyRI0dqxowZevjhh9u0vdVqlcViUVlZmQIDAx0p96wGLlxr//rQE9M77Ljna+G7O/TWl3mSpDFRFq3534nyYrp3AICTaev126ErXG1trTIzM5WSktKsPSUlRZs2bWrTMRoaGlReXq7g4DP/S7+mpkZWq7XZq6f48OsCexDx8/bQMz++kCACAHBpDl3lSkpKZLPZFBYW1qw9LCxMhYWFbTrGH/7wB1VWVuqWW2454zapqamyWCz2V3R0tCNlOq0ia7UWvrfT/v7Ra0YqNtTfxIoAAOh87fon9/ef6DAMo01Pebz55pt69NFHtXr1avXr1++M2y1atEhlZWX2V15eXnvKdCqGYWjhuzvs071PGxWuHyVGmVwVAACdz9ORjUNDQ+Xh4dHiLkhxcXGLuyXft3r1as2ePVvvvPOOpkyZctZtfXx85OPTs1aifSczX5/vOSZJ6hvgo9/ecAGP8QIAegSH7ox4e3srISFB6enpzdrT09M1ceLEM+735ptv6o477tAbb7yh6dO7z0DR7uLoiZN67J/Z9vepN1ygPv7eJlYEAEDXcejOiCQtWLBAt99+uxITE5WUlKTnn39eubm5mjNnjqTGLpYjR47otddek9QYRGbOnKk//elPuvjii+13VXr16iWLxdKBH8U5GYahX7+7Q+U1jbOs3jQuSlNGnP0uEwAArsThMDJjxgyVlpZq6dKlKigo0KhRo5SWlqaYmBhJUkFBQbM5R/7617+qvr5eP//5z/Xzn//c3j5r1iy98sor5/8JnNybW/KUsa9EkhQe6KuHrxlhckUAAHQth+cZMYOrzjOSd7xKVz2zXpW1NknSK3depMuGnXlgLwAAzqRT5hlBx2loaOyeORVEfnxRNEEEANAjEUZMsuqLw9q0v1RS4yJ4i6fHm1wRAADmIIyY4OiJk3ri37vt75+8abQCfFkEDwDQMxFGuphhGHr4H7vs3TO3jo/W5LhQk6sCAMA8hJEu9u+vC/VJTpGkxsnNFk6jewYA0LMRRrpQ2ck6PfLBLvv7JdeOlKUX3TMAgJ6NMNKFnvj3bh0rr5EkTYnvp2mjwk2uCAAA8xFGusiWg8f15pbGyeD8vT209LpRrD0DAIAII12ipt6mRe/tsL9/8MphigzqZWJFAAB0H4SRLrDs8/3af6xSkjQ2Oki3Jw00tyAAALoRwkgnO3CsQiv+840kydPdTU/cdIE83OmeAQDgFMJIJzIMQ4/+M1t1tsblf352ySAND++4tXUAAHAFhJFO9NGuIq3fe0ySFGnx1X2XDzG5IgAAuh/CSCc5WWvTY//Ktr//fz8cIT9vTxMrAgCgeyKMdJIV//lGR06clCQlx4XqKuYUAQCgVYSRTnC4tFLPrT8gSfLycNOj145kThEAAM6AMNIJlvwzW7X1DZKkuybHanDf3iZXBABA90UYaWIYRocc55PsIn22u1iSFB7oq7mXx3XIcQEAcFWEkQ5UXWfTkn99txDe4unx8vdh0CoAAGdDGGnSETdGXtl0SHnHGwetJg0K0Q9HR5z/QQEAcHGEkQ5SUlGjZZ81zrTq7iY9cu0IBq0CANAGhJEm53tj5JlP9qq8pl6SNOOiaGZaBQCgjQgjTc5nAOu+onK98UWuJMnf20Pzpw7tqLIAAHB5hJEO8Nu0HDU0ZZl7fzBE/QJ8zS0IAAAnQhhp0t77Iuv3HtPne75bf2b25NiOKwoAgB6AMHIebA2G/m9tjv39r6cNl6+Xh4kVAQDgfAgjTdozZOTtrXnaU1QuSRoTHaRrRkd2cFUAALg+wkgTw8GOmqraej2dvtf+/v9Nj5e7O4/yAgDgKMJIO7288ZCOlddIkq4aGa7EgcEmVwQAgHMijDRxpJvm28paPfef/ZIaJzh74MphnVQVAACujzDSDs+t22+f4OxHCdEa0o9VeQEAaC/CiIMKyk7qlU2HJEnenu6aN5VVeQEAOB+EkSZt7ab50yf7VFPfIEm6Y+JARVh6dWJVAAC4PsJIk9zjVefc5pviCr29NU+SFODjqf+9dHBnlwUAgMsjjDSxNZz71sjT6Xvs077fc+kg9fH37uSqAABwfYSRJjvyT5z1+9vzTihtZ6EkKbS3j+5i2ncAADoEYaTJwvd26vn1+8/4/d9/tMf+9f1XDJGft2dXlAUAgMsjjJzmt2m71dBKd83Gb0q04ZsSSdKAYD/NuGhAV5cGAIDLIox8z5qv8pu9NwxDv/twt/39gqlD5e3JaQMAoKNwVf2eX63ZIeO053w/2lWo7fllkqTh4QG6dgyL4QEA0JEY+NCKtTsLdN8bWS3aH7xyGIvhAQDQwbgz0orWgkhCTB9dPryfCdUAAODaCCNtNH/KULm5cVcEAICORhhpsvfxaWf8Xh8/L02OC+3CagAA6DkYM9LE29Nd//rFZP3wzxskSRcPCtZbP0syuSoAAFwfYeQ0o/pbdOiJ6WaXAQBAj0I3DQAAMBVhBAAAmIowAgAATEUYAQAApiKMAAAAUxFGAACAqQgjAADAVIQRAABgKsIIAAAwFWEEAACYql1hZPny5YqNjZWvr68SEhKUkZFx1u3XrVunhIQE+fr6atCgQXruuefaVSwAAHA9DoeR1atXa968eVq8eLGysrKUnJysadOmKTc3t9XtDx48qKuvvlrJycnKysrSb37zG82dO1fvvvvueRcPAACcn5thGIYjO0yYMEHjxo3TihUr7G3x8fG6/vrrlZqa2mL7X//61/rggw+Uk5Njb5szZ462b9+uzZs3t+lnWq1WWSwWlZWVKTAw0JFyz2rgwrX2r1kgDwCAjtXW67dDd0Zqa2uVmZmplJSUZu0pKSnatGlTq/ts3ry5xfZXXnmltm7dqrq6ulb3qampkdVqbfYCAACuyaEwUlJSIpvNprCwsGbtYWFhKiwsbHWfwsLCVrevr69XSUlJq/ukpqbKYrHYX9HR0Y6UCQAAnEi7BrC6ubk1e28YRou2c23fWvspixYtUllZmf2Vl5fXnjLP6cIBQZKk+VOGdsrxAQDAuXk6snFoaKg8PDxa3AUpLi5ucffjlPDw8Fa39/T0VEhISKv7+Pj4yMfHx5HS2uX9eyd1+s8AAABn59CdEW9vbyUkJCg9Pb1Ze3p6uiZOnNjqPklJSS22//jjj5WYmCgvLy8HywUAAK7G4W6aBQsW6MUXX9TKlSuVk5Oj+fPnKzc3V3PmzJHU2MUyc+ZM+/Zz5szR4cOHtWDBAuXk5GjlypV66aWX9MADD3TcpwAAAE7LoW4aSZoxY4ZKS0u1dOlSFRQUaNSoUUpLS1NMTIwkqaCgoNmcI7GxsUpLS9P8+fO1bNkyRUZG6tlnn9VNN93UcZ8CAAA4LYfnGTFDZ80zAgAAOk+nzDMCAADQ0QgjAADAVIQRAABgKsIIAAAwFWEEAACYijACAABMRRgBAACmIowAAABTEUYAAICpHJ4O3gynJom1Wq0mVwIAANrq1HX7XJO9O0UYKS8vlyRFR0ebXAkAAHBUeXm5LBbLGb/vFGvTNDQ06OjRowoICJCbm1uHHddqtSo6Olp5eXmsedPJONddg/PcNTjPXYPz3DU68zwbhqHy8nJFRkbK3f3MI0Oc4s6Iu7u7oqKiOu34gYGB/KJ3Ec511+A8dw3Oc9fgPHeNzjrPZ7sjcgoDWAEAgKkIIwAAwFQ9Ooz4+PjokUcekY+Pj9mluDzOddfgPHcNznPX4Dx3je5wnp1iACsAAHBdPfrOCAAAMB9hBAAAmIowAgAATEUYAQAApnL5MLJ8+XLFxsbK19dXCQkJysjIOOv269atU0JCgnx9fTVo0CA999xzXVSpc3PkPL/33nuaOnWq+vbtq8DAQCUlJemjjz7qwmqdm6O/06ds3LhRnp6eGjt2bOcW6CIcPc81NTVavHixYmJi5OPjo8GDB2vlypVdVK3zcvQ8r1q1SmPGjJGfn58iIiJ05513qrS0tIuqdU7r16/XNddco8jISLm5uenvf//7Offp8muh4cLeeustw8vLy3jhhReM7Oxs4/777zf8/f2Nw4cPt7r9gQMHDD8/P+P+++83srOzjRdeeMHw8vIy1qxZ08WVOxdHz/P9999vPPnkk8aWLVuMvXv3GosWLTK8vLyMr776qosrdz6OnutTTpw4YQwaNMhISUkxxowZ0zXFOrH2nOdrr73WmDBhgpGenm4cPHjQ+OKLL4yNGzd2YdXOx9HznJGRYbi7uxt/+tOfjAMHDhgZGRnGyJEjjeuvv76LK3cuaWlpxuLFi413333XkGS8//77Z93ejGuhS4eR8ePHG3PmzGnWNnz4cGPhwoWtbv+rX/3KGD58eLO2e+65x7j44os7rUZX4Oh5bs2IESOMJUuWdHRpLqe953rGjBnGQw89ZDzyyCOEkTZw9Dz/+9//NiwWi1FaWtoV5bkMR8/z73//e2PQoEHN2p599lkjKiqq02p0NW0JI2ZcC122m6a2tlaZmZlKSUlp1p6SkqJNmza1us/mzZtbbH/llVdq69atqqur67RanVl7zvP3NTQ0qLy8XMHBwZ1Rosto77l++eWXtX//fj3yyCOdXaJLaM95/uCDD5SYmKjf/e536t+/v4YOHaoHHnhAJ0+e7IqSnVJ7zvPEiROVn5+vtLQ0GYahoqIirVmzRtOnT++KknsMM66FTrFQXnuUlJTIZrMpLCysWXtYWJgKCwtb3aewsLDV7evr61VSUqKIiIhOq9dZtec8f98f/vAHVVZW6pZbbumMEl1Ge871vn37tHDhQmVkZMjT02X/uHeo9pznAwcOaMOGDfL19dX777+vkpIS3XvvvTp+/DjjRs6gPed54sSJWrVqlWbMmKHq6mrV19fr2muv1Z///OeuKLnHMONa6LJ3Rk5xc3Nr9t4wjBZt59q+tXY05+h5PuXNN9/Uo48+qtWrV6tfv36dVZ5Laeu5ttlsuu2227RkyRINHTq0q8pzGY78Tjc0NMjNzU2rVq3S+PHjdfXVV+vpp5/WK6+8wt2Rc3DkPGdnZ2vu3Ll6+OGHlZmZqQ8//FAHDx7UnDlzuqLUHqWrr4Uu+0+l0NBQeXh4tEjYxcXFLRLfKeHh4a1u7+npqZCQkE6r1Zm15zyfsnr1as2ePVvvvPOOpkyZ0pllugRHz3V5ebm2bt2qrKws3XfffZIaL5qGYcjT01Mff/yxLr/88i6p3Zm053c6IiJC/fv3b7ZUenx8vAzDUH5+vuLi4jq1ZmfUnvOcmpqqSZMm6cEHH5QkjR49Wv7+/kpOTtbjjz/O3esOYsa10GXvjHh7eyshIUHp6enN2tPT0zVx4sRW90lKSmqx/ccff6zExER5eXl1Wq3OrD3nWWq8I3LHHXfojTfeoL+3jRw914GBgdq5c6e2bdtmf82ZM0fDhg3Ttm3bNGHChK4q3am053d60qRJOnr0qCoqKuxte/fulbu7u6Kiojq1XmfVnvNcVVUld/fmly0PDw9J3/3LHefPlGthpw2N7QZOPTb20ksvGdnZ2ca8efMMf39/49ChQ4ZhGMbChQuN22+/3b79qceZ5s+fb2RnZxsvvfQSj/a2gaPn+Y033jA8PT2NZcuWGQUFBfbXiRMnzPoITsPRc/19PE3TNo6e5/LyciMqKsq4+eabjV27dhnr1q0z4uLijLvvvtusj+AUHD3PL7/8suHp6WksX77c2L9/v7FhwwYjMTHRGD9+vFkfwSmUl5cbWVlZRlZWliHJePrpp42srCz7I9Td4Vro0mHEMAxj2bJlRkxMjOHt7W2MGzfOWLdunf17s2bNMi699NJm2//nP/8xLrzwQsPb29sYOHCgsWLFii6u2Dk5cp4vvfRSQ1KL16xZs7q+cCfk6O/06Qgjbefoec7JyTGmTJli9OrVy4iKijIWLFhgVFVVdXHVzsfR8/zss88aI0aMMHr16mVEREQYP/nJT4z8/Pwurtq5fP7552f9O7c7XAvdDIN7WwAAwDwuO2YEAAA4B8IIAAAwFWEEAACYijACAABMRRgBAACmIowAAABTEUYAAICpCCMAAMBUhBEAAGAqwggAADAVYQQAAJiKMAIAAEz1/wFQiY264h9+dAAAAABJRU5ErkJggg==",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.plot(recallsE, precisionsE, linewidth=2, label=\"Precision/Recall curve\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1262,
   "id": "5fb8c0c7-2185-4c06-b8b3-586138149385",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[<matplotlib.lines.Line2D at 0x7f761d626390>]"
      ]
     },
     "execution_count": 1262,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.plot(recallsI, precisionsI, linewidth=2, label=\"Precision/Recall curve\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1263,
   "id": "65161291-86ff-4869-ad27-59955c455c43",
   "metadata": {},
   "outputs": [],
   "source": [
    "fpr, tpr, thresholds = roc_curve(people_labels, people_scores, pos_label=\"Extrovert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1264,
   "id": "051bb926-9e2f-442d-981d-8e92ca0922f1",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[<matplotlib.lines.Line2D at 0x7f761d78eed0>]"
      ]
     },
     "execution_count": 1264,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAiMAAAGdCAYAAADAAnMpAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAMg5JREFUeJzt3X1wVHWe7/FPP6Q7DyThIRASCBFUEGVHx3BBQNY7jsaLXqe8O1tS5V5RF7dMObMKjM7KsqWj11vUzOxQjjOCzghaU4suV0envPeySu69O4jAjAMDjgvM4shDAiTEBEiHPHT64dw/On262wRIN0mfPqffr6oufzk5p/PtU8z8Pv07v/M7LsMwDAEAAFjEbXUBAAAgvxFGAACApQgjAADAUoQRAABgKcIIAACwFGEEAABYijACAAAsRRgBAACW8lpdwHBEo1GdOnVKpaWlcrlcVpcDAACGwTAMdXV1qbq6Wm73hcc/bBFGTp06pZqaGqvLAAAAGWhubtbUqVMv+HtbhJHS0lJJsQ9TVlZmcTUAAGA4AoGAampqzH78QmwRRuKXZsrKyggjAADYzKWmWDCBFQAAWIowAgAALEUYAQAAliKMAAAASxFGAACApQgjAADAUoQRAABgKcIIAACwFGEEAABYKu0w8uGHH+ruu+9WdXW1XC6XfvWrX13ymO3bt6uurk6FhYWaMWOGXn755UxqBQAADpR2GOnu7tb111+vn/70p8Pa/+jRo7rzzju1ePFi7du3T3//93+vxx57TL/85S/TLhYAADhP2s+mWbJkiZYsWTLs/V9++WVNmzZNL7zwgiRp9uzZ2rNnj/7xH/9R3/zmN9P98wAAwGFGfc7I7t27VV9fn7Ltjjvu0J49exQKhYY8JhgMKhAIpLwAAMDI+957B/Qf/vv/0aot+y2rYdTDSGtrqyorK1O2VVZWKhwOq729fchj1q5dq/LycvNVU1Mz2mUCAJB3jrV36/Vdx/RFV1Cfnuy0rI6s3E3z5UcHG4Yx5Pa41atXq7Oz03w1NzePeo0AAOSbj4+eMduftZ23rI6054yka/LkyWptbU3Z1tbWJq/XqwkTJgx5jN/vl9/vH+3SAADIa+d6+832vXOnWlbHqI+MLFiwQI2NjSnbtm3bprlz56qgoGC0/zwAALiA/7HnhNm+ZeYky+pIO4ycP39e+/fv1/79+yXFbt3dv3+/mpqaJMUusSxbtszcv6GhQcePH9eqVat06NAhbdq0SRs3btQTTzwxMp8AAABkpLc/YrZnTS61rI60L9Ps2bNHX/va18yfV61aJUl64IEH9Prrr6ulpcUMJpI0ffp0bd26VStXrtRLL72k6upqvfjii9zWCwCAxdxJQxJXTRpjWR0uIz6bNIcFAgGVl5ers7NTZWVlVpcDAIDthSNRXbXmXyRJVeWF2r366yP+N4bbf/NsGgAA8lDyrbwe99B3t2YLYQQAgDz0v/7QYrZvqBlrXSEijAAAkJfO9iRu651Vad3kVYkwAgBAXnInLTz6tWusu61XIowAAJCX3vl9Yo2R8iJr1/0ijAAAkIeiSffSTiy1dtVzwggAAHkmFImm/FxY4LGokhjCCAAAeeZ4R7fVJaQgjAAAkGeaz/aa7b+4cYqFlcQQRgAAyDPbDpw221PGFllYSQxhBACAPBMMJR6Q92dTyi2sJIYwAgBAngkmTWCdXWX9M98IIwAA5Jn/d6jNbPu91kcB6ysAAABZVeL3mu0yixc8kwgjAADknf5wYs6I1WuMSIQRAADyimEYCvSFJUnXTLb2AXlxhBEAAPJIW1fQbHd0919kz+whjAAAkEd+c6TDbM+oKLGwkgTCCAAAeeR//6HFbF9bbf1tvRJhBACAvLLtYGL11f84a5KFlSQQRgAAyFM3X1VhdQmSCCMAAOSVYl/iVl6P22VhJQmEEQAA8oRhGOrpj60x8pWp1j+TJo4wAgBAnjiTdCvv+WDYwkpSEUYAAMgT8VERSQpHDAsrSUUYAQAgT4SjiQBSVzvOwkpSEUYAAMgT4UjUbHtzZPKqRBgBACBvJI+MeD2EEQAAkGXNZ3rMttedOxEgdyoBAACj6vMvus322Z7ceEieRBgBACBvBMOJu2muq2adEQAAkGVG0t28syaPsa6QLyGMAACQJyJJE1j9Xs9F9swuwggAAHki+W6aXHkujUQYAQAgb0SirDMCAAAs9PHRM2abkREAAJB140p8Zps5IwAAIOs8rsRoSGWZ38JKUhFGAADIE+d6Q2bb582dCJA7lQAAgFG19/hZs13gyZ0IkDuVAACAUTVlbJHZ9jMyAgAAsu3kuV5JUnlRgVwu7qYBAABZFIok1hgpK/JaWMlghBEAAPJAbyjxkLzmM70WVjIYYQQAgDwQjiSWgr/1mkkWVjIYYQQAgDwQTloKPpdWX5UIIwAA5IXkkZECD2EEAABkWWugz2x73LnV/edWNQAAYFT830OnzXbH+aCFlQxGGAEAIA+8/2+tZvvPppRbWMlghBEAAPLA1HHFZvu/3DjFwkoGI4wAAJAHth/+wmzPqBhjYSWDEUYAAMgDFWP8Zpu7aQAAQNZFktYZyaXn0kiEEQAAHM8wDJ3tCUmSZlWWWlzNYIQRAAAc7lhHj9k+HwxbWMnQCCMAADhcMJx4SN7k8kILKxlaRmFk/fr1mj59ugoLC1VXV6cdO3ZcdP/Nmzfr+uuvV3FxsaqqqvTQQw+po6Mjo4IBAEB6ItHEUvCzJjvgMs2WLVu0YsUKrVmzRvv27dPixYu1ZMkSNTU1Dbn/Rx99pGXLlmn58uU6cOCA3nrrLf3ud7/Tww8/fNnFAwCAS0uauypPjk1elTIII+vWrdPy5cv18MMPa/bs2XrhhRdUU1OjDRs2DLn/b37zG11xxRV67LHHNH36dN1888165JFHtGfPnssuHgAAXFrESIyM5NoTe6U0w0h/f7/27t2r+vr6lO319fXatWvXkMcsXLhQJ06c0NatW2UYhk6fPq23335bd9111wX/TjAYVCAQSHkBAIDMJF+mcdt9ZKS9vV2RSESVlZUp2ysrK9Xa2jrkMQsXLtTmzZu1dOlS+Xw+TZ48WWPHjtVPfvKTC/6dtWvXqry83HzV1NSkUyYAAEgSTRkZsbCQC8iopC8vlmIYxgUXUDl48KAee+wxPf3009q7d6/ef/99HT16VA0NDRd8/9WrV6uzs9N8NTc3Z1ImAACQFAonJo24c/AyjTednSsqKuTxeAaNgrS1tQ0aLYlbu3atFi1apCeffFKS9JWvfEUlJSVavHixnn/+eVVVVQ06xu/3y+/3D9oOAADS98GBRL9dXJBW158VaY2M+Hw+1dXVqbGxMWV7Y2OjFi5cOOQxPT09crtT/4zH45EUG1EBAACjy5PUD1ePdcA6I6tWrdKrr76qTZs26dChQ1q5cqWamprMyy6rV6/WsmXLzP3vvvtuvfPOO9qwYYOOHDminTt36rHHHtO8efNUXV09cp8EAABc0oyJufXEXinNyzSStHTpUnV0dOi5555TS0uL5syZo61bt6q2tlaS1NLSkrLmyIMPPqiuri799Kc/1Xe+8x2NHTtWt956q77//e+P3KcAAAAXlPyQvFx7Yq8kuQwbXCsJBAIqLy9XZ2enysrKrC4HAABbWfPup9r829hAwf9+7GZdV12elb873P47B2/wAQAAI6k76eF4Xnfudf25VxEAABhRvz78hdm2/QqsAADAfq5JejjepLLcWzqDMAIAgMP95sgZs13qt/k6IwAAwH68SZdmLrRiupUIIwAAOFxRgcfqEi6KMAIAgMN1DdxNM7sqN5fHIIwAAOBg7eeDZtvnzc1uPzerAgAAI2LbgdNm+3Rnn4WVXBhhBAAAB9tzLHEnzV/cOMXCSi6MMAIAgIP1RxLPpfkPV4y3sJILI4wAAOBguz/vMNtXVJRYWMmFEUYAAHCwiaWJFVenjiuysJILI4wAAOBgf2ztMtsFntzs9nOzKgAAcNmiUcPqEoaFMAIAgEM1n+0x2/4cXWNEIowAAOBYwXDiTpr5MyZYWMnFEUYAAHCocz0hs107vtjCSi6OMAIAgEPtazprtvtCEQsruTjCCAAADnXgVMBs105gZAQAAGTZe5+cMttzc3T1VYkwAgCAY40v8ZntOVPKLazk4ggjAAA4kGEYOtPdb/48xu+1sJqLI4wAAOBAHUlBxON2WVjJpRFGAABwoOMd3Wb75qsqLKzk0ggjAAA4UHcwcStvb3/u3tYrEUYAAHCk7mDYbN8ya6KFlVwaYQQAAAc6lPS0Xi9zRgAAQLZ19iQmsPpy+CF5EmEEAABHOh0Imu0ZE8dYWMmlEUYAAHAgf0Gii68uL7SwkksjjAAA4EChSNRsjynM3QXPJMIIAACOtPXTVrNd4Mnt7j63qwMAABmZMbHEbJcyMgIAALIt+WZev9djWR3DQRgBAMCBTp7rlSRNSHpyb64ijAAA4DDBcER9odgE1lyfLyIRRgAAcJwvuhJrjAT6QhZWMjyEEQAAHGZf0zmzfdefVVlXyDARRgAAcJhNO4+a7YhhWFjJ8BBGAABwGLcrcS/NX3x1qoWVDA9hBAAAh0m+rfemGeMtq2O4CCMAADhMfCl4l0vycjcNAADItk9OdEqSfDYIIhJhBAAAx8r9qasxhBEAABwkeV2R/nD0InvmDsIIAAAOcuZ8v9kuy/EH5MURRgAAcJD+SGI05D/NmWxhJcNHGAEAwEFaOvvMts9rj27eHlUCAIBh6QmGzXZrZ/Aie+YOwggAAA7S0Z2YM7LgygkWVjJ8hBEAABzkYEvAbBcVeCysZPgIIwAAOMg/f9xktseXFFhYyfARRgAAcJBo0kpn11WXW1dIGggjAAA4yOSyQrNdM77YwkqGjzACAICDtHXFbu2dZpMgImUYRtavX6/p06ersLBQdXV12rFjx0X3DwaDWrNmjWpra+X3+3XllVdq06ZNGRUMAACGFo0a5mUau6wxIklprxO7ZcsWrVixQuvXr9eiRYv0yiuvaMmSJTp48KCmTZs25DH33nuvTp8+rY0bN+qqq65SW1ubwuHwkPsCAIDMnOlJ3Nb7p7bzFlaSnrTDyLp167R8+XI9/PDDkqQXXnhBH3zwgTZs2KC1a9cO2v/999/X9u3bdeTIEY0fP16SdMUVV1xe1QAAYJDPkwLIn8+caGEl6UlrDKe/v1979+5VfX19yvb6+nrt2rVryGPee+89zZ07Vz/4wQ80ZcoUzZw5U0888YR6e3sv+HeCwaACgUDKCwAAXFwokriVxu2ysJA0pTUy0t7erkgkosrKypTtlZWVam1tHfKYI0eO6KOPPlJhYaHeffddtbe369FHH9WZM2cuOG9k7dq1evbZZ9MpDQCAvNcbipjtumnjLKwkPRnNbnG5UuOWYRiDtsVFo1G5XC5t3rxZ8+bN05133ql169bp9ddfv+DoyOrVq9XZ2Wm+mpubMykTAIC88m8nO812gVMnsFZUVMjj8QwaBWlraxs0WhJXVVWlKVOmqLw8sfDK7NmzZRiGTpw4oauvvnrQMX6/X36/P53SAADIe6WFiW49krz6WY5LKzb5fD7V1dWpsbExZXtjY6MWLlw45DGLFi3SqVOndP58YlLN4cOH5Xa7NXXq1AxKBgAAQwmGo2b76kljLKwkPWmP4axatUqvvvqqNm3apEOHDmnlypVqampSQ0ODpNgllmXLlpn733fffZowYYIeeughHTx4UB9++KGefPJJ/fVf/7WKiopG7pMAAJDnfnOkw2w7ep2RpUuXqqOjQ88995xaWlo0Z84cbd26VbW1tZKklpYWNTUlHtIzZswYNTY26m//9m81d+5cTZgwQffee6+ef/75kfsUAABAE0sTUxzs8sReSXIZhpHzF5UCgYDKy8vV2dmpsrIyq8sBACAn1f23RnV0xxY+2/XUraoea+0ViOH23/YZwwEAABeVPIE1uZ3rCCMAADhEf9IE1tLCAgsrSQ9hBAAAhzjVGXtib8UYey2PQRgBAMABQpHEqEj7+aCFlaSPMAIAgAO0DoyKSNL1NWOtKyQDhBEAABygszeUaPf0W1hJ+ggjAAA4QDCceEje12cP/YiWXEUYAQDAAf7Ulnjsip1WX5UIIwAAOILL5TLb53pCF9kz9xBGAABwgOQ7aObWjrOwkvQRRgAAcIDDrV1m2+N2XWTP3EMYAQDAAcaV+Mz2+KS2HRBGAABwgEg08dzbscX2WQpeIowAAOAIoUgijHjd9ure7VUtAAAYUlsgsQJrgYc5IwAAIMv+cLLTbDOBFQAAZN2MihKzXVVeZGEl6SOMAADgAOGkCayswAoAALKuOxiWJLldXKYBAAAW+OPAomcFHvt17farGAAApPiiK7EUfInfa2ElmSGMAABgc384cc5sX1tVZl0hGSKMAABgc3uOnzXbk8r8FlaSGcIIAAA250+6e+arNWOtKyRDhBEAAGwuGI6a7SsnjbGwkswQRgAAsLm9SZdpfNxNAwAAsi15WZHCAo91hWSIMAIAgM3928mA2a4ZV2xhJZkhjAAAYHPnB1ZflaQiHyMjAAAgi/pCkZSf7fZcGokwAgCArfX2J8LI2OICCyvJHGEEAAAb600aGVkwY4KFlWSOMAIAgI01n+kx2+GoYWElmSOMAABgY/2RxIJndlxjRCKMAABga39s6TLbs6tKLawkc4QRAABs7NeH28y2y+W6yJ65izACAICNVZUXme262nEWVpI5wggAADYWSpozUlVeaGElmSOMAABgY//empgz4vfab/VViTACAICt/XEgjHjdLpUWei2uJjOEEQAAbCqcdIkmHDVU4ieMAACALGrp7DPbN19VYWEll4cwAgCATe09ftZsf9EVtLCSy0MYAQDAps4Hw2Z7wZX2fC6NRBgBAMC2PjjQarZvqBlrXSGXiTACAIBNuZNWXK0eW3SRPXMbYQQAAJvafvgLs33VpDEWVnJ5CCMAADjAuOICq0vIGGEEAAAbCoYjKT/b9SF5EmEEAABbOtSSWAZ+VmWphZVcPsIIAAA2tPNP7Wa7vMi+l2gkwggAALb0x6QH5P3n66ssrOTyEUYAALCh9qQVV6+tKrOwkstHGAEAwIZ2H+kw2zMm2ve2XokwAgCALdVOKDbbzBkBAABZFwpHzbbHbd/beqUMw8j69es1ffp0FRYWqq6uTjt27BjWcTt37pTX69UNN9yQyZ8FAAADTnX2SZKmjrPvMvBxaYeRLVu2aMWKFVqzZo327dunxYsXa8mSJWpqarrocZ2dnVq2bJm+/vWvZ1wsAACQIlHDbHttPioiZRBG1q1bp+XLl+vhhx/W7Nmz9cILL6impkYbNmy46HGPPPKI7rvvPi1YsCDjYgEAgNTdHzbbxzp6LKxkZKQVRvr7+7V3717V19enbK+vr9euXbsueNxrr72mzz//XM8888yw/k4wGFQgEEh5AQCAmP6k+SK3zZ5kYSUjI60w0t7erkgkosrKypTtlZWVam1tHfKYzz77TE899ZQ2b94sr9c7rL+zdu1alZeXm6+ampp0ygQAwNHOdPebbZ/X/veiZPQJvvwwHsMwhnxATyQS0X333adnn31WM2fOHPb7r169Wp2dnearubk5kzIBAHCkQG/IbP+p7byFlYyM4Q1VDKioqJDH4xk0CtLW1jZotESSurq6tGfPHu3bt0/f/va3JUnRaFSGYcjr9Wrbtm269dZbBx3n9/vl9/vTKQ0AgLwRiiQmsP7HWXl2mcbn86murk6NjY0p2xsbG7Vw4cJB+5eVlenTTz/V/v37zVdDQ4NmzZql/fv3a/78+ZdXPQAAeSgcTcwZ8Xnsf5kmrZERSVq1apXuv/9+zZ07VwsWLNDPfvYzNTU1qaGhQVLsEsvJkyf1i1/8Qm63W3PmzEk5ftKkSSosLBy0HQAADE8okggjXo/9b+1NO4wsXbpUHR0deu6559TS0qI5c+Zo69atqq2tlSS1tLRccs0RAACQuX9vTcwT8QwxZ9NuXIZhGJfezVqBQEDl5eXq7OxUWZm9n0wIAMDlen3nUX3vfx6UJP3trVfpO/WzLK5oaMPtv+1/oQkAgDzTn3SZZnaV/b+kE0YAALCZ3x45Y7YLHDCB1f6fAACAPJO80NkYf9rTP3MOYQQAAJtpGXhiryRdX1NuYSUjgzACAIDN9IUikmJrjBT7GBkBAABZFl8OvsjnsbiSkUEYAQDARsKRqE4NXKYZV1xgcTUjgzACAICNHOvoNtuR3F8qbFgIIwAA2EjyQ/KunDjGwkpGDmEEAAAbCSeFkZpxxRZWMnIIIwAA2EjyE3s9bvs/l0YijAAAYCuBvrDZLnDAE3slwggAALay51hiKfiyQu6mAQAAWfbazmNmu7aixLpCRhBhBAAAGzkfTFymuWXmRAsrGTmEEQAAbCIUiab8XF7EZRoAAJBFwXAijCyYMcHCSkYWYQQAAJto7wqa7TGF9n9AXhxhBAAAmzjT02+2J5b6LaxkZBFGAACwiXd/f9JsO2W+iEQYAQDANpJXXy0u8FhYycgijAAAYBPJy79/7ZpJFlYysggjAADYRPOZXrPt9zqnC3fOJwEAwOEOn+4y2z7CCAAAyLaaccVme3J5oYWVjCzCCAAANtGftAKrz+OcLtw5nwQAAIfrC0UkSQUel1wu1yX2tg/CCAAANvHH1ticEa/bWd23sz4NAAAOVuqPLQHfOzBC4hSEEQAAbCIcNSRJY4uds/qqRBgBAMA24iuwVpcXWVzJyCKMAABgE/GRkQKPcyavSoQRAABsoT8clRHLIvI66LZeiTACAIAtNJ/tsbqEUUMYAQDABv7pN8fN9szKUgsrGXmEEQAAbGDnn9rN9sRSv4WVjDzCCAAANjA16bk0/3X+NAsrGXmEEQAAbOD//bHNbJcVsc4IAADIssqyxKUZv9dZ3bezPg0AAHnASQ/JkwgjAADYQsf5fknSFROKL7Gn/RBGAADIceeDYXP1VZ/DLtFIhBEAAHLeqXO9ZrsvFLWwktFBGAEAIMedTAojt8ycaGElo4MwAgBAjuvqC5ttj9tZk1clwggAADnvRNJzaZjACgAAsu7k2cRlGn+Bx8JKRgdhBACAHNcfTkxanTaekREAAJBlHx87Y7bHFjtrKXiJMAIAQM5LHg2pKi+ysJLRQRgBACDHJV+mKfYxZwQAAGTZb48mLtP4PM7rup33iQAAcDA364wAAIBsG+fASavJCCMAAOS4sz0hSdLVk8ZYXMnoIIwAAJDDzvX0m20nPrFXyjCMrF+/XtOnT1dhYaHq6uq0Y8eOC+77zjvv6Pbbb9fEiRNVVlamBQsW6IMPPsi4YAAA8knzmcTqq4G+kIWVjJ60w8iWLVu0YsUKrVmzRvv27dPixYu1ZMkSNTU1Dbn/hx9+qNtvv11bt27V3r179bWvfU1333239u3bd9nFAwDgdEc7us32nXOqLKxk9LgMwzDSOWD+/Pm68cYbtWHDBnPb7Nmzdc8992jt2rXDeo/rrrtOS5cu1dNPPz2s/QOBgMrLy9XZ2amysrJ0ygUAwNZ+ufeEvvPWJ5Kkv7hxitbde4O1BaVhuP13WiMj/f392rt3r+rr61O219fXa9euXcN6j2g0qq6uLo0fP/6C+wSDQQUCgZQXAAD5KBJNjBl8ddo4CysZPWmFkfb2dkUiEVVWVqZsr6ysVGtr67De40c/+pG6u7t17733XnCftWvXqry83HzV1NSkUyYAAI4RiiZWX/UzgTXB5UpdcMUwjEHbhvLmm2/qe9/7nrZs2aJJkyZdcL/Vq1ers7PTfDU3N2dSJgAAtpc8MuJ14IJnkuRNZ+eKigp5PJ5BoyBtbW2DRku+bMuWLVq+fLneeust3XbbbRfd1+/3y+/3p1MaAACOtPvzDrPtdeBS8FKaIyM+n091dXVqbGxM2d7Y2KiFCxde8Lg333xTDz74oN544w3dddddmVUKAEAe+td/bzPbpYVpjSHYRtqfatWqVbr//vs1d+5cLViwQD/72c/U1NSkhoYGSbFLLCdPntQvfvELSbEgsmzZMv34xz/WTTfdZI6qFBUVqby8fAQ/CgAAzvNnU8r1u2NnJUk3TZ9gcTWjI+0wsnTpUnV0dOi5555TS0uL5syZo61bt6q2tlaS1NLSkrLmyCuvvKJwOKxvfetb+ta3vmVuf+CBB/T6669f/icAAMDB4kHE7ZKKfB6Lqxkdaa8zYgXWGQEA5KNzPf264bnY1AiXSzq61l5THUZlnREAAJA9gd6w2c79oYPMEUYAAMhR/ZHEGiPfvHGqhZWMLsIIAAA5qi8UMds+rzPXGJEIIwAA5KwTZ3vMdldf+CJ72hthBACAnJUYDRlf4rOwjtFFGAEAIEclzxmpnVBiYSWjizACAECOau3sNds+D3NGAABAlp0OBM12f8S59/YSRgAAyFHFSSuuTh1XZGElo4swAgBAjjrS3m22xxUzgRUAAGTZ8Y5EGPF5ndtlO/eTAQBgc8faE+uMTBtfbGElo4swAgBAjipIuoNmjN9rYSWjizACAECOKvYlAgiXaQAAQNaFo7FFzyaXFVpcyegijAAAkKPCA2uLeB284JlEGAEAIGd1dPdLkgo8zu6unf3pAACwqb5QxGw7+Ym9EmEEAICc1B1MBJBQ0gPznIgwAgBADjrXGzLbN19VYWElo48wAgBADmpLekjeiXO9F9nT/ggjAADkoPhtvZJ047Sx1hWSBYQRAAByUPKcESc/JE8ijAAAkJMOnz5vdQlZQxgBACAHlSQ9i4Z1RgAAQNaFk27nvWKCc5/YKxFGAADIScFwIox4GRkBAADZ9ovdx802z6YBAABZ5/cmuugpY4ssrGT0EUYAAMhByeuMXD1pjIWVjD7CCAAAOSgSNSRJU8cVyeXiMg0AAMiy8EAYcfptvRJhBACAnBSJxMKIx+3sURGJMAIAQE7qGlgO3ksYAQAA2dbTn3guTShp8TOnIowAAJBjzvWEzPbpQNDCSrKDMAIAQI45cbbXbN9+baWFlWQHYQQAgBzTfj4xGtLR3W9hJdlBGAEAIMd8fPSM2f7zqyssrCQ7CCMAAOSYw6e7zPbYYp+FlWQHYQQAgByTvNDZddVlFlaSHYQRAAByzPbDX5jtyrJCCyvJDsIIAAA55MyXJqyOLSqwqJLsIYwAAJBDkhc8m1Tql5sVWAEAQDaFBp5JI0k3X+X8O2kkwggAADklGI6Y7Xx4Yq9EGAEAIKcca+8x28nBxMkIIwAA5ChGRgAAQNb94cQ5sz27yvlrjEiEEQAAcsr6X39utksLvRZWkj2EEQAActSt10yyuoSsIIwAAJAjktcYKSxwa8IYv4XVZE9+jP8AADCKDMNQKGKoPxJV1DBkRKWoYShqGIoYhgwj/rMUjcZ+7u4PKxI1FAxH1X4+qEjU0JbfNZvv6fd6LPxE2UUYAQBclGEYCkcNRQY60chAJxsd2BYd6Gh7+iMKRaLqD0fV0d0vj8tldsix/RMdsjHQSUcNqS3Qp2KfV26XZCi2jzGwTzT5v4q34527ZMhQV19YX3QFVVnmV2Tgb0SiidcnJ86pZnyxCtwuRQbCQHyf+H/7I1GdD0YUCkeTapb5OZPbpzr7NMYf6z5Dkagi0dj5GWn/5atTRvw9cxVhBACyoC8UiXWOA9+akzv06EAHH45E1ReKxjr0SFQd5/vldbvMTjPWcQ4cO9ABNp/pUVlRgT5pPqeq8kKFIobC0ahOnetVR3e/qsuL1B+JKhyJauefOnR15RgVeNwp72d2zoah7mBYpwNBVYzxKxSJqrM3ZPWpGxF/ONE5ou93Phi+9E6X6R7CCACkJxyJqicUUaA3lPQNONbRSsnfLGPfZpOHrQ3DUKAvrAK3y/wGHv+2GYpEdTrQp9JCb0pHnPyt9o+tXZo2vjjl23ZyR/+bIx26ZnJZ4ht+Umd+rL1bvaGIqscWxd436Xfx964s88vndSsajX3WxDdzw/y2nvxzNBrfHvtdXzj27dkK+3Qu5ecDpwLDOq79fHAUqsl9HrdLBR6X3C6XPC6XXC7J7Y63XXK7Yvu0dPZJkq6ZXKoCj1set0set0t7j5/Vn8+cKLdLcg/s7xp4L7dbA+8R2x6KRHXqXJ9unDZO/gK3Tgf6dM3kUhX5vLrj2kpNyoOn9cZlFEbWr1+vH/7wh2ppadF1112nF154QYsXL77g/tu3b9eqVat04MABVVdX67vf/a4aGhoyLhrIVLyzSe7sogNDtP0Dw7OJTk4p30gDvWG53TKPC0cMRaJRtXT2qbSwINYhKbljinW60YFeKd7pxoahE/sZMnTki25VjIk9ECv5W2qs0411cqfO9ao3FNGk0kJFjUTt4WhU54NhfdLcqa9MLR/USV5ymNuIdbh+r1uVZYUpQ+TxIfPYfl/altThZuNb4uW6VCd84mzvBX93OuC8jjneqcY7ylg7ts3jcqmju1+TSv0aV+xTgdelfzsZ0PU1Y1Xi88Q6U/dAx5zUjkQNNZ/p1bzp4+X1uHS8o0dzppSbHXP8b8Q7Y3dSx9zWFdSUcUUqcLulgd+5pFgHrthx8TDgUryTj21zSQpHDfm9bhX7vGYwiH8W98CtGiUDv4uHC3dS0HC7XfJ73SosyJ95Grkk7TCyZcsWrVixQuvXr9eiRYv0yiuvaMmSJTp48KCmTZs2aP+jR4/qzjvv1N/8zd/on/7pn7Rz5049+uijmjhxor75zW+OyIfA8EWjhkLRqKLRwdd9IwPfWoPhiDnUGwrHOuqz3f3yemLXf2Od8MAQ8dkelRcVqD8cVVtXUIHekMYV+7703snfkg3tOXZWsyaXxupJ+QabdC154P13fNaueVeMT/mWHe9Y40Eh3vl+1nZefq/b/PvJ14zt0FmOhF2fd2R8bDAcVdOZnkvvmAdcLsnrdpkdaE9/bEnuijF+sxOMd5YuV6KDTd2e2nkWFrg1ptBrvmdyR2z+7HbpyBfnddWkMZpQ4pfX41LzmR5dV1020OEnOk7PQEfqcrnUEwxr2vhiuVzSxNJCFXhcKvC45Xa5VOL3yOd1q8Dtltfjkt/rUYEndhyQK1yGYaQ1djh//nzdeOON2rBhg7lt9uzZuueee7R27dpB+//d3/2d3nvvPR06dMjc1tDQoE8++US7d+8e1t8MBAIqLy9XZ2enyspGZzW6vlDEHJZNHqqNd2r9A8Os8c4zPpErMVM6MUv6bE9Ifq/b7ODjHWJ0YLZ185keVYzxmX8v/t9DLQFNLiuUe+AacfIwdCRq6OOjZzRrcqnCA/tGooYmlRUmDSnHavz8i265XdL4Er8i0ajCkdjve0P58YwDXFhyZ+l2uQa+gcbavaGIDEOqGOMzv20mDzO7kjrhoTrcL7qC8nndmjK2SD39Ec2uKov9Xknfgt2Skn9Oem/DkJrOxL5Je1wueQeGyr0Dl256QxFNGVtoduaepLDgdrnUF4powhifGQqSv7HHay4rKjC/KXvdbrndsf28brcKfe6kEQI6amAkDLf/TmtkpL+/X3v37tVTTz2Vsr2+vl67du0a8pjdu3ervr4+Zdsdd9yhjRs3KhQKqaCgYNAxwWBQwWBiWDQQGN41znT09kdU/8J2ne8L62yPfSZoHWnvTvk5ft3yy6KGc6/5pgzvupM6R5fU0x+bJDhlbNEQQ7UuHWoJqK52nAo8roHOyGV+A/7DiXOae8U4FXo9KUO35nC2pKMdPbphark8A98yPQPHt3T2aXpFSdK34yGGlofaljTcfD4YUcUYX0q9nqTO1ON2KWpIY/xeeeO/S6rf4459+3YPdKTJnX38vwCQi9IKI+3t7YpEIqqsrEzZXllZqdbW1iGPaW1tHXL/cDis9vZ2VVVVDTpm7dq1evbZZ9MpLW1+r1vNZy58jdhOvEmdUrzzDYai6g1FdMWE4oEOM9Z5hiOG/v10lxbMmGB2ZsnXfeO34h0+3aWbZkyQd2C4tzsYlmFI08YXy+OJ/y23PC7pTE9IMypK5PO6FY4aKi30yu91J3XmqZ1qvMMu/tK15+Rh6+TtPq87FhzoUAHAkTKawPrlDsEwjIt2EkPtP9T2uNWrV2vVqlXmz4FAQDU1NZmUekFut0s144vkkktj/F6d6uxVeVGBaieUyJN0DTfRYbt04GSnvjptnHxed8p13uQh4fi332Pt3Zozpczs4JO/pbskBfrCmjquSF53Yha21+1SMBzR+BK/PG4peQja63ENDF27VeL3yO/xyF/AZCsAgP2lFUYqKirk8XgGjYK0tbUNGv2Imzx58pD7e71eTZgwYchj/H6//P7RXwJ3x3dvHfW/AQAALi6tZ9P4fD7V1dWpsbExZXtjY6MWLlw45DELFiwYtP+2bds0d+7cIeeLAACA/JL2g/JWrVqlV199VZs2bdKhQ4e0cuVKNTU1meuGrF69WsuWLTP3b2ho0PHjx7Vq1SodOnRImzZt0saNG/XEE0+M3KcAAAC2lfackaVLl6qjo0PPPfecWlpaNGfOHG3dulW1tbWSpJaWFjU1NZn7T58+XVu3btXKlSv10ksvqbq6Wi+++CJrjAAAAEkZrDNihWysMwIAAEbWcPvvtC/TAAAAjCTCCAAAsBRhBAAAWIowAgAALEUYAQAAliKMAAAASxFGAACApQgjAADAUoQRAABgqbSXg7dCfJHYQCBgcSUAAGC44v32pRZ7t0UY6erqkiTV1NRYXAkAAEhXV1eXysvLL/h7WzybJhqN6tSpUyotLZXL5Rqx9w0EAqqpqVFzczPPvBllnOvs4DxnB+c5OzjP2TGa59kwDHV1dam6ulpu94VnhthiZMTtdmvq1Kmj9v5lZWX8Q88SznV2cJ6zg/OcHZzn7Bit83yxEZE4JrACAABLEUYAAICl8jqM+P1+PfPMM/L7/VaX4nic6+zgPGcH5zk7OM/ZkQvn2RYTWAEAgHPl9cgIAACwHmEEAABYijACAAAsRRgBAACWcnwYWb9+vaZPn67CwkLV1dVpx44dF91/+/btqqurU2FhoWbMmKGXX345S5XaWzrn+Z133tHtt9+uiRMnqqysTAsWLNAHH3yQxWrtLd1/03E7d+6U1+vVDTfcMLoFOkS65zkYDGrNmjWqra2V3+/XlVdeqU2bNmWpWvtK9zxv3rxZ119/vYqLi1VVVaWHHnpIHR0dWarWnj788EPdfffdqq6ulsvl0q9+9atLHpP1vtBwsH/+5382CgoKjJ///OfGwYMHjccff9woKSkxjh8/PuT+R44cMYqLi43HH3/cOHjwoPHzn//cKCgoMN5+++0sV24v6Z7nxx9/3Pj+979vfPzxx8bhw4eN1atXGwUFBcbvf//7LFduP+me67hz584ZM2bMMOrr643rr78+O8XaWCbn+Rvf+IYxf/58o7Gx0Th69Kjx29/+1ti5c2cWq7afdM/zjh07DLfbbfz4xz82jhw5YuzYscO47rrrjHvuuSfLldvL1q1bjTVr1hi//OUvDUnGu+++e9H9regLHR1G5s2bZzQ0NKRsu+aaa4ynnnpqyP2/+93vGtdcc03KtkceecS46aabRq1GJ0j3PA/l2muvNZ599tmRLs1xMj3XS5cuNf7hH/7BeOaZZwgjw5Duef6Xf/kXo7y83Ojo6MhGeY6R7nn+4Q9/aMyYMSNl24svvmhMnTp11Gp0muGEESv6Qsdepunv79fevXtVX1+fsr2+vl67du0a8pjdu3cP2v+OO+7Qnj17FAqFRq1WO8vkPH9ZNBpVV1eXxo8fPxolOkam5/q1117T559/rmeeeWa0S3SETM7ze++9p7lz5+oHP/iBpkyZopkzZ+qJJ55Qb29vNkq2pUzO88KFC3XixAlt3bpVhmHo9OnTevvtt3XXXXdlo+S8YUVfaIsH5WWivb1dkUhElZWVKdsrKyvV2to65DGtra1D7h8Oh9Xe3q6qqqpRq9euMjnPX/ajH/1I3d3duvfee0ejRMfI5Fx/9tlneuqpp7Rjxw55vY79n/uIyuQ8HzlyRB999JEKCwv17rvvqr29XY8++qjOnDnDvJELyOQ8L1y4UJs3b9bSpUvV19encDisb3zjG/rJT36SjZLzhhV9oWNHRuJcLlfKz4ZhDNp2qf2H2o5U6Z7nuDfffFPf+973tGXLFk2aNGm0ynOU4Z7rSCSi++67T88++6xmzpyZrfIcI51/09FoVC6XS5s3b9a8efN05513at26dXr99dcZHbmEdM7zwYMH9dhjj+npp5/W3r179f777+vo0aNqaGjIRql5Jdt9oWO/KlVUVMjj8QxK2G1tbYMSX9zkyZOH3N/r9WrChAmjVqudZXKe47Zs2aLly5frrbfe0m233TaaZTpCuue6q6tLe/bs0b59+/Ttb39bUqzTNAxDXq9X27Zt06233pqV2u0kk3/TVVVVmjJlSsqj0mfPni3DMHTixAldffXVo1qzHWVynteuXatFixbpySeflCR95StfUUlJiRYvXqznn3+e0esRYkVf6NiREZ/Pp7q6OjU2NqZsb2xs1MKFC4c8ZsGCBYP237Ztm+bOnauCgoJRq9XOMjnPUmxE5MEHH9Qbb7zB9d5hSvdcl5WV6dNPP9X+/fvNV0NDg2bNmqX9+/dr/vz52SrdVjL5N71o0SKdOnVK58+fN7cdPnxYbrdbU6dOHdV67SqT89zT0yO3O7Xb8ng8khLf3HH5LOkLR21qbA6I3za2ceNG4+DBg8aKFSuMkpIS49ixY4ZhGMZTTz1l3H///eb+8duZVq5caRw8eNDYuHEjt/YOQ7rn+Y033jC8Xq/x0ksvGS0tLebr3LlzVn0E20j3XH8Zd9MMT7rnuaury5g6darxl3/5l8aBAweM7du3G1dffbXx8MMPW/URbCHd8/zaa68ZXq/XWL9+vfH5558bH330kTF37lxj3rx5Vn0EW+jq6jL27dtn7Nu3z5BkrFu3zti3b595C3Uu9IWODiOGYRgvvfSSUVtba/h8PuPGG280tm/fbv7ugQceMG655ZaU/X/9618bX/3qVw2fz2dcccUVxoYNG7JcsT2lc55vueUWQ9Kg1wMPPJD9wm0o3X/TyQgjw5fueT506JBx2223GUVFRcbUqVONVatWGT09PVmu2n7SPc8vvviice211xpFRUVGVVWV8Vd/9VfGiRMnsly1vfzrv/7rRf8/Nxf6QpdhMLYFAACs49g5IwAAwB4IIwAAwFKEEQAAYCnCCAAAsBRhBAAAWIowAgAALEUYAQAAliKMAAAASxFGAACApQgjAADAUoQRAABgKcIIAACw1P8HMkUoODZRBaIAAAAASUVORK5CYII=",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.plot(fpr,tpr, linewidth=2, label=\"ROC curve\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1265,
   "id": "c202ba08-00f2-4a5a-ad3b-9ae7788e035b",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "np.float64(0.9581809798907451)"
      ]
     },
     "execution_count": 1265,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.metrics import roc_auc_score\n",
    "\n",
    "roc_auc_score(people_labels, people_scores)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f32064f3-65e0-4359-8d0a-c193876bae29",
   "metadata": {},
   "source": [
    "<h2>Random forest Classifier</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1266,
   "id": "04359b25-75b5-4e81-aa01-2711b0f747ac",
   "metadata": {},
   "outputs": [],
   "source": [
    "forest_clf = RandomForestClassifier(random_state=42)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1267,
   "id": "04d7f628-3820-44e6-b64c-9f1f5ba507d8",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<style>#sk-container-id-45 {\n",
       "  /* Definition of color scheme common for light and dark mode */\n",
       "  --sklearn-color-text: #000;\n",
       "  --sklearn-color-text-muted: #666;\n",
       "  --sklearn-color-line: gray;\n",
       "  /* Definition of color scheme for unfitted estimators */\n",
       "  --sklearn-color-unfitted-level-0: #fff5e6;\n",
       "  --sklearn-color-unfitted-level-1: #f6e4d2;\n",
       "  --sklearn-color-unfitted-level-2: #ffe0b3;\n",
       "  --sklearn-color-unfitted-level-3: chocolate;\n",
       "  /* Definition of color scheme for fitted estimators */\n",
       "  --sklearn-color-fitted-level-0: #f0f8ff;\n",
       "  --sklearn-color-fitted-level-1: #d4ebff;\n",
       "  --sklearn-color-fitted-level-2: #b3dbfd;\n",
       "  --sklearn-color-fitted-level-3: cornflowerblue;\n",
       "\n",
       "  /* Specific color for light theme */\n",
       "  --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, white)));\n",
       "  --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-icon: #696969;\n",
       "\n",
       "  @media (prefers-color-scheme: dark) {\n",
       "    /* Redefinition of color scheme for dark theme */\n",
       "    --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, #111)));\n",
       "    --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-icon: #878787;\n",
       "  }\n",
       "}\n",
       "\n",
       "#sk-container-id-45 {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 pre {\n",
       "  padding: 0;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 input.sk-hidden--visually {\n",
       "  border: 0;\n",
       "  clip: rect(1px 1px 1px 1px);\n",
       "  clip: rect(1px, 1px, 1px, 1px);\n",
       "  height: 1px;\n",
       "  margin: -1px;\n",
       "  overflow: hidden;\n",
       "  padding: 0;\n",
       "  position: absolute;\n",
       "  width: 1px;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-dashed-wrapped {\n",
       "  border: 1px dashed var(--sklearn-color-line);\n",
       "  margin: 0 0.4em 0.5em 0.4em;\n",
       "  box-sizing: border-box;\n",
       "  padding-bottom: 0.4em;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-container {\n",
       "  /* jupyter's `normalize.less` sets `[hidden] { display: none; }`\n",
       "     but bootstrap.min.css set `[hidden] { display: none !important; }`\n",
       "     so we also need the `!important` here to be able to override the\n",
       "     default hidden behavior on the sphinx rendered scikit-learn.org.\n",
       "     See: https://github.com/scikit-learn/scikit-learn/issues/21755 */\n",
       "  display: inline-block !important;\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-text-repr-fallback {\n",
       "  display: none;\n",
       "}\n",
       "\n",
       "div.sk-parallel-item,\n",
       "div.sk-serial,\n",
       "div.sk-item {\n",
       "  /* draw centered vertical line to link estimators */\n",
       "  background-image: linear-gradient(var(--sklearn-color-text-on-default-background), var(--sklearn-color-text-on-default-background));\n",
       "  background-size: 2px 100%;\n",
       "  background-repeat: no-repeat;\n",
       "  background-position: center center;\n",
       "}\n",
       "\n",
       "/* Parallel-specific style estimator block */\n",
       "\n",
       "#sk-container-id-45 div.sk-parallel-item::after {\n",
       "  content: \"\";\n",
       "  width: 100%;\n",
       "  border-bottom: 2px solid var(--sklearn-color-text-on-default-background);\n",
       "  flex-grow: 1;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-parallel {\n",
       "  display: flex;\n",
       "  align-items: stretch;\n",
       "  justify-content: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-parallel-item {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-parallel-item:first-child::after {\n",
       "  align-self: flex-end;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-parallel-item:last-child::after {\n",
       "  align-self: flex-start;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-parallel-item:only-child::after {\n",
       "  width: 0;\n",
       "}\n",
       "\n",
       "/* Serial-specific style estimator block */\n",
       "\n",
       "#sk-container-id-45 div.sk-serial {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "  align-items: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  padding-right: 1em;\n",
       "  padding-left: 1em;\n",
       "}\n",
       "\n",
       "\n",
       "/* Toggleable style: style used for estimator/Pipeline/ColumnTransformer box that is\n",
       "clickable and can be expanded/collapsed.\n",
       "- Pipeline and ColumnTransformer use this feature and define the default style\n",
       "- Estimators will overwrite some part of the style using the `sk-estimator` class\n",
       "*/\n",
       "\n",
       "/* Pipeline and ColumnTransformer style (default) */\n",
       "\n",
       "#sk-container-id-45 div.sk-toggleable {\n",
       "  /* Default theme specific background. It is overwritten whether we have a\n",
       "  specific estimator or a Pipeline/ColumnTransformer */\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "/* Toggleable label */\n",
       "#sk-container-id-45 label.sk-toggleable__label {\n",
       "  cursor: pointer;\n",
       "  display: flex;\n",
       "  width: 100%;\n",
       "  margin-bottom: 0;\n",
       "  padding: 0.5em;\n",
       "  box-sizing: border-box;\n",
       "  text-align: center;\n",
       "  align-items: start;\n",
       "  justify-content: space-between;\n",
       "  gap: 0.5em;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 label.sk-toggleable__label .caption {\n",
       "  font-size: 0.6rem;\n",
       "  font-weight: lighter;\n",
       "  color: var(--sklearn-color-text-muted);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 label.sk-toggleable__label-arrow:before {\n",
       "  /* Arrow on the left of the label */\n",
       "  content: \"▸\";\n",
       "  float: left;\n",
       "  margin-right: 0.25em;\n",
       "  color: var(--sklearn-color-icon);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 label.sk-toggleable__label-arrow:hover:before {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "/* Toggleable content - dropdown */\n",
       "\n",
       "#sk-container-id-45 div.sk-toggleable__content {\n",
       "  max-height: 0;\n",
       "  max-width: 0;\n",
       "  overflow: hidden;\n",
       "  text-align: left;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-toggleable__content.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-toggleable__content pre {\n",
       "  margin: 0.2em;\n",
       "  border-radius: 0.25em;\n",
       "  color: var(--sklearn-color-text);\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-toggleable__content.fitted pre {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 input.sk-toggleable__control:checked~div.sk-toggleable__content {\n",
       "  /* Expand drop-down */\n",
       "  max-height: 200px;\n",
       "  max-width: 100%;\n",
       "  overflow: auto;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 input.sk-toggleable__control:checked~label.sk-toggleable__label-arrow:before {\n",
       "  content: \"▾\";\n",
       "}\n",
       "\n",
       "/* Pipeline/ColumnTransformer-specific style */\n",
       "\n",
       "#sk-container-id-45 div.sk-label input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-label.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator-specific style */\n",
       "\n",
       "/* Colorize estimator box */\n",
       "#sk-container-id-45 div.sk-estimator input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-estimator.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-label label.sk-toggleable__label,\n",
       "#sk-container-id-45 div.sk-label label {\n",
       "  /* The background is the default theme color */\n",
       "  color: var(--sklearn-color-text-on-default-background);\n",
       "}\n",
       "\n",
       "/* On hover, darken the color of the background */\n",
       "#sk-container-id-45 div.sk-label:hover label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "/* Label box, darken color on hover, fitted */\n",
       "#sk-container-id-45 div.sk-label.fitted:hover label.sk-toggleable__label.fitted {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator label */\n",
       "\n",
       "#sk-container-id-45 div.sk-label label {\n",
       "  font-family: monospace;\n",
       "  font-weight: bold;\n",
       "  display: inline-block;\n",
       "  line-height: 1.2em;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-label-container {\n",
       "  text-align: center;\n",
       "}\n",
       "\n",
       "/* Estimator-specific */\n",
       "#sk-container-id-45 div.sk-estimator {\n",
       "  font-family: monospace;\n",
       "  border: 1px dotted var(--sklearn-color-border-box);\n",
       "  border-radius: 0.25em;\n",
       "  box-sizing: border-box;\n",
       "  margin-bottom: 0.5em;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-estimator.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "/* on hover */\n",
       "#sk-container-id-45 div.sk-estimator:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-45 div.sk-estimator.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Specification for estimator info (e.g. \"i\" and \"?\") */\n",
       "\n",
       "/* Common style for \"i\" and \"?\" */\n",
       "\n",
       ".sk-estimator-doc-link,\n",
       "a:link.sk-estimator-doc-link,\n",
       "a:visited.sk-estimator-doc-link {\n",
       "  float: right;\n",
       "  font-size: smaller;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1em;\n",
       "  height: 1em;\n",
       "  width: 1em;\n",
       "  text-decoration: none !important;\n",
       "  margin-left: 0.5em;\n",
       "  text-align: center;\n",
       "  /* unfitted */\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted,\n",
       "a:link.sk-estimator-doc-link.fitted,\n",
       "a:visited.sk-estimator-doc-link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "div.sk-estimator:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "div.sk-estimator.fitted:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "/* Span, style for the box shown on hovering the info icon */\n",
       ".sk-estimator-doc-link span {\n",
       "  display: none;\n",
       "  z-index: 9999;\n",
       "  position: relative;\n",
       "  font-weight: normal;\n",
       "  right: .2ex;\n",
       "  padding: .5ex;\n",
       "  margin: .5ex;\n",
       "  width: min-content;\n",
       "  min-width: 20ex;\n",
       "  max-width: 50ex;\n",
       "  color: var(--sklearn-color-text);\n",
       "  box-shadow: 2pt 2pt 4pt #999;\n",
       "  /* unfitted */\n",
       "  background: var(--sklearn-color-unfitted-level-0);\n",
       "  border: .5pt solid var(--sklearn-color-unfitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted span {\n",
       "  /* fitted */\n",
       "  background: var(--sklearn-color-fitted-level-0);\n",
       "  border: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link:hover span {\n",
       "  display: block;\n",
       "}\n",
       "\n",
       "/* \"?\"-specific style due to the `<a>` HTML tag */\n",
       "\n",
       "#sk-container-id-45 a.estimator_doc_link {\n",
       "  float: right;\n",
       "  font-size: 1rem;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1rem;\n",
       "  height: 1rem;\n",
       "  width: 1rem;\n",
       "  text-decoration: none;\n",
       "  /* unfitted */\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 a.estimator_doc_link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "#sk-container-id-45 a.estimator_doc_link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "#sk-container-id-45 a.estimator_doc_link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "</style><div id=\"sk-container-id-45\" class=\"sk-top-container\"><div class=\"sk-text-repr-fallback\"><pre>RandomForestClassifier(random_state=42)</pre><b>In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. <br />On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.</b></div><div class=\"sk-container\" hidden><div class=\"sk-item\"><div class=\"sk-estimator fitted sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-45\" type=\"checkbox\" checked><label for=\"sk-estimator-id-45\" class=\"sk-toggleable__label fitted sk-toggleable__label-arrow\"><div><div>RandomForestClassifier</div></div><div><a class=\"sk-estimator-doc-link fitted\" rel=\"noreferrer\" target=\"_blank\" href=\"https://scikit-learn.org/1.6/modules/generated/sklearn.ensemble.RandomForestClassifier.html\">?<span>Documentation for RandomForestClassifier</span></a><span class=\"sk-estimator-doc-link fitted\">i<span>Fitted</span></span></div></label><div class=\"sk-toggleable__content fitted\"><pre>RandomForestClassifier(random_state=42)</pre></div> </div></div></div></div>"
      ],
      "text/plain": [
       "RandomForestClassifier(random_state=42)"
      ]
     },
     "execution_count": 1267,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "forest_clf.fit(people_data,people_labels)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1268,
   "id": "8eed8032-e52e-4c84-8d17-b4eb3071fed3",
   "metadata": {},
   "outputs": [],
   "source": [
    "y_pred_forest = cross_val_predict(forest_clf, people_data, people_labels, cv=3)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1269,
   "id": "f95d2731-f5ad-4b68-a888-24875cf3ad9b",
   "metadata": {},
   "outputs": [],
   "source": [
    "y_probas_forest = cross_val_predict(forest_clf, people_data, people_labels, cv=3, method=\"predict_proba\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1270,
   "id": "a2030ed7-8e28-41d8-bd45-6d7b250b6ff3",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[0.92054762, 0.07945238],\n",
       "       [1.        , 0.        ]])"
      ]
     },
     "execution_count": 1270,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "y_probas_forest[:2]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1271,
   "id": "788d9f56-e1aa-4f2a-b50e-4c92a26789f6",
   "metadata": {},
   "outputs": [],
   "source": [
    "y_scores_forest = y_probas_forest[:,1]\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1272,
   "id": "c2f912bd-7548-48df-b84b-3f30ebbb767c",
   "metadata": {},
   "outputs": [],
   "source": [
    "precisions_forest, recalls_forest,threshold_forest = precision_recall_curve(people_labels,y_scores_forest,pos_label=\"Extrovert\")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1273,
   "id": "f45a05a5-df86-4477-bb1d-4dcca3e5a2a7",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.plot(recalls_forest, precisions_forest, \"b-\", linewidth=2, label=\"Random Forest\")\n",
    "plt.plot(recallsE,precisionsE, \"--\", linewidth=2, label=\"SGD\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1274,
   "id": "20c3c4a7-22ef-4dbd-8b4f-0a2498b81f8c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9753068381063705"
      ]
     },
     "execution_count": 1274,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f1_score(people_labels,y_pred_forest,pos_label=\"Extrovert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1275,
   "id": "b0804baa-901f-4a7a-be33-5c0031bb451e",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9301075268817204"
      ]
     },
     "execution_count": 1275,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f1_score(people_labels,y_pred_forest,pos_label=\"Introvert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1276,
   "id": "ba40925e-9c56-4984-9b63-6aa027274f9c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "np.float64(0.9621629126894403)"
      ]
     },
     "execution_count": 1276,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "roc_auc_score(people_labels, y_scores_forest)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9ed3ea6e-91d6-4d1c-ae6d-5963692777c9",
   "metadata": {},
   "source": [
    "<p>The score for random forest is better by a small margin</p>\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "66d8db26-0ca0-4b31-be7e-4f159e55bf86",
   "metadata": {},
   "source": [
    "<h2>Preprocess the test data</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1277,
   "id": "970d202a-ee67-478f-be1b-bb6bb02f1bec",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test = pd.read_csv('dataset/test.csv')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1278,
   "id": "320f1891-b523-4be7-ba2d-070ceb8c88cf",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Stage_fear</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Drained_after_socializing</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>18524</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>7.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>No</td>\n",
       "      <td>6.0</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>18525</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Yes</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>5.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>18526</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>5.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>No</td>\n",
       "      <td>15.0</td>\n",
       "      <td>9.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>18527</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>No</td>\n",
       "      <td>5.0</td>\n",
       "      <td>6.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>18528</td>\n",
       "      <td>9.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>2.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6170</th>\n",
       "      <td>24694</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>5.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>No</td>\n",
       "      <td>9.0</td>\n",
       "      <td>6.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6171</th>\n",
       "      <td>24695</td>\n",
       "      <td>8.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>2.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6172</th>\n",
       "      <td>24696</td>\n",
       "      <td>2.0</td>\n",
       "      <td>No</td>\n",
       "      <td>4.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>9.0</td>\n",
       "      <td>7.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6173</th>\n",
       "      <td>24697</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>No</td>\n",
       "      <td>11.0</td>\n",
       "      <td>9.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6174</th>\n",
       "      <td>24698</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>6175 rows × 8 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "         id  Time_spent_Alone Stage_fear  Social_event_attendance  \\\n",
       "0     18524               3.0         No                      7.0   \n",
       "1     18525               NaN        Yes                      0.0   \n",
       "2     18526               3.0         No                      5.0   \n",
       "3     18527               3.0         No                      4.0   \n",
       "4     18528               9.0        Yes                      1.0   \n",
       "...     ...               ...        ...                      ...   \n",
       "6170  24694               3.0         No                      5.0   \n",
       "6171  24695               8.0        Yes                      2.0   \n",
       "6172  24696               2.0         No                      4.0   \n",
       "6173  24697               3.0         No                      4.0   \n",
       "6174  24698               NaN        Yes                      1.0   \n",
       "\n",
       "      Going_outside Drained_after_socializing  Friends_circle_size  \\\n",
       "0               4.0                        No                  6.0   \n",
       "1               0.0                       Yes                  5.0   \n",
       "2               6.0                        No                 15.0   \n",
       "3               4.0                        No                  5.0   \n",
       "4               2.0                       Yes                  1.0   \n",
       "...             ...                       ...                  ...   \n",
       "6170            5.0                        No                  9.0   \n",
       "6171            1.0                       Yes                  0.0   \n",
       "6172            3.0                        No                  9.0   \n",
       "6173            4.0                        No                 11.0   \n",
       "6174            1.0                       Yes                  1.0   \n",
       "\n",
       "      Post_frequency  \n",
       "0                NaN  \n",
       "1                1.0  \n",
       "2                9.0  \n",
       "3                6.0  \n",
       "4                1.0  \n",
       "...              ...  \n",
       "6170             6.0  \n",
       "6171             0.0  \n",
       "6172             7.0  \n",
       "6173             9.0  \n",
       "6174             0.0  \n",
       "\n",
       "[6175 rows x 8 columns]"
      ]
     },
     "execution_count": 1278,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1279,
   "id": "b487dc84-afbc-4c73-ac95-1208f2ff92f3",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test_num = people_test.select_dtypes(include=[np.number])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1280,
   "id": "f5a8fe7c-a4cb-4ddd-9b76-a25c18b6ec0d",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>18524</td>\n",
       "      <td>3.0</td>\n",
       "      <td>7.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>18525</td>\n",
       "      <td>NaN</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>18526</td>\n",
       "      <td>3.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>15.0</td>\n",
       "      <td>9.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>18527</td>\n",
       "      <td>3.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>6.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>18528</td>\n",
       "      <td>9.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>2.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6170</th>\n",
       "      <td>24694</td>\n",
       "      <td>3.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>9.0</td>\n",
       "      <td>6.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6171</th>\n",
       "      <td>24695</td>\n",
       "      <td>8.0</td>\n",
       "      <td>2.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6172</th>\n",
       "      <td>24696</td>\n",
       "      <td>2.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>9.0</td>\n",
       "      <td>7.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6173</th>\n",
       "      <td>24697</td>\n",
       "      <td>3.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>11.0</td>\n",
       "      <td>9.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6174</th>\n",
       "      <td>24698</td>\n",
       "      <td>NaN</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>6175 rows × 6 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "         id  Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "0     18524               3.0                      7.0            4.0   \n",
       "1     18525               NaN                      0.0            0.0   \n",
       "2     18526               3.0                      5.0            6.0   \n",
       "3     18527               3.0                      4.0            4.0   \n",
       "4     18528               9.0                      1.0            2.0   \n",
       "...     ...               ...                      ...            ...   \n",
       "6170  24694               3.0                      5.0            5.0   \n",
       "6171  24695               8.0                      2.0            1.0   \n",
       "6172  24696               2.0                      4.0            3.0   \n",
       "6173  24697               3.0                      4.0            4.0   \n",
       "6174  24698               NaN                      1.0            1.0   \n",
       "\n",
       "      Friends_circle_size  Post_frequency  \n",
       "0                     6.0             NaN  \n",
       "1                     5.0             1.0  \n",
       "2                    15.0             9.0  \n",
       "3                     5.0             6.0  \n",
       "4                     1.0             1.0  \n",
       "...                   ...             ...  \n",
       "6170                  9.0             6.0  \n",
       "6171                  0.0             0.0  \n",
       "6172                  9.0             7.0  \n",
       "6173                 11.0             9.0  \n",
       "6174                  1.0             0.0  \n",
       "\n",
       "[6175 rows x 6 columns]"
      ]
     },
     "execution_count": 1280,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test_num"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1281,
   "id": "e75273c3-dc53-4772-a884-eca9d84e15c8",
   "metadata": {},
   "outputs": [],
   "source": [
    "imputer = SimpleImputer(strategy=\"median\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1282,
   "id": "4900c810-07cc-4dc1-b6e4-8a308c15fb7a",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<style>#sk-container-id-46 {\n",
       "  /* Definition of color scheme common for light and dark mode */\n",
       "  --sklearn-color-text: #000;\n",
       "  --sklearn-color-text-muted: #666;\n",
       "  --sklearn-color-line: gray;\n",
       "  /* Definition of color scheme for unfitted estimators */\n",
       "  --sklearn-color-unfitted-level-0: #fff5e6;\n",
       "  --sklearn-color-unfitted-level-1: #f6e4d2;\n",
       "  --sklearn-color-unfitted-level-2: #ffe0b3;\n",
       "  --sklearn-color-unfitted-level-3: chocolate;\n",
       "  /* Definition of color scheme for fitted estimators */\n",
       "  --sklearn-color-fitted-level-0: #f0f8ff;\n",
       "  --sklearn-color-fitted-level-1: #d4ebff;\n",
       "  --sklearn-color-fitted-level-2: #b3dbfd;\n",
       "  --sklearn-color-fitted-level-3: cornflowerblue;\n",
       "\n",
       "  /* Specific color for light theme */\n",
       "  --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, white)));\n",
       "  --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, black)));\n",
       "  --sklearn-color-icon: #696969;\n",
       "\n",
       "  @media (prefers-color-scheme: dark) {\n",
       "    /* Redefinition of color scheme for dark theme */\n",
       "    --sklearn-color-text-on-default-background: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-background: var(--sg-background-color, var(--theme-background, var(--jp-layout-color0, #111)));\n",
       "    --sklearn-color-border-box: var(--sg-text-color, var(--theme-code-foreground, var(--jp-content-font-color1, white)));\n",
       "    --sklearn-color-icon: #878787;\n",
       "  }\n",
       "}\n",
       "\n",
       "#sk-container-id-46 {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 pre {\n",
       "  padding: 0;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 input.sk-hidden--visually {\n",
       "  border: 0;\n",
       "  clip: rect(1px 1px 1px 1px);\n",
       "  clip: rect(1px, 1px, 1px, 1px);\n",
       "  height: 1px;\n",
       "  margin: -1px;\n",
       "  overflow: hidden;\n",
       "  padding: 0;\n",
       "  position: absolute;\n",
       "  width: 1px;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-dashed-wrapped {\n",
       "  border: 1px dashed var(--sklearn-color-line);\n",
       "  margin: 0 0.4em 0.5em 0.4em;\n",
       "  box-sizing: border-box;\n",
       "  padding-bottom: 0.4em;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-container {\n",
       "  /* jupyter's `normalize.less` sets `[hidden] { display: none; }`\n",
       "     but bootstrap.min.css set `[hidden] { display: none !important; }`\n",
       "     so we also need the `!important` here to be able to override the\n",
       "     default hidden behavior on the sphinx rendered scikit-learn.org.\n",
       "     See: https://github.com/scikit-learn/scikit-learn/issues/21755 */\n",
       "  display: inline-block !important;\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-text-repr-fallback {\n",
       "  display: none;\n",
       "}\n",
       "\n",
       "div.sk-parallel-item,\n",
       "div.sk-serial,\n",
       "div.sk-item {\n",
       "  /* draw centered vertical line to link estimators */\n",
       "  background-image: linear-gradient(var(--sklearn-color-text-on-default-background), var(--sklearn-color-text-on-default-background));\n",
       "  background-size: 2px 100%;\n",
       "  background-repeat: no-repeat;\n",
       "  background-position: center center;\n",
       "}\n",
       "\n",
       "/* Parallel-specific style estimator block */\n",
       "\n",
       "#sk-container-id-46 div.sk-parallel-item::after {\n",
       "  content: \"\";\n",
       "  width: 100%;\n",
       "  border-bottom: 2px solid var(--sklearn-color-text-on-default-background);\n",
       "  flex-grow: 1;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-parallel {\n",
       "  display: flex;\n",
       "  align-items: stretch;\n",
       "  justify-content: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  position: relative;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-parallel-item {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-parallel-item:first-child::after {\n",
       "  align-self: flex-end;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-parallel-item:last-child::after {\n",
       "  align-self: flex-start;\n",
       "  width: 50%;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-parallel-item:only-child::after {\n",
       "  width: 0;\n",
       "}\n",
       "\n",
       "/* Serial-specific style estimator block */\n",
       "\n",
       "#sk-container-id-46 div.sk-serial {\n",
       "  display: flex;\n",
       "  flex-direction: column;\n",
       "  align-items: center;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  padding-right: 1em;\n",
       "  padding-left: 1em;\n",
       "}\n",
       "\n",
       "\n",
       "/* Toggleable style: style used for estimator/Pipeline/ColumnTransformer box that is\n",
       "clickable and can be expanded/collapsed.\n",
       "- Pipeline and ColumnTransformer use this feature and define the default style\n",
       "- Estimators will overwrite some part of the style using the `sk-estimator` class\n",
       "*/\n",
       "\n",
       "/* Pipeline and ColumnTransformer style (default) */\n",
       "\n",
       "#sk-container-id-46 div.sk-toggleable {\n",
       "  /* Default theme specific background. It is overwritten whether we have a\n",
       "  specific estimator or a Pipeline/ColumnTransformer */\n",
       "  background-color: var(--sklearn-color-background);\n",
       "}\n",
       "\n",
       "/* Toggleable label */\n",
       "#sk-container-id-46 label.sk-toggleable__label {\n",
       "  cursor: pointer;\n",
       "  display: flex;\n",
       "  width: 100%;\n",
       "  margin-bottom: 0;\n",
       "  padding: 0.5em;\n",
       "  box-sizing: border-box;\n",
       "  text-align: center;\n",
       "  align-items: start;\n",
       "  justify-content: space-between;\n",
       "  gap: 0.5em;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 label.sk-toggleable__label .caption {\n",
       "  font-size: 0.6rem;\n",
       "  font-weight: lighter;\n",
       "  color: var(--sklearn-color-text-muted);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 label.sk-toggleable__label-arrow:before {\n",
       "  /* Arrow on the left of the label */\n",
       "  content: \"▸\";\n",
       "  float: left;\n",
       "  margin-right: 0.25em;\n",
       "  color: var(--sklearn-color-icon);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 label.sk-toggleable__label-arrow:hover:before {\n",
       "  color: var(--sklearn-color-text);\n",
       "}\n",
       "\n",
       "/* Toggleable content - dropdown */\n",
       "\n",
       "#sk-container-id-46 div.sk-toggleable__content {\n",
       "  max-height: 0;\n",
       "  max-width: 0;\n",
       "  overflow: hidden;\n",
       "  text-align: left;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-toggleable__content.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-toggleable__content pre {\n",
       "  margin: 0.2em;\n",
       "  border-radius: 0.25em;\n",
       "  color: var(--sklearn-color-text);\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-toggleable__content.fitted pre {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 input.sk-toggleable__control:checked~div.sk-toggleable__content {\n",
       "  /* Expand drop-down */\n",
       "  max-height: 200px;\n",
       "  max-width: 100%;\n",
       "  overflow: auto;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 input.sk-toggleable__control:checked~label.sk-toggleable__label-arrow:before {\n",
       "  content: \"▾\";\n",
       "}\n",
       "\n",
       "/* Pipeline/ColumnTransformer-specific style */\n",
       "\n",
       "#sk-container-id-46 div.sk-label input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-label.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator-specific style */\n",
       "\n",
       "/* Colorize estimator box */\n",
       "#sk-container-id-46 div.sk-estimator input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-estimator.fitted input.sk-toggleable__control:checked~label.sk-toggleable__label {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-label label.sk-toggleable__label,\n",
       "#sk-container-id-46 div.sk-label label {\n",
       "  /* The background is the default theme color */\n",
       "  color: var(--sklearn-color-text-on-default-background);\n",
       "}\n",
       "\n",
       "/* On hover, darken the color of the background */\n",
       "#sk-container-id-46 div.sk-label:hover label.sk-toggleable__label {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "/* Label box, darken color on hover, fitted */\n",
       "#sk-container-id-46 div.sk-label.fitted:hover label.sk-toggleable__label.fitted {\n",
       "  color: var(--sklearn-color-text);\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Estimator label */\n",
       "\n",
       "#sk-container-id-46 div.sk-label label {\n",
       "  font-family: monospace;\n",
       "  font-weight: bold;\n",
       "  display: inline-block;\n",
       "  line-height: 1.2em;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-label-container {\n",
       "  text-align: center;\n",
       "}\n",
       "\n",
       "/* Estimator-specific */\n",
       "#sk-container-id-46 div.sk-estimator {\n",
       "  font-family: monospace;\n",
       "  border: 1px dotted var(--sklearn-color-border-box);\n",
       "  border-radius: 0.25em;\n",
       "  box-sizing: border-box;\n",
       "  margin-bottom: 0.5em;\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-0);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-estimator.fitted {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-0);\n",
       "}\n",
       "\n",
       "/* on hover */\n",
       "#sk-container-id-46 div.sk-estimator:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-2);\n",
       "}\n",
       "\n",
       "#sk-container-id-46 div.sk-estimator.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-2);\n",
       "}\n",
       "\n",
       "/* Specification for estimator info (e.g. \"i\" and \"?\") */\n",
       "\n",
       "/* Common style for \"i\" and \"?\" */\n",
       "\n",
       ".sk-estimator-doc-link,\n",
       "a:link.sk-estimator-doc-link,\n",
       "a:visited.sk-estimator-doc-link {\n",
       "  float: right;\n",
       "  font-size: smaller;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1em;\n",
       "  height: 1em;\n",
       "  width: 1em;\n",
       "  text-decoration: none !important;\n",
       "  margin-left: 0.5em;\n",
       "  text-align: center;\n",
       "  /* unfitted */\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted,\n",
       "a:link.sk-estimator-doc-link.fitted,\n",
       "a:visited.sk-estimator-doc-link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "div.sk-estimator:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link:hover,\n",
       ".sk-estimator-doc-link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "div.sk-estimator.fitted:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover,\n",
       "div.sk-label-container:hover .sk-estimator-doc-link.fitted:hover,\n",
       ".sk-estimator-doc-link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "/* Span, style for the box shown on hovering the info icon */\n",
       ".sk-estimator-doc-link span {\n",
       "  display: none;\n",
       "  z-index: 9999;\n",
       "  position: relative;\n",
       "  font-weight: normal;\n",
       "  right: .2ex;\n",
       "  padding: .5ex;\n",
       "  margin: .5ex;\n",
       "  width: min-content;\n",
       "  min-width: 20ex;\n",
       "  max-width: 50ex;\n",
       "  color: var(--sklearn-color-text);\n",
       "  box-shadow: 2pt 2pt 4pt #999;\n",
       "  /* unfitted */\n",
       "  background: var(--sklearn-color-unfitted-level-0);\n",
       "  border: .5pt solid var(--sklearn-color-unfitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link.fitted span {\n",
       "  /* fitted */\n",
       "  background: var(--sklearn-color-fitted-level-0);\n",
       "  border: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "\n",
       ".sk-estimator-doc-link:hover span {\n",
       "  display: block;\n",
       "}\n",
       "\n",
       "/* \"?\"-specific style due to the `<a>` HTML tag */\n",
       "\n",
       "#sk-container-id-46 a.estimator_doc_link {\n",
       "  float: right;\n",
       "  font-size: 1rem;\n",
       "  line-height: 1em;\n",
       "  font-family: monospace;\n",
       "  background-color: var(--sklearn-color-background);\n",
       "  border-radius: 1rem;\n",
       "  height: 1rem;\n",
       "  width: 1rem;\n",
       "  text-decoration: none;\n",
       "  /* unfitted */\n",
       "  color: var(--sklearn-color-unfitted-level-1);\n",
       "  border: var(--sklearn-color-unfitted-level-1) 1pt solid;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 a.estimator_doc_link.fitted {\n",
       "  /* fitted */\n",
       "  border: var(--sklearn-color-fitted-level-1) 1pt solid;\n",
       "  color: var(--sklearn-color-fitted-level-1);\n",
       "}\n",
       "\n",
       "/* On hover */\n",
       "#sk-container-id-46 a.estimator_doc_link:hover {\n",
       "  /* unfitted */\n",
       "  background-color: var(--sklearn-color-unfitted-level-3);\n",
       "  color: var(--sklearn-color-background);\n",
       "  text-decoration: none;\n",
       "}\n",
       "\n",
       "#sk-container-id-46 a.estimator_doc_link.fitted:hover {\n",
       "  /* fitted */\n",
       "  background-color: var(--sklearn-color-fitted-level-3);\n",
       "}\n",
       "</style><div id=\"sk-container-id-46\" class=\"sk-top-container\"><div class=\"sk-text-repr-fallback\"><pre>SimpleImputer(strategy=&#x27;median&#x27;)</pre><b>In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. <br />On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.</b></div><div class=\"sk-container\" hidden><div class=\"sk-item\"><div class=\"sk-estimator fitted sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-46\" type=\"checkbox\" checked><label for=\"sk-estimator-id-46\" class=\"sk-toggleable__label fitted sk-toggleable__label-arrow\"><div><div>SimpleImputer</div></div><div><a class=\"sk-estimator-doc-link fitted\" rel=\"noreferrer\" target=\"_blank\" href=\"https://scikit-learn.org/1.6/modules/generated/sklearn.impute.SimpleImputer.html\">?<span>Documentation for SimpleImputer</span></a><span class=\"sk-estimator-doc-link fitted\">i<span>Fitted</span></span></div></label><div class=\"sk-toggleable__content fitted\"><pre>SimpleImputer(strategy=&#x27;median&#x27;)</pre></div> </div></div></div></div>"
      ],
      "text/plain": [
       "SimpleImputer(strategy='median')"
      ]
     },
     "execution_count": 1282,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "imputer.fit(people_test_num)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1283,
   "id": "659d0bd8-4a67-4024-86e7-14a3a1bf82c2",
   "metadata": {},
   "outputs": [],
   "source": [
    "X = imputer.transform(people_test_num)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1284,
   "id": "36dede14-0743-4c94-802a-7efb5e67d313",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test_num = pd.DataFrame(X, columns=people_test_num.columns)\n",
    "people_test_num = people_test_num.drop('id',axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1285,
   "id": "ad1e7816-829b-4567-8271-48be468bcbdf",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>3.0</td>\n",
       "      <td>7.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>5.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>2.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>3.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>15.0</td>\n",
       "      <td>9.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>3.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>6.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>9.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>2.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6170</th>\n",
       "      <td>3.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>9.0</td>\n",
       "      <td>6.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6171</th>\n",
       "      <td>8.0</td>\n",
       "      <td>2.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6172</th>\n",
       "      <td>2.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>9.0</td>\n",
       "      <td>7.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6173</th>\n",
       "      <td>3.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>11.0</td>\n",
       "      <td>9.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6174</th>\n",
       "      <td>2.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>6175 rows × 5 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "      Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "0                  3.0                      7.0            4.0   \n",
       "1                  2.0                      0.0            0.0   \n",
       "2                  3.0                      5.0            6.0   \n",
       "3                  3.0                      4.0            4.0   \n",
       "4                  9.0                      1.0            2.0   \n",
       "...                ...                      ...            ...   \n",
       "6170               3.0                      5.0            5.0   \n",
       "6171               8.0                      2.0            1.0   \n",
       "6172               2.0                      4.0            3.0   \n",
       "6173               3.0                      4.0            4.0   \n",
       "6174               2.0                      1.0            1.0   \n",
       "\n",
       "      Friends_circle_size  Post_frequency  \n",
       "0                     6.0             5.0  \n",
       "1                     5.0             1.0  \n",
       "2                    15.0             9.0  \n",
       "3                     5.0             6.0  \n",
       "4                     1.0             1.0  \n",
       "...                   ...             ...  \n",
       "6170                  9.0             6.0  \n",
       "6171                  0.0             0.0  \n",
       "6172                  9.0             7.0  \n",
       "6173                 11.0             9.0  \n",
       "6174                  1.0             0.0  \n",
       "\n",
       "[6175 rows x 5 columns]"
      ]
     },
     "execution_count": 1285,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test_num"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1286,
   "id": "d82eabd1-d4c3-4a7b-86e6-2508fbd9635c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Time_spent_Alone           0\n",
       "Social_event_attendance    0\n",
       "Going_outside              0\n",
       "Friends_circle_size        0\n",
       "Post_frequency             0\n",
       "dtype: int64"
      ]
     },
     "execution_count": 1286,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test_num.isna().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1287,
   "id": "218d6e5e-daa2-4df2-8e18-7d933a8601a8",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Stage_fear</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Drained_after_socializing</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>18524</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>7.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>No</td>\n",
       "      <td>6.0</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>18525</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Yes</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>5.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>18526</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>5.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>No</td>\n",
       "      <td>15.0</td>\n",
       "      <td>9.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>18527</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>No</td>\n",
       "      <td>5.0</td>\n",
       "      <td>6.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>18528</td>\n",
       "      <td>9.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>2.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6170</th>\n",
       "      <td>24694</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>5.0</td>\n",
       "      <td>5.0</td>\n",
       "      <td>No</td>\n",
       "      <td>9.0</td>\n",
       "      <td>6.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6171</th>\n",
       "      <td>24695</td>\n",
       "      <td>8.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>2.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6172</th>\n",
       "      <td>24696</td>\n",
       "      <td>2.0</td>\n",
       "      <td>No</td>\n",
       "      <td>4.0</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>9.0</td>\n",
       "      <td>7.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6173</th>\n",
       "      <td>24697</td>\n",
       "      <td>3.0</td>\n",
       "      <td>No</td>\n",
       "      <td>4.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>No</td>\n",
       "      <td>11.0</td>\n",
       "      <td>9.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6174</th>\n",
       "      <td>24698</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>6175 rows × 8 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "         id  Time_spent_Alone Stage_fear  Social_event_attendance  \\\n",
       "0     18524               3.0         No                      7.0   \n",
       "1     18525               NaN        Yes                      0.0   \n",
       "2     18526               3.0         No                      5.0   \n",
       "3     18527               3.0         No                      4.0   \n",
       "4     18528               9.0        Yes                      1.0   \n",
       "...     ...               ...        ...                      ...   \n",
       "6170  24694               3.0         No                      5.0   \n",
       "6171  24695               8.0        Yes                      2.0   \n",
       "6172  24696               2.0         No                      4.0   \n",
       "6173  24697               3.0         No                      4.0   \n",
       "6174  24698               NaN        Yes                      1.0   \n",
       "\n",
       "      Going_outside Drained_after_socializing  Friends_circle_size  \\\n",
       "0               4.0                        No                  6.0   \n",
       "1               0.0                       Yes                  5.0   \n",
       "2               6.0                        No                 15.0   \n",
       "3               4.0                        No                  5.0   \n",
       "4               2.0                       Yes                  1.0   \n",
       "...             ...                       ...                  ...   \n",
       "6170            5.0                        No                  9.0   \n",
       "6171            1.0                       Yes                  0.0   \n",
       "6172            3.0                        No                  9.0   \n",
       "6173            4.0                        No                 11.0   \n",
       "6174            1.0                       Yes                  1.0   \n",
       "\n",
       "      Post_frequency  \n",
       "0                NaN  \n",
       "1                1.0  \n",
       "2                9.0  \n",
       "3                6.0  \n",
       "4                1.0  \n",
       "...              ...  \n",
       "6170             6.0  \n",
       "6171             0.0  \n",
       "6172             7.0  \n",
       "6173             9.0  \n",
       "6174             0.0  \n",
       "\n",
       "[6175 rows x 8 columns]"
      ]
     },
     "execution_count": 1287,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1288,
   "id": "bcb17d9d-9404-4865-b62f-4bf654176df4",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test_cat = people_test[['Stage_fear','Drained_after_socializing']]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1289,
   "id": "a3d26f9c-fa5a-48af-9f47-2bd318eb55b8",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test_cat = people_test_cat.fillna(people_test_cat.mode().iloc[0])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1290,
   "id": "321e4999-c8da-4539-9c4f-a8a4b3ff134d",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Stage_fear                   0\n",
       "Drained_after_socializing    0\n",
       "dtype: int64"
      ]
     },
     "execution_count": 1290,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test_cat.isna().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1291,
   "id": "1f06cf95-baaa-496f-92af-f8ae00aa61cc",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Stage_fear</th>\n",
       "      <th>Drained_after_socializing</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6170</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6171</th>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6172</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6173</th>\n",
       "      <td>No</td>\n",
       "      <td>No</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6174</th>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>6175 rows × 2 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "     Stage_fear Drained_after_socializing\n",
       "0            No                        No\n",
       "1           Yes                       Yes\n",
       "2            No                        No\n",
       "3            No                        No\n",
       "4           Yes                       Yes\n",
       "...         ...                       ...\n",
       "6170         No                        No\n",
       "6171        Yes                       Yes\n",
       "6172         No                        No\n",
       "6173         No                        No\n",
       "6174        Yes                       Yes\n",
       "\n",
       "[6175 rows x 2 columns]"
      ]
     },
     "execution_count": 1291,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test_cat"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1292,
   "id": "4e498f1f-7463-4db3-b0e0-4a594122abba",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test_num_std_scaled = std_scaler.fit_transform(people_test_num)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1293,
   "id": "b6ecfd16-07a2-4a9a-ba94-6425b72ff224",
   "metadata": {},
   "outputs": [],
   "source": [
    "cat_encoder = OneHotEncoder()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1294,
   "id": "2aaeb913-4236-4093-bcbb-fea15df6e5fc",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test_cat_1hot = cat_encoder.fit_transform(people_test_cat)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1295,
   "id": "8021a6fd-d3f9-4bd1-aaf0-f305302cd7dd",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[1., 0., 1., 0.],\n",
       "       [0., 1., 0., 1.],\n",
       "       [1., 0., 1., 0.],\n",
       "       ...,\n",
       "       [1., 0., 1., 0.],\n",
       "       [1., 0., 1., 0.],\n",
       "       [0., 1., 0., 1.]])"
      ]
     },
     "execution_count": 1295,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test_cat_1hot.toarray()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1296,
   "id": "91c05eae-4e2a-4cdd-8d76-b38f566d7b01",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[-0.01381849,  0.64847139, -0.01778896, -0.49313243, -0.00976086],\n",
       "       [-0.35928072, -1.97460328, -2.05198894, -0.73872423, -1.45343363],\n",
       "       [-0.01381849, -0.10097851,  0.99931103,  1.71719378,  1.4339119 ],\n",
       "       ...,\n",
       "       [-0.35928072, -0.47570347, -0.52633895,  0.24364298,  0.71207552],\n",
       "       [-0.01381849, -0.47570347, -0.01778896,  0.73482658,  1.4339119 ],\n",
       "       [-0.35928072, -1.59987833, -1.54343894, -1.72109143, -1.81435182]])"
      ]
     },
     "execution_count": 1296,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test_num_std_scaled"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1297,
   "id": "fa4426f7-87c9-4bd3-b120-0c6432aa23b0",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_people_test_num = pd.DataFrame(people_test_num_std_scaled,columns=std_scaler.get_feature_names_out(),index=people_test.id)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1298,
   "id": "8e44030f-2c4e-4b1c-b060-41ab2908176d",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>id</th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>18524</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>0.648471</td>\n",
       "      <td>-0.017789</td>\n",
       "      <td>-0.493132</td>\n",
       "      <td>-0.009761</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18525</th>\n",
       "      <td>-0.359281</td>\n",
       "      <td>-1.974603</td>\n",
       "      <td>-2.051989</td>\n",
       "      <td>-0.738724</td>\n",
       "      <td>-1.453434</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18526</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.100979</td>\n",
       "      <td>0.999311</td>\n",
       "      <td>1.717194</td>\n",
       "      <td>1.433912</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18527</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.475703</td>\n",
       "      <td>-0.017789</td>\n",
       "      <td>-0.738724</td>\n",
       "      <td>0.351157</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18528</th>\n",
       "      <td>2.058955</td>\n",
       "      <td>-1.599878</td>\n",
       "      <td>-1.034889</td>\n",
       "      <td>-1.721091</td>\n",
       "      <td>-1.453434</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24694</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.100979</td>\n",
       "      <td>0.490761</td>\n",
       "      <td>0.243643</td>\n",
       "      <td>0.351157</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24695</th>\n",
       "      <td>1.713493</td>\n",
       "      <td>-1.225153</td>\n",
       "      <td>-1.543439</td>\n",
       "      <td>-1.966683</td>\n",
       "      <td>-1.814352</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24696</th>\n",
       "      <td>-0.359281</td>\n",
       "      <td>-0.475703</td>\n",
       "      <td>-0.526339</td>\n",
       "      <td>0.243643</td>\n",
       "      <td>0.712076</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24697</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.475703</td>\n",
       "      <td>-0.017789</td>\n",
       "      <td>0.734827</td>\n",
       "      <td>1.433912</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24698</th>\n",
       "      <td>-0.359281</td>\n",
       "      <td>-1.599878</td>\n",
       "      <td>-1.543439</td>\n",
       "      <td>-1.721091</td>\n",
       "      <td>-1.814352</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>6175 rows × 5 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "       Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "id                                                                \n",
       "18524         -0.013818                 0.648471      -0.017789   \n",
       "18525         -0.359281                -1.974603      -2.051989   \n",
       "18526         -0.013818                -0.100979       0.999311   \n",
       "18527         -0.013818                -0.475703      -0.017789   \n",
       "18528          2.058955                -1.599878      -1.034889   \n",
       "...                 ...                      ...            ...   \n",
       "24694         -0.013818                -0.100979       0.490761   \n",
       "24695          1.713493                -1.225153      -1.543439   \n",
       "24696         -0.359281                -0.475703      -0.526339   \n",
       "24697         -0.013818                -0.475703      -0.017789   \n",
       "24698         -0.359281                -1.599878      -1.543439   \n",
       "\n",
       "       Friends_circle_size  Post_frequency  \n",
       "id                                          \n",
       "18524            -0.493132       -0.009761  \n",
       "18525            -0.738724       -1.453434  \n",
       "18526             1.717194        1.433912  \n",
       "18527            -0.738724        0.351157  \n",
       "18528            -1.721091       -1.453434  \n",
       "...                    ...             ...  \n",
       "24694             0.243643        0.351157  \n",
       "24695            -1.966683       -1.814352  \n",
       "24696             0.243643        0.712076  \n",
       "24697             0.734827        1.433912  \n",
       "24698            -1.721091       -1.814352  \n",
       "\n",
       "[6175 rows x 5 columns]"
      ]
     },
     "execution_count": 1298,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_people_test_num"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1299,
   "id": "5300aed7-ad6d-40f9-92d0-cd269e0302f0",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_people_test_cat = pd.DataFrame(people_test_cat_1hot.toarray(),index=people_test.id)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1300,
   "id": "00a2965e-2b67-4638-9cac-2a86e8efb17b",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>id</th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>18524</th>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18525</th>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18526</th>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18527</th>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18528</th>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24694</th>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24695</th>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24696</th>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24697</th>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24698</th>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>6175 rows × 4 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "         0    1    2    3\n",
       "id                       \n",
       "18524  1.0  0.0  1.0  0.0\n",
       "18525  0.0  1.0  0.0  1.0\n",
       "18526  1.0  0.0  1.0  0.0\n",
       "18527  1.0  0.0  1.0  0.0\n",
       "18528  0.0  1.0  0.0  1.0\n",
       "...    ...  ...  ...  ...\n",
       "24694  1.0  0.0  1.0  0.0\n",
       "24695  0.0  1.0  0.0  1.0\n",
       "24696  1.0  0.0  1.0  0.0\n",
       "24697  1.0  0.0  1.0  0.0\n",
       "24698  0.0  1.0  0.0  1.0\n",
       "\n",
       "[6175 rows x 4 columns]"
      ]
     },
     "execution_count": 1300,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_people_test_cat"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1301,
   "id": "1f8cd37c-1d25-4adc-9d29-d7707038d5ba",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test_data = pd.merge(df_people_test_num,df_people_test_cat, on='id')\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1302,
   "id": "9ab327ff-97ed-4469-96a7-e0adb588949f",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>id</th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>18524</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>0.648471</td>\n",
       "      <td>-0.017789</td>\n",
       "      <td>-0.493132</td>\n",
       "      <td>-0.009761</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18525</th>\n",
       "      <td>-0.359281</td>\n",
       "      <td>-1.974603</td>\n",
       "      <td>-2.051989</td>\n",
       "      <td>-0.738724</td>\n",
       "      <td>-1.453434</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18526</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.100979</td>\n",
       "      <td>0.999311</td>\n",
       "      <td>1.717194</td>\n",
       "      <td>1.433912</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18527</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.475703</td>\n",
       "      <td>-0.017789</td>\n",
       "      <td>-0.738724</td>\n",
       "      <td>0.351157</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18528</th>\n",
       "      <td>2.058955</td>\n",
       "      <td>-1.599878</td>\n",
       "      <td>-1.034889</td>\n",
       "      <td>-1.721091</td>\n",
       "      <td>-1.453434</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24694</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.100979</td>\n",
       "      <td>0.490761</td>\n",
       "      <td>0.243643</td>\n",
       "      <td>0.351157</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24695</th>\n",
       "      <td>1.713493</td>\n",
       "      <td>-1.225153</td>\n",
       "      <td>-1.543439</td>\n",
       "      <td>-1.966683</td>\n",
       "      <td>-1.814352</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24696</th>\n",
       "      <td>-0.359281</td>\n",
       "      <td>-0.475703</td>\n",
       "      <td>-0.526339</td>\n",
       "      <td>0.243643</td>\n",
       "      <td>0.712076</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24697</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.475703</td>\n",
       "      <td>-0.017789</td>\n",
       "      <td>0.734827</td>\n",
       "      <td>1.433912</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24698</th>\n",
       "      <td>-0.359281</td>\n",
       "      <td>-1.599878</td>\n",
       "      <td>-1.543439</td>\n",
       "      <td>-1.721091</td>\n",
       "      <td>-1.814352</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>6175 rows × 9 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "       Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "id                                                                \n",
       "18524         -0.013818                 0.648471      -0.017789   \n",
       "18525         -0.359281                -1.974603      -2.051989   \n",
       "18526         -0.013818                -0.100979       0.999311   \n",
       "18527         -0.013818                -0.475703      -0.017789   \n",
       "18528          2.058955                -1.599878      -1.034889   \n",
       "...                 ...                      ...            ...   \n",
       "24694         -0.013818                -0.100979       0.490761   \n",
       "24695          1.713493                -1.225153      -1.543439   \n",
       "24696         -0.359281                -0.475703      -0.526339   \n",
       "24697         -0.013818                -0.475703      -0.017789   \n",
       "24698         -0.359281                -1.599878      -1.543439   \n",
       "\n",
       "       Friends_circle_size  Post_frequency    0    1    2    3  \n",
       "id                                                              \n",
       "18524            -0.493132       -0.009761  1.0  0.0  1.0  0.0  \n",
       "18525            -0.738724       -1.453434  0.0  1.0  0.0  1.0  \n",
       "18526             1.717194        1.433912  1.0  0.0  1.0  0.0  \n",
       "18527            -0.738724        0.351157  1.0  0.0  1.0  0.0  \n",
       "18528            -1.721091       -1.453434  0.0  1.0  0.0  1.0  \n",
       "...                    ...             ...  ...  ...  ...  ...  \n",
       "24694             0.243643        0.351157  1.0  0.0  1.0  0.0  \n",
       "24695            -1.966683       -1.814352  0.0  1.0  0.0  1.0  \n",
       "24696             0.243643        0.712076  1.0  0.0  1.0  0.0  \n",
       "24697             0.734827        1.433912  1.0  0.0  1.0  0.0  \n",
       "24698            -1.721091       -1.814352  0.0  1.0  0.0  1.0  \n",
       "\n",
       "[6175 rows x 9 columns]"
      ]
     },
     "execution_count": 1302,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "pd.DataFrame(people_test_data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1303,
   "id": "ea4b3996-9aa4-46be-abac-1d4947732204",
   "metadata": {},
   "outputs": [],
   "source": [
    "people_test_data.columns = people_test_data.columns.astype(str)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "8c338f2b-a008-42c5-b59f-a0435b40f85d",
   "metadata": {},
   "source": [
    "<h2>Predict</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1304,
   "id": "425fe836-9a96-4ea2-af28-e264ed1e93b6",
   "metadata": {},
   "outputs": [],
   "source": [
    "y_test_pred_forest = forest_clf.predict(people_test_data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1305,
   "id": "ab1b4fd4-4ca6-4c98-bc60-88fb0188f3c5",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Time_spent_Alone</th>\n",
       "      <th>Social_event_attendance</th>\n",
       "      <th>Going_outside</th>\n",
       "      <th>Friends_circle_size</th>\n",
       "      <th>Post_frequency</th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>id</th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>18524</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>0.648471</td>\n",
       "      <td>-0.017789</td>\n",
       "      <td>-0.493132</td>\n",
       "      <td>-0.009761</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18525</th>\n",
       "      <td>-0.359281</td>\n",
       "      <td>-1.974603</td>\n",
       "      <td>-2.051989</td>\n",
       "      <td>-0.738724</td>\n",
       "      <td>-1.453434</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18526</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.100979</td>\n",
       "      <td>0.999311</td>\n",
       "      <td>1.717194</td>\n",
       "      <td>1.433912</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18527</th>\n",
       "      <td>-0.013818</td>\n",
       "      <td>-0.475703</td>\n",
       "      <td>-0.017789</td>\n",
       "      <td>-0.738724</td>\n",
       "      <td>0.351157</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18528</th>\n",
       "      <td>2.058955</td>\n",
       "      <td>-1.599878</td>\n",
       "      <td>-1.034889</td>\n",
       "      <td>-1.721091</td>\n",
       "      <td>-1.453434</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "       Time_spent_Alone  Social_event_attendance  Going_outside  \\\n",
       "id                                                                \n",
       "18524         -0.013818                 0.648471      -0.017789   \n",
       "18525         -0.359281                -1.974603      -2.051989   \n",
       "18526         -0.013818                -0.100979       0.999311   \n",
       "18527         -0.013818                -0.475703      -0.017789   \n",
       "18528          2.058955                -1.599878      -1.034889   \n",
       "\n",
       "       Friends_circle_size  Post_frequency    0    1    2    3  \n",
       "id                                                              \n",
       "18524            -0.493132       -0.009761  1.0  0.0  1.0  0.0  \n",
       "18525            -0.738724       -1.453434  0.0  1.0  0.0  1.0  \n",
       "18526             1.717194        1.433912  1.0  0.0  1.0  0.0  \n",
       "18527            -0.738724        0.351157  1.0  0.0  1.0  0.0  \n",
       "18528            -1.721091       -1.453434  0.0  1.0  0.0  1.0  "
      ]
     },
     "execution_count": 1305,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "people_test_data.head()\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1306,
   "id": "7f6ad50f-fcd1-4efe-8250-73ce267217f0",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_results = pd.DataFrame(y_test_pred_forest,columns=['Personality'],index=people_test_data.index)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1307,
   "id": "6f8f5dbf-f6f1-4f6a-ad84-be094c9623a5",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Personality</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>id</th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>18524</th>\n",
       "      <td>Extrovert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18525</th>\n",
       "      <td>Introvert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18526</th>\n",
       "      <td>Extrovert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18527</th>\n",
       "      <td>Extrovert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18528</th>\n",
       "      <td>Introvert</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "      Personality\n",
       "id               \n",
       "18524   Extrovert\n",
       "18525   Introvert\n",
       "18526   Extrovert\n",
       "18527   Extrovert\n",
       "18528   Introvert"
      ]
     },
     "execution_count": 1307,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_results.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "98984556-6e82-4ddf-a3d3-487268020e5c",
   "metadata": {},
   "source": [
    "<h2>Create submission</h2>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1308,
   "id": "2e20f9dc-4cde-48fa-802a-e7103f4b273c",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "      Personality\n",
      "id               \n",
      "18524   Extrovert\n",
      "18525   Introvert\n",
      "18526   Extrovert\n",
      "18527   Extrovert\n",
      "18528   Introvert\n",
      "Submitted successfully with Random Forest\n"
     ]
    }
   ],
   "source": [
    "# \n",
    "submission = df_results\n",
    "print(submission.head())\n",
    "submission.to_csv('dataset/submission.csv')\n",
    "print(\"Submitted successfully with Random Forest\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.13"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}